In [60]:
# Euclid algorithm
# input: two integers a, b
# output: gcd(a, b) 
def euclid(a, b):
    a, b = abs(a), abs(b)
    while b != 0: 
        a, b = b, a % b
    return a 

In [61]:
# Extended Euclid Algorithm
# input: Two integers a, b
# output: d=gcd(a, b) and two integers x, y satifying ax+by = d
import numpy as np

def ex_euclid(a, b):
    a0, b0 = abs(a), abs(b)
    x0, x1 = 1, 0
    y0, y1 = 0, 1
    while b0 != 0:
        r = a0 % b0
        q = (a0-r)//b0
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
        a0, b0 = b0, r
    return [a0, np.sign(a)*x0, np.sign(b)*y0]

In [62]:
# Solve ax≡1 (mod. n) ( inversion of mod n)
# input: integers a, n (n > 0)
# output: integer x satifying ax≡1 (mod. n) (x>0)
import sys # 
def inv(a, n):
    if euclid(a, n) > 1:
        sys.stderr.write('gcd(a, n) is not 1') # there is nosolution if gcd(a, n)>1
        return None
    _, x, _ = ex_euclid(a, n)
    return x % n

In [63]:
# mod binary method
# input: integers g, k, n (k, n > 0)
# output: g^k (mod. n)
def mod_binary(g,k,n):
    bk = bin(k)[2:]
    y = 1
    for i in bk:
        y = (y*y) % n
        if i == '1':
            y = (y*g) % n 
    return y

In [64]:
# Fermat test
# input: integers n, a (gcd(a, n)=1)
# output: True or False
import random
def fermat_test(a, n):
    return mod_binary(a, n-1, n) == 1

In [65]:
# Prime number generator
# input : iv, k (iv: initial value)
# output : prime number equal or larger than iv
def prime_gen(iv):
    k = 5
    p = iv
    if p == 2:
        return p
    if p % 2 == 0:
        p += 1
    while True:
        A = []
        for i in range(k):
            a = random.randrange(2, p)
            while a in A:
                a = random.randrange(2, p)
            A.append(a)
            if euclid(a, p) != 1:
                break
            if fermat_test(a, p) == False:
                break
            if i == k - 1:
                return p
        p += 2

In [66]:
# 絶対値最小剰余の計算
# Input：整数 𝑎, 𝑛 (𝑛>0)
# Output：𝑎 を 𝑛 で割った時の絶対値最小剰余
def mod(a, n):
    r = a % n
    if r > n/2:
        return r-n # 戻り値を指定
    else:
        return r # 戻り値を指定

In [67]:
# 数論変換（NTT）
# Input: 多項式の係数のリスト 𝑓, 法 𝑝, w_list =[𝑤𝑁^𝑖∣ 0 ≤ 𝑖 < 𝑁] (wN: Fp の1の原始N乗根) (N=2^k)
# Output: fの数論変換 𝑓^ の係数のリスト

def ntt(f, w_list, p):
    if len(f) == 1:
        return f
    # f を f0 (偶数次数パート)と f1 (奇数次数パート) に分ける
    f0_ntt = ntt(f[::2], w_list[::2], p)
    f1_ntt = ntt(f[1::2], w_list[::2], p)
    # 残りを実装
    return [mod(f0_ntt[i % len(f0_ntt)] + w_list[i]*f1_ntt[i % len(f1_ntt)], p) for i in range(len(w_list))]
    # y = [0] * len(f)
    # for i in range(len(w_list)):
    #     y[i] = mod(f0_ntt[i] + w_list[0] * f1_ntt[i], p)
    #     y[i+len(f)//2] = mod(f0_ntt[i] - w_list[0] * f1_ntt[i], p)

    # return y

In [68]:
# 数論逆変換（inverse NTT (INTT))
# 入力: 数論変換した多項式の係数のリスト 𝑓_hat, 法 𝑝, w_list = [𝑤𝑁^(-i) | 0 ≤ 𝑖 < 𝑁] (wN: Fp の1の原始N乗根) , 𝑁, ninv: n^(-1) (mod n)
# 出力: 𝑓 の係数のリスト
import sympy as sp
def intt(f_hat, w_list, p, ninv, N):
    if len(f_hat) == 1:
        return f_hat
    f0_intt = intt(f_hat[::2], w_list[::2], p, ninv, N)
    f1_intt = intt(f_hat[1::2], w_list[::2], p, ninv, N)
    # 残りを実装
    f_intt = [0 for i in range(len(w_list))]
    for i in range(len(w_list)):
        f_intt[i] += mod(f0_intt[i % len(f0_intt)] + w_list[i]*f1_intt[i % len(f1_intt)], p)
    if N == len(w_list):
        return [mod(f_intt[i] * ninv, p) for i in range(N)]
    else:
        return f_intt
    # return [mod(f0_intt[i % len(f0_intt)] + w_list[i]*f1_intt[i % len(f1_intt)], p) for i in range(len(w_list))]


    # y = [0] * len(f_hat)
    # for i in range(len(w_list)):
    #     y[i] = (f0_intt[i] + w_list[i] * f1_intt[i]) % p
    #     y[i+len(f_hat)//2] = (f0_intt[i] - w_list[i] * f1_intt[i]) % p
    # return 

In [69]:
p = 337
N = 8
w_list = [pow(85,i,p) for i in range(N)]
f = [2,1,2,5,0,0,0,0]
f_hat = ntt(f, w_list, p)

print(f_hat)

f = intt(f_hat, w_list, p, inv(N, p), N)
print(f)



[10, -73, 82, -95, -2, -5, -82, -156]
[2, 0, 0, 0, 0, 5, 2, 1]


In [70]:
# 数論変換を利用した有限体上の多項式の乗算
# Input: f, g: 多項式f, gの係数のリスト, k(poly_red()の入力), N, Ninv (N*Ninv≡1 (mod p)): 自然数, 
#        w_list: [𝑤𝑁^𝑖∣ 0 ≤ 𝑖 < 𝑁](wN: Fpの1の原始N乗根, 
#        w_inv_list:　[𝑤𝑁^(-i) | 0 ≤ 𝑖 < 𝑁] (wN: Fp の1の原始N乗根), p: 法（素数）
# Output: fgの係数のリスト

def ntt_poly_mult(f, g, kn, N, Ninv, w_list, w_inv_list, p):
    f_hat = ntt(f, w_list, p)
    g_hat = ntt(g, w_list, p)
    fg_hat = [mod(f_hat[i] * g_hat[i], p) for i in range(N)]
    ans = intt(fg_hat, w_inv_list, p, Ninv, N)
    return poly_red(ans, kn, p)

In [71]:
# 絶対値最小剰余の計算
# Input：整数 𝑎, 𝑛 (𝑛>0)
# Output：𝑎 を 𝑛 で割った時の絶対値最小剰余
def mod(a, n):
    r = a % n
    if r > n/2:
        return r-n # 戻り値を指定
    else:
        return r # 戻り値を指定

In [72]:
# ランダムな多項式の生成
# Input：𝑘, 𝑝
# Output： ℤ/𝑝ℤ[𝑋]/⟨Φ_2n ⟩ (𝑛=2^𝑘 ) の一様ランダムな多項式の係数のリスト
#   (係数の範囲が (−𝑝/2,𝑝/2]内の整数であり，次数が 𝑛 未満の多項式の係数のリストを出力する）

import random, math
def random_poly(k, p):
    N = 1<<k # N=2^k (2のk乗)
    return [random.randint(int(-p/2)+1, int(p//2)) for _ in range(N)] # リスト内包表記
    # 残りを実装

In [73]:
# 多項式のリダクション
# Input：𝑘, 𝑝, 多項式の係数のリスト
# Output：入力多項式と⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_red(f, k, p):
    N =1<<k # N=2^k (2のk乗)
    if N > len(f)-1: # N=2^k > deg(f) なら係数だけを mod して出力
        return [mod(f[i], p) for i in range(len(f))]
    else:
        q, r = divmod(len(f)-1, N) # def(f) = len(f)-1. deg(f) を N=2^k で割った時の商 q と余り r を取得
        # fの係数 f[i] を更新する 
        while len(f) -N > 0:
            f[-N-1] = mod(f[-N-1] - f[-1], p)
            f = f[:-1]
        
        return [mod(f[i], p) for i in range(N)] # 更新した f[i] に mod関数を適用

In [74]:
# 2つの多項式の加法
# 入力：𝑘, 𝑝, 多項式 𝑓, 𝑔 の係数のリスト
# 出力： 𝑓+𝑔 と ⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_add(f, g, k, p):
    add = [] # f+g の係数を入れるためのリストを準備
    if len(f) < len(g): # 係数のリストに0を足して、見かけ上はf, gの次数を等しくする
        for i in range(len(f)-len(g)):
            f.append(0)
    elif len(f) > len(g):
        for i in range(len(f)-len(g)):
            g.append(0)
    for i in range(len(f)):
        add.append((f[i]+g[i])%p)
             # g の次数 i の係数 g[i] を f の次数 i の係数に足して mod 計算 
    return poly_red(add, k, p)

In [75]:
# 2つの多項式の乗法
# 入力：𝑘, 𝑝, 多項式 𝑓, 𝑔 の係数のリスト
# 出力： 𝑓𝑔 と ⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_mult(f, g, k, p):
    if len(f) < len(g): # deg(f) < deg(g)=> fとgを入れ替える（簡単化のため deg(f)≧deg(g)とする. fg = gf より入れ替えても問題ない）
        # deg(f) < deg(g)=> fとgを入れ替える操作を追加（簡単化のため deg(f)≧deg(g)とする. fg = gf より入れ替えても問題ない）
        tmp = f
        f = g
        g = tmp
    mult = [] # fg の係数を入れるためのリストを準備
    for d in range(len(g)): # 次数が0～deg(g)までの係数を計算
        coef = 0
        for i in range(d+1):
            coef += f[i]*g[d-i]
        mult.append(mod(coef, p))
            # 係数を計算して mod して mult に格納
    
    for d in range(len(g), len(f)): # 次数がdeg(g)+1～deg(f)までの係数を計算(deg(f)=deg(g)の時は実行されない)
        coef = 0
        for i in range(len(g)):
            coef += f[d-i]*g[i]
        mult.append(mod(coef, p))
            # 係数を計算して mod して mult に格納
    
    # 次数が deg(f)+1～deg(fg)までの係数を計算
    for d in range(len(f), len(f) + len(g) - 1): # deg(f) = len(f)-1, deg(g) = len(g)-1 => deg(fg) = deg(f)+deg(g)=len(f)+len(g)-2
        coef = 0
        for i in range(d-len(f)+1, len(g)): # d-len(f)+1=d-(len(f)-1)=d-deg(f)
            # 係数を計算して mod して mult に格納
            coef += f[d-i]*g[i]
        mult.append(mod(coef, p))
    
    return poly_red(mult, k, p)

In [76]:
# 数論変換のテスト(パラメータ)
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
#wN = pow(wN, 2**4, p)
F = []
G = []
for i in range(100):
    F.append(random_poly(k, p))

for i in range(100):
    G.append(random_poly(k, p))

In [77]:
# 数論変換のテスト(結果)
N=1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]

for i in range(10):
    fg=poly_mult(F[i], G[i], k, p)
    fg_ntt = ntt_poly_mult(F[i], G[i], k, N, Ninv, w_list, w_inv_list, p)
    print("fg=fg_ntt?",fg==fg_ntt)

fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True
fg=fg_ntt? True


In [78]:
# 数論変換の効果検証
import time
t0 = time.perf_counter()
for i in range(100):
    fg = poly_mult(F[i], G[i], k, p)
t = time.perf_counter() - t0
print("通常の多項式乗算の平均時間=", t/100)

N=1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]

t0 = time.perf_counter()
for i in range(100):
    fg_ntt = ntt_poly_mult(F[i], G[i], k, N, Ninv, w_list, w_inv_list, p)
t = time.perf_counter() - t0
print("NTTによる多項式乗算の平均時間=", t/100)

通常の多項式乗算の平均時間= 0.16283875399996758
NTTによる多項式乗算の平均時間= 0.055048945999878925


In [79]:
# bitdecomp
# Input: 整数値からなるリスト Num_list, 正の整数 l
# Output: Num_listの各要素の2進展開のl-1次までの係数のリスト

def bitdecomp(Num_list, l):
    ans = []
    for i in range(len(Num_list)):
        q, r = divmod(Num_list[i], 2)
        ans.append(r)
        count = 1
        while count < l:
            q, r = divmod(q, 2)
            ans.append(r)
            count += 1
    return ans

In [80]:
# bitdecomp_inv
# Input: 正の整数 l, n, l*n個の整数値からなるリスト Num_list
# Output: n個の整数値のリスト（bitdecomp_inv(bitdecomp(num_list, l), n, l) = num_list)

def bitdecomp_inv(Num_list, n, l):
    ans = []
    for i in range(n):
        add = 0
        for j in range(l):
            add = add + Num_list[i*l+j]*pow(2,j)
        ans.append(add)
    return ans

In [81]:
# flatten
# Input: 正の整数 l, l*n個の整数値からなるリスト a
# Output: bitdecomp(bitdecomp_inv(a, n, l), l)

def flatten(a, n, l):
    return bitdecomp(bitdecomp_inv(a, n, l), l)

In [82]:
# powerof2
# Input: 正の整数 l, 整数値からなるリスト a
# Output: l*#a 個の整数値からなるリスト

def powerof2(a, l):
    return [pow(2,j)*a[i] for i in range(len(a)) for j in range(l)]

In [83]:
# 行列の掛け算 (行列とベクトルの積は計算不可)
# Input: 行列(リストのリスト)M1, M2, 法q
# Output: M1*M2 (mod q)

def matrix_mult(M1, M2, q):
    Mult = []
    for i in range(len(M1)):
        mult_row = []
        for j in range(len(M2[0])):
            sum = 0
            for k in range(len(M2)): # C1*C2の計算
                sum += M1[i][k]*M2[k][j]
            mult_row.append(sum % q)
        Mult.append(mult_row)
    
    return Mult

In [84]:
# 行列とベクトルの掛け算
# Input: 行列(リストのリスト)M, ベクトル(リスト)vec, 法q
# Output: M*vec (mod q)
import numpy as np
def matrix_vec_mult(M, vec, q):
    if len(M[0])==len(vec):
        return [np.inner(np.array(M[i]),np.array(vec)) % q for i in range(len(M))]
    else:
        return None

In [85]:
# GSW方式鍵生成
# Input: 法 q, 正の整数n, m, n個のランダムな整数値（0～q-1）のリスト t, m個のランダムな小さい整数値（エラー）のリスト e, 
#        ランダムなm×n行列(リストのリスト) B
# Output: GSW方式の秘密鍵 sk, 公開鍵 A

def GSW_keygen(q, n, m, t, e, B):
    s = [1]
    for t_i in t:
        s.append((-t_i)%q)
    b = (np.array(B) @ np.array(t) + np.array(e)).tolist()
    A = [[b[i]%q] + B[i] for i in range(m)]
    # B = np.array(B).transpose().tolist()
    # A = np.array(B + b.tolist()).transpose().tolist()
    # A =[[A[i][j]%q for j in range(n)] for i in range(m)]
    return s, A
    # b = [[B[i][j]*t[j] + e[] for j in range(n)] for i in range(m)]


In [86]:
# GSW方式暗号化
# Input: 平文 mu, 公開鍵 A, 0, 1からなるランダムな行列（リスト）R, 
#        法 q, 正の整数n, m, l, N
# Output: GSW方式の暗号文 

def GSW_enc(mu, A, R, q, n, m, l, N):
    print("R.shape = ", np.array(R).shape)
    print("A.shape = ", np.array(A).shape)
    RA = matrix_mult(R, A, q)
    RA = [bitdecomp(RA[i], l) for i in range(N)]
    RA = (mu*np.identity(N) + np.array(RA)).tolist()
    C = [flatten(RA[i], n+1, l) for i in range(N)]
    return C

In [87]:
# GSW方式復号
# GSW方式暗号化
# Input: 暗号文 C, 秘密鍵 sk, 正の整数 l, 法 q 
# Output: Cに対応する平文

def GSW_dec(C, sk, l, q):
    v = powerof2(sk, l)
    i = 0
    while v[i] <= q//4 or v[i] > q//2:
        i += 1
    xi = np.inner(np.array(C[i]), np.array(v)) % q
    return round(xi/v[i])

In [88]:
# GSW方式復号2 (1より大きい平文の復号)
# GSW方式暗号化
# Input: 暗号文 C, 秘密鍵 sk, 正の整数 l, 法 q 
# Output: Cに対応する平文

def GSW_dec2(C, sk, l, q):
    v = powerof2(sk, l)
    vi = 1<<(l-2)

    i = l-2
    xi = np.inner(np.array(C[i]), np.array(v)) % q
    mu_list = []
    while len(mu_list) < l-1:
        mu_list.append(round(xi/vi))
        i -= 1
        if i >= 0:
            mu_prev = 0
            for j in range(len(mu_list)):
                mu_prev += mu_list[j]*(1<<(i+j))
            xi (np.inner(np.array(C[i]), np.array(v)) - mu_prev) % q
    mu_c = 0
    for j in range(l-1):
        mu_c += mu_list[j]*(1<<j)
    return mu_c

In [89]:
# GSW方式　加法
# Input: 暗号文C1, C2, 法 q, 正の整数 n, l, N
# Output: flatten(C1 + C2)

def GSW_add(C1, C2, q, n, l, N):
    add = []
    for i in range(N):
        add.append(flatten([(C1[i][j] + C2[i][j]) % q for j in range(N)], n+1, l))
    return add

In [90]:
# GSW方式　乗法
# Input: 暗号文C1, C2, 法 q, 正の整数 n, l, N
# Output: flatten(C1*C2)

def GSW_mult(C1, C2, q, n, l, N):
    mult = matrix_mult(C1, C2, q)
    mult = [flatten(mult[i], n+1, l) for i in range(N)]
    return mult

In [91]:
# GSSW方式 スカラー倍
# Input: 暗号文C, 法 q, 正の整数 n, l, N, alpha (0<alpha<q)
# Output: flatten(flatten(alpha*I_N)*C) (I_N: N×N単位行列)

def GSW_const(C, q, n, l, N, alpha):
    alpha_IN = (alpha * np.identity(N)).tolist()

    M_alpha = [flatten(alpha_IN[i], n+1, l) for i in range(N)]
    C_alpha = matrix_mult(M_alpha, C, q)
    return [flatten(C_alpha[i], n+1, l) for i in range(N)]

In [92]:
# GSW方式テスト(0, 1の平文) (テストデータ)
q=1<<10
n=2
m=10
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5 # エラーの大きさの上限
mu= 1
t= [605, 729]
e= [5, 4, 1, 2, 3, 3, 0, 2, 3, 0]
R= [[0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 0, 0, 1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 1, 1, 1, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1, 1], [1, 0, 0, 1, 0, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1, 1, 1, 0], [1, 0, 1, 1, 1, 1, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 1, 0, 1, 1, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 1]]
B= [[36, 469], [84, 705], [936, 259], [873, 22], [313, 900], [12, 510], [810, 633], [685, 41], [516, 412], [358, 968]]
# 公開鍵 A、秘密鍵 sk、暗号文 C、復号結果 mu_cand
sk= [1, 419, 295]
A= [[166, 36, 469], [545, 84, 705], [404, 936, 259], [461, 873, 22], [668, 313, 900], [173, 12, 510], [211, 810, 633], [924, 685, 41], [179, 516, 412], [662, 358, 968]]
C= [[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0], [1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0], [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0], [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0], [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0], [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0], [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1]]
mu_cand = 1

s_try, A_try = GSW_keygen(q, n, m, t, e, B)
print("s_try =", s_try)
print("A_try =", A_try)
print(s_try == sk)
print(A_try == A)

C_try = GSW_enc(mu, A, R, q, n, m, l, N)
print("C =", C_try)
print(C_try == C)

mu_cand_try = GSW_dec(C, sk, l, q)
print("mu_cand =", mu_cand_try)
print(mu_cand_try == mu_cand)

s_try = [1, 419, 295]
A_try = [[166, 36, 469], [545, 84, 705], [404, 936, 259], [461, 873, 22], [668, 313, 900], [173, 12, 510], [211, 810, 633], [924, 685, 41], [179, 516, 412], [662, 358, 968]]
True
True
R.shape =  (33, 10)
A.shape =  (10, 3)
C = [[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1

In [93]:
# GSW方式テスト(0, 1の平文) (テストデータ)
q=1<<10
n=2
m=10
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5 # エラーの大きさの上限
mu= 0
t= [342, 13]
e= [1, 4, 5, 2, 4, 2, 0, 0, 0, 5]
R= [[1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 1, 1, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 1, 1, 0], [0, 1, 0, 0, 1, 1, 0, 1, 1, 1], [0, 0, 1, 1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 1, 1, 0, 1, 1, 0, 1], [1, 0, 0, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 1, 0, 1, 0, 1, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 1, 0, 0, 0, 1, 0]]
B= [[83, 773], [285, 3], [798, 471], [474, 348], [637, 483], [755, 455], [297, 611], [806, 394], [953, 330], [722, 608]]
# 公開鍵 A、秘密鍵 sk、暗号文 C、復号結果 mu_cand
sk= [1, 682, 1011]
A= [[548, 83, 773], [233, 285, 3], [516, 798, 471], [746, 474, 348], [905, 637, 483], [959, 755, 455], [973, 297, 611], [198, 806, 394], [488, 953, 330], [881, 722, 608]]
C= [[1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0], [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0], [1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]]
mu_cand = 0

In [94]:
# GSW方式テスト(0, 1の平文)
q=1<<10
n=2
m=10
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5 # エラーの大きさの上限
mu = random.randrange(0,2) # 平文
t = [random.randrange(0,q) for i in range(n)] # (Z/qZ)^n のランダムベクトル
e = [random.randint(0,e_bound) for i in range(m)] # エラーベクトル
B = [[random.randrange(0,q) for i in range(n)] for j in range(m)] # [[1行目], [2行目], ..., [m行目]] というように行列を構成
R = [[random.randrange(0,2) for i in range(m)] for i in range(N)]
print("mu=", mu)
print("t=", t)
print("e=", e)
print("R=", R)
print("B=", B)

sk, A = GSW_keygen(q, n, m, t, e, B)
print("sk=", sk)
print("A=", A)
C = GSW_enc(mu, A, R, q, n, m, l, N)
print("C=", C)
mu_cand = GSW_dec(C, sk, l, q)
print("復号結果=平文?", mu==mu_cand)
print("復号結果=", mu_cand)
print("平文=", mu)

mu= 0
t= [963, 295]
e= [2, 5, 5, 3, 3, 1, 2, 4, 3, 3]
R= [[0, 1, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 0, 0, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 1, 0, 1, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 1, 1, 1], [1, 0, 1, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 1, 0, 1, 1], [1, 0, 1, 0, 1, 0, 1, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 0, 1, 1, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0, 0, 1, 1], [1, 1, 1, 0, 0, 1, 0, 0, 1, 1], [0, 0, 0, 1, 0, 0, 1, 0, 1, 0], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 0, 1, 1, 0, 0, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 1, 0, 0

In [95]:
# GSW方式テスト(1より大きい平文) テストデータ
q=1<<10
n=2
m=10
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5
e_bound = 5 # エラーの大きさの上限
mu= 20
t= [92, 826]
e= [3, 1, 3, 3, 1, 1, 4, 4, 3, 4]
R= [[0, 0, 0, 1, 1, 0, 1, 0, 0, 1], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 0, 0, 0, 0, 1, 1], [1, 1, 0, 1, 1, 1, 0, 1, 0, 0], [1, 1, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 0, 1, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 0, 0, 1]]
B= [[855, 967], [961, 703], [142, 63], [617, 874], [1011, 545], [483, 260], [1006, 436], [502, 21], [229, 69], [168, 941]]
# 公開鍵 A、秘密鍵 sk、暗号文 C、復号結果 mu_cand
sk= [1, 932, 198]
A= [[861, 855, 967], [419, 961, 703], [593, 142, 63], [451, 617, 874], [463, 1011, 545], [125, 483, 260], [84, 1006, 436], [46, 502, 21], [241, 229, 69], [150, 168, 941]]
C= [[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0], [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1], [1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
mu_cand = 20

In [96]:
# GSW方式テスト(1より大きい平文) テストデータ
q=1<<10
n=2
m=10
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5
e_bound = 5 # エラーの大きさの上限
mu= 44
t= [987, 54]
e= [1, 2, 3, 1, 4, 4, 4, 3, 3, 0]
R= [[0, 1, 1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 0, 1, 1, 1, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 0, 1, 0, 1, 1], [1, 0, 0, 1, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 1, 0, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1, 1, 1, 0, 1], [0, 1, 0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 1, 1], [1, 0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 1, 0, 1], [0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 1, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 1, 0, 1, 1], [1, 0, 0, 0, 1, 1, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
B= [[170, 812], [1001, 213], [331, 737], [664, 396], [200, 794], [544, 425], [838, 165], [254, 810], [779, 956], [590, 284]]
# 公開鍵 A、秘密鍵 sk、暗号文 C、復号結果 mu_cand
sk= [1, 37, 970]
A= [[695, 170, 812], [67, 1001, 213], [930, 331, 737], [913, 664, 396], [664, 200, 794], [778, 544, 425], [436, 838, 165], [553, 254, 810], [276, 779, 956], [674, 590, 284]]
C= [[0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0], [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0], [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0], [1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0]]
[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
mu_cand = 44

In [97]:
# GSW方式テスト(1より大きい平文)
q=1<<15
n=35
m=512
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5
e_bound = 5 # エラーの大きさの上限
mu = random.randrange(0,q) # 平文
t = [random.randrange(0,q) for i in range(n)] # (Z/qZ)^n のランダムベクトル
e = [random.randint(0,e_bound) for i in range(m)] # エラーベクトル
B = [[random.randrange(0,q) for i in range(n)] for j in range(m)] # [[1行目], [2行目], ..., [m行目]] というように行列を構成
R = [[random.randrange(0,2) for i in range(m)] for i in range(N)]
print("mu=", mu)
print("t=", t)
print("e=", e)
print("R=", R)
print("B=", B)

sk, A = GSW_keygen(q, n, m, t, e, B)
print("sk=", sk)
print("A=", A)
C = GSW_enc(mu, A, R, q, n, m, l, N)
print("C=", C)
mu_cand = GSW_dec2(C, sk, l, q)
print("復号結果=平文?", mu==mu_cand)
print("復号結果=", mu_cand)
print("平文=", mu)

mu= 16789
t= [10439, 6266, 24009, 15482, 21933, 3978, 18247, 5695, 32351, 18566, 5113, 8523, 66, 31695, 4047, 20390, 9067, 11369, 2894, 31445, 8633, 6897, 3287, 16250, 3622, 27032, 10318, 25899, 10738, 9087, 25480, 15527, 23026, 4801, 15021]
e= [5, 4, 1, 0, 2, 2, 3, 1, 4, 5, 0, 0, 3, 3, 4, 5, 0, 0, 5, 0, 5, 2, 4, 0, 2, 4, 2, 0, 2, 0, 4, 3, 4, 2, 5, 0, 0, 4, 4, 4, 0, 3, 0, 0, 3, 5, 0, 4, 0, 2, 0, 2, 2, 1, 1, 5, 1, 2, 2, 1, 0, 2, 2, 5, 3, 2, 2, 3, 2, 4, 4, 3, 1, 1, 1, 2, 2, 4, 5, 3, 3, 1, 0, 3, 3, 4, 3, 3, 4, 2, 3, 0, 4, 3, 3, 2, 3, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 4, 3, 4, 5, 4, 3, 0, 2, 2, 4, 3, 2, 0, 4, 0, 1, 2, 4, 4, 2, 3, 4, 1, 4, 5, 0, 2, 3, 1, 1, 2, 1, 2, 4, 0, 0, 1, 5, 5, 5, 3, 5, 0, 5, 4, 4, 1, 2, 5, 4, 1, 4, 5, 0, 2, 1, 4, 2, 3, 3, 1, 0, 5, 2, 5, 2, 2, 4, 5, 3, 2, 4, 3, 1, 2, 0, 0, 0, 1, 2, 2, 3, 1, 1, 0, 4, 1, 1, 1, 2, 2, 3, 4, 3, 4, 0, 4, 3, 3, 3, 3, 0, 3, 4, 2, 0, 0, 1, 2, 1, 2, 3, 1, 3, 2, 5, 0, 3, 0, 4, 3, 1, 4, 2, 1, 2, 5, 3, 0, 2, 0, 1, 5, 4, 1, 0, 4, 5, 3, 2, 1, 5, 3, 3, 3

TypeError: 'numpy.float64' object is not callable

In [ ]:
# GSW方式の加法準同型性テスト(1より大きい平文)
q=1<<16
n=35
m=512
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 5
e_bound = 5 # エラーの大きさの上限
mu1 = random.randrange(0,q//2) # 平文1
mu2 = random.randrange(0,q//2) # 平文2
t = [random.randrange(0,q) for i in range(n)] # (Z/qZ)^n のランダムベクトル
e = [random.randint(0,e_bound) for i in range(m)] # エラーベクトル
B = [[random.randrange(0,q) for i in range(n)] for j in range(m)] # [[1行目], [2行目], ..., [m行目]] というように行列を構成
R = [[random.randrange(0,2) for i in range(m)] for i in range(N)]
print("mu1=", mu1)
print("mu2=", mu2)

sk, A = GSW_keygen(q, n, m, t, e, B)
C1 = GSW_enc(mu1, A, R, q, n, m, l, N)
C2 = GSW_enc(mu2, A, R, q, n, m, l, N)
C_add = GSW_add(C1, C2, q, n, l, N)
mu_cand = GSW_dec2(C_add, sk, l, q)
print("mu1+mu2 = mu_cand?",mu1+mu2==mu_cand)

mu1= 26802
mu2= 23385
R.shape =  (612, 512)
A.shape =  (512, 36)
R.shape =  (612, 512)
A.shape =  (512, 36)


UnboundLocalError: local variable 'i' referenced before assignment

In [ ]:
# GSW方式の乗法準同型性テスト(1より大きい平文)
q=1<<16
n=10
m=160
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 2 # エラーの大きさの上限
import math
m_bound = int(math.sqrt(q))
mu1 = random.randrange(0,m_bound) # 平文1
mu2 = random.randrange(0,m_bound) # 平文2
t = [random.randrange(0,q) for i in range(n)] # (Z/qZ)^n のランダムベクトル
e = [random.randint(0,e_bound) for i in range(m)] # エラーベクトル
B = [[random.randrange(0,q) for i in range(n)] for j in range(m)] # [[1行目], [2行目], ..., [m行目]] というように行列を構成
R = [[random.randrange(0,2) for i in range(m)] for i in range(N)]
print("mu1=", mu1)
print("mu2=", mu2)

sk, A = GSW_keygen(q, n, m, t, e, B)
C1 = GSW_enc(mu1, A, R, q, n, m, l, N)
C2 = GSW_enc(mu2, A, R, q, n, m, l, N)
C_mult = GSW_mult(C1, C2, q, n, l, N)
mu_cand = GSW_dec2(C_mult, sk, l, q)
print("mu1*mu2 = mu_cand?",mu1*mu2==mu_cand)

In [ ]:
# GSW方式のスカラー倍準同型性テスト(1より大きい平文)
q=1<<16
n=10
m=160
alpha = 8 # スカラー
l = q.bit_length() # qのビット長
N = (n+1)*l
e_bound = 10 # エラーの大きさの上限
mu = random.randrange(0,q//alpha) # 平文
t = [random.randrange(0,q) for i in range(n)] # (Z/qZ)^n のランダムベクトル
e = [random.randint(0,e_bound) for i in range(m)] # エラーベクトル
B = [[random.randrange(0,q) for i in range(n)] for j in range(m)] # [[1行目], [2行目], ..., [m行目]] というように行列を構成
R = [[random.randrange(0,2) for i in range(m)] for i in range(N)]
print(mu)

sk, A = GSW_keygen(q, n, m, t, e, B)
C = GSW_enc(mu, A, R, q, n, m, l, N)
C_alpha = GSW_const(C, q, n, l, N, alpha)
mu_cand = GSW_dec2(C_alpha, sk, l, q)
print(mu_cand)
print(alpha*mu)
print("alpha*mu1 = mu_cand?",alpha*mu==mu_cand)

7561
R.shape =  (187, 160)
A.shape =  (160, 11)


NameError: name 'GSW_dec2' is not defined

In [ ]:
# BGV方式の鍵生成
# Input: 自然数 k, t, n = 2^k, 法 q, 多項式のリスト s1 (秘密鍵の一部), 多項式 a, エラー te (t*e)
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 秘密鍵 s, 公開鍵　A

import random
def BGV_keygen(n, t, N, Ninv, q, k, s1, a, te, w_list, w_inv_list):
    s = [[1] + [0 for i in range(n-1)], s1]
    b = ntt_poly_mult(a, s1, k, N, Ninv, w_list, w_inv_list, q)
    b = poly_add(b, te, k, q)
    A = [b, [mod(-a[i], q) for i in range(len(a))]]
    
    # b = (np.array(a) @ np.array(s1) + np.array(te)).tolist()
    # _a = [-val % q for val in a]
    # print("b = ", b)
    # print("_a = ", _a)
    # A = [[b[i]%q] + [_a[i]%q] for i in range(m)]
    return s, A

In [ ]:
# BGV方式の暗号化
# Input: 公開鍵 A, 平文多項式m, 自然数 k, m, t, n=2^k, 法 q, 係数が0,1のランダムな多項式のリスト r
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 暗号文 C=m + r * A

def BGV_enc(A, m, n, t, k, N, Ninv, q, r, w_list, w_inv_list):
    m_prime = [m] + [[0]*len(m)]
    Ar = [ntt_poly_mult(A[i], r, k, N, Ninv, w_list, w_inv_list, q) for i in range(2)]
    c = [poly_add(m_prime[i], Ar[i], k, q) for i in range(2)]
    print("m_prime = ", m_prime)
    print("Ar = ", Ar)
    # Ar = matrix_vec_mult(A, r, q)
    # c = [m_prime[0]+Ar[0], m_prime[1]+Ar[1]]
    # c = [c[0]%q, c[1]%q]
    return c

In [ ]:
# BGV方式の復号
# Input: 秘密鍵 s, 暗号文c, 自然数k, t, d, n=2^k, 法 q
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 平文多項式 m

def BGV_dec(s, c, n, k, t, q, N, Ninv, w_list, w_inv_list):
    # print("s=", s)
    # print("c=", c)
    print("len(s[0]) =",len(s[0]))
    print("len(c[0]) =",len(c[0]))
    cs = [0 for i in range(n)]
    for i in range(len(s)):
        cs = poly_add(cs, ntt_poly_mult(c[i], s[i], k, N, Ninv, w_list, w_inv_list, q), k, q)
        # cs_i = ntt_poly_mult(c[i], s[i], k, N, Ninv, w_list, w_inv_list, q)
        # cs = poly_add(cs, cs_i, k, q)
    cs = [cs[i] % t for i in range(n)]
    return cs
    # c = np.inner(np.array(c), np.array(s)).tolist()
    # c = [c[i] % t for i in range(2)]

In [ ]:
# BGV方式の加法準同型処理
# Input: 暗号文 c1, c2, 自然数 k, 法q
# Output: c1+c2

def BGV_add(c1, c2, k, q):
    c = [poly_add(c1[i],c2[i],k,q) for i in range(2)]
    return c

In [ ]:
# BGV方式のチェック(テストデータ)
import sympy as sp
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k
t = 10
e_bound = 10 # エラー多項式の係数の最大値
#s= [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-13050489207666472, -9556036188972064, -4179162532145459, -9184992799613515, -70583699527867344, 47913284109821845, -3256030134006019, 64673602292555156, 14676442396673738, -29209988800670998, -9360936448027361, 29061240223506387, 20559832094793040, -16712077582493088, 71069353735653646, -4124241428120350, -18012624124096378, 22978625382454570, 927594348504817, 45122835545618132, -31116353270297151, -47875472189309821, -45256943348078102, 42155606030624109, -40053318129479828, -31324723765546719, -260045495243101, 5318576170981815, 32453685698800868, -64593201221304168, 55837067447497182, -61745651173377633, 46282055642323428, 39894820695472574, -3938043113592197, -40491997876204889, -13532047996168052, 20990670870791758, -71830348114183533, -26046015312116757, -10124908598467150, -33913651987964964, -1662403410819272, -51992161799380655, -34131430071070719, -9114685971128424, -58622216640685511, -26480238847463572, 33216532580908149, -45217603002250412, -40709356678520970, 15889975263860687, -29913855131741892, -11240649466048035, 66963361609270547, 43957242279583418, 9409357757609791, -46213032497565046, -55719466573677462, 64855575702689622, -50550767466793903, 70765787295860574, -71658982494234918, -63865737858432086, -4639084988391753, 25460694630711103, 16220098122204625, 52211257096781726, 5822963360957353, -47245710183015298, 34891627461345758, -71993329304689359, 57643158634620057, -21120777252991309, 51983529605709751, 41665753882631104, -50789992560139600, 32241797004818967, 64558141862701254, -18026850017244257, -390306439092173, -23861512759377831, 1481077761719298, 65900340595398328, -62959345521095176, 40361445089797620, 65514095721005612, -25552413813922902, 16544160622400462, 58044710462345234, 65949303905766321, 18798757313432405, 39789765608801476, -60085467289173603, -53696943064671935, -29468353488924897, 12514131222814703, 52832213627663460, -671332935826633, -10290954789520259, 21313404454621369, -31730059963821385, -5622583361092193, 62771686057848228, 9133731950043551, 28593102905723154, 29093113384805855, -10800761936361485, 35210435701940434, 23880469003128688, -61185645275912708, -41371736162817969, -9287571860216962, -67536844999781529, 8168517327590510, -3126496477294919, -43962151742941933, -56796900094683698, 52682162042162296, -48947149202552905, 57466433877796124, -43043500448519872, -32689673501619761, -47878076408959246, -67279627895244621, 13231766115291114, 52090253036726321, 47056908174040008, -33106899390192187, -26726586167748205, -28346430640411053, -66956310575440727, 58143304193726167, 3218357749555409, -29751903601494352, 56388831142050112, -29101651671514296, -1951753994552997, -37966380415685366, 17545726614083101, 61342652652244759, 68979518372876477, 1599873937009315, 70389477196248413, 26934369053292449, 59798731822011995, -62308658556993859, 17656095246289147, -47877186619993066, -56172477582061708, 68456577134315108, -15952743788517511, 45365354843101099, -41915356601402779, 24316493131842889, -70973805094163260, 19151198038079625, 24958865207148324, 52054730544034928, 64939849555951396, -62142037145085011, 56338264198770698, -35363539399358305, 16511168964245499, -15269775732250237, 34202494319709599, -66216250371571629, 17408873507482787, 69823623800521062, -24734437900327651, 34165832653964357, 55775822263096668, 4616812211046794, 31098618528557775, -59294587112088131, 26759394640356064, -3316835972728680, 17026139847138212, 56770114053652526, 49819768574374794, -33654700429263741, 40245616527469202, 30630476992330435, 30862177462352636, -761948013275703, 44989354047632843, 70945891304785942, -4384894468652153, 5070994405495921, 346268687488809, 44213994139722875, -69710112065517091, 32886572547755091, 49986645777429989, -35364075023976172, -56791525396919538, 7628868321443261, -35508108859590682, 58440677894926399, 64554892234071219, -6443137466355622, 54997216670251435, 7517735373146051, -43580771562108466, 48338996495218489, 4982738542441459, 52326515432886240, 59714727944139993, 22016341609355267, 66557689819737078, -33159676629274329, 695545276550516, 7452291377811744, 43331664341251826, 9425358062916854, 55927039784210834, -26968097028594845, -34823241210132323, -71591553798231357, -60917772764130403, 65893982390233328, -54695098205605475, -54432830854445092, 21546810702011418, 68544739909730114, 15790001471283972, 15384928505858136, 10252436160263429, 28499399863856147, 6139486018321998, -49942139015778564, -34445120319675062, -59022959644544351, -69519294096033640, -6648311264008882, -33613060412053717, -43044059652524139, 6277505550692454, -49647487260375079, -22332325102940470, -26321393564644824, -70056430520242666, -48953921512660588, 66331700951165571, -68551821389376856, 8466407259326945, -1104863346389750, -31216972900808254, 12942675695075027, -41910210019367456, 49205853801940211, -26663587773267634, -70847628159903532, -35386391732915881, 28596761396694343, -37741788783453038, 20387773100537951, 51331641444078778, 42393723183223726, 8038175594168333, -23185054350297063, -12873276341009443, -11251770990650016, 59813809541062429, -36065627544467127, -63521273834599633, -18127117190629719, 11951343186462412, -3592183700208682, -46800757576804777, -42144321725440215, -55970898495994790, 19636713212663712, -53481696535209619, 59204710037876268, 6232827530328418, -70133152245365599, 63426890421748275, -39199175238064091, -14163426122481730, 40452154386125655, -138168533933043, -47778897372456399, 13422486765414798, -43930436332386023, 52505138386105906, -23963685878947273, -20440010335773220, -11544905267132922, -2837618148966439, -15188313468251322, -5424382061397233, -34558625220065377, -5112189293567501, 62149895385781782, -37709823787864367, 29340840511764037, -28087231743003328, 29911731737702497, -36805523522805177, 14509534352622396, 64012273013124854, 54154802994203600, -18956000724608843, 52601850072695723, -34903392642543244, -41816065137120349, 31134013944517719, 10856566026429176, 6101163171647192, -44226595008105757, -59964656214025855, 69238589649408241, -16953335496201057, -14294221265697487, 46732050080948845, 18345352379789489, -70535027265160806, 24698758540545629, -5378210895791854, -18443458976182457, 41449969275488569, -13616941017147938, 59883818167802155, 29072466236645610, 58968679481030378, -10642805965959832, 55058722951954158, -3158386095755857, 3359124841804057, 12581511762455304, 2988351412709079, 17377693010722984, 38172549019365928, 69282077723099592, -35691223030381346, 33974299718841025, -54402134207474734, -19599398180276746, -57680279590902184, 3907150970799157, -66580366404593645, -32860049611106187, 38522130073127554, -63604087961024397, -17230824320931825, 68236139067117195, 70292467085794070, -46322280237493901, 56128270284980305, -54742541316739870, -4916086327216812, -52284893316875047, -55905546213877835, -5899499018012127, 39076646945899830, -39846242830143160, -42987978092216230, -31526168258992036, -53991456315018856, 37657513794509696, -68726787544332563, 49122267226932330, 46843140355859545, -36205940402972288, 67713682732171993, -24563564035113327, 35435187278095016, 36452722847543132, 49265051810398404, -51392021298026149, 3357202696988091, 11147649964550236, 64626836061346977, 43407123902416404, 57472695252027673, 4704035768302811, -67300141464137637, -9197039976252406, 29868707210238351, 8912710647603295, -27690612920242850, 46443036815337078, 7453102194030942, -57452486640709194, 65641941185206647, -70914081746209363, -25001592266694290, 49735371411483724, -54290638056683503, 57325894657576438, 10717289149123218, -66917419417362674, 32681261244586577, 30506165566277607, 34165241324330395, -33532289271132125, -37186392532989326, 22340227250929511, -14304381557485686, -23371119424038530, -69867082351792447, 47974147284572594, -1577306010948127, -43496448092486012, -5439926845294823, 10050928922707033, -55822913425214263, 803188808389617, 1029131375152929, 71568971509949788, -53866653410069996, 14746359753656895, 70625270042935515, 61147344301603521, 66077972545421040, 71600796619776329, -43612677103319397, 4701797285651880, 47809312388982682, -51176184103393597, 52313112578558629, 66344526480022552, -3779300436673539, 63405349580119165, 57120189352536966, -63126375425634471, -10694763821080800, -35429320640200269, -5340345336909674, -40314538761573445, 62687996456063675, -33950068845429176, -62968978410557563, -57364846160269917, -65256933377794658, -9570969664321153, 6807059981478617, -39020638261997657, -22536148607290525, 24831659858363329, -10105677529578711, -55813949059124148, -20871844432168395, 35867722329707354, 64646985827544994, 10216897121882919, -35051656050466730, -16967681592123445, -57354105075349607, 12896758530275070, 4596107040826075, 18319782608521969, 66919705976583561, -56816316250651626, -12945821379938424, -8546496421380435, 57207241974019989, -40915435498352560, -69414248430842730, -17267698520383171, -35651265719698758, -3397639732028517, -55665237892495921, 58142688127938509, 71047923758657183, 63385217835533616, -22712353954522247, 4568323680207728, 13125718844870459, 68903128947996394, 43468977628542153, -53461739967254516, 62517328033499167, 71471266971132725, -32283397452047433, -40333464554744884, 49595062266180071, -48196186764840542, -54824615929782661, 30924202615371512, -44056700796355371, -65281357922841236, 12879931845335353, -70991537666922446, -18162700785429522, 59730166239314175, 44055042612379700, -20879672987432198, 69003149527875819, -25866278480754301, 59112292312872484, 42441985466222577, 39378996856733356, 32720300779554706, 70439317642311216, 27147676933713687, 69770334798190054, -42016346614919889, -23518648192838513, 47298773658016101, 21214867351418033, 59436283471485939, -5559042442067071, 53443936222705016, -55235301351926232, -30832488705804457, 24448461331735778, 45583818568464331, -21713638127473554, -22954525958063090, 49794043322083784, 66993925182313495, 66899964378742823, 71096976162086049, 50284247378329137, 58215274750205940, 69806921257424887, -59209175655004091, 55472478542769995, -32703370422607462, 16937112806999938, -36655575861311677, -55352526222422063, 65656678328480559, -57483030179580764, -54224903344750809, 35281192556355060, -41342436741291825, 2988240561336341, 58320204928451304, 140993600958981, 10186711232897608, 7446406399010652, -67964683461185235, 25518982948297584, 60532411725213523, 33517369551542508, 46765312393216107, 43589666452891207, -10451330492679037, -54055754038541749, -3378424509846695, 26474375577437484, 42009345890249138, 43330095601520120, 23514702470872086, 50276522932376170, -30311549011251509, 69149185253836372, -58483285985051712, -8302293213196164, 16280613833977852, -9302513981841308, 46006316449009027, -23490503901452593, -59262108065941860, 70366702599992161, -38277903025081412, -239108037115651, -69850298131728590, -44577097346536624, -9356981522410900, -53354522431048312, -38323336359464151, 67770610535756519, -9240167133325697, 16882666151842301, -29256076105318906, -53625366220568219, -13606725818502824, 15528673452985128, 52112057558093018, 50781140895888065, -27140738384903909, 3103538609191674, -28199420806520128, 15529296736719119, -46671476100684905, 52795276755449254, 50472440294926791, 61121779246385819, 63727540340747418, -10717138927685395, -10562780016062156, -46052792902193176, 5493369569843086, 45825143079166821, -60735163923004122, -39125789727237226, 35100035799432002, -12974575909823530, 24078771476274036, 40717126238908239, 17660300009444832, 52013348156556902, -5111974755228291, -49679243512583188, 50734551916361453, 20547589795821425, -36216211590554405, 29503585844834078, -65604802254963556, -46938489474428617, -9157608487416098, -33238943063532785, -65607286623790763, 2053275045173306, -15096252134382034, 52530816050760034, -42983994097919205, 52625012825797051, -17471295690330108, -52208898689807274, 51541801739685133, -28995574415436978, 70043544145708995, -24544680946312950, -8352283168619299, -56855186387172803, -55951514543454182, -66306379074993736, -58465748205722848, -70618197402484541, 21881085273447525, -39664057054640543, -43936318986950001, 34666399224772261, 40977342934114783, -69604473301136362, -9514993693175980, -66385733519055868, -20877175686894941, 43434846145225896, -58234695628594507, -62515019351174265, 60473593599606235, 12745710554437454, -2145573158465738, 65597505753879566, -27496748446758944, 66968849721194119, 47051390252866913, -69341241775610726, 40932952994168527, 66426494042412341, 1034053831145988, 23645412223059657, 14945714567790578, 26109043161296273, -45291902979759155, -2460918447404628, 23636276587917419, -46862128315194233, -8730826685014611, 12412523315766240, -44352518739705491, 56140061897335191, -43004629808268514, -28913436093795550, -43341156625223042, 52356957285669268, 44590267737339159, -66229593412247325, -22572605259509448, -70927537229287107, -11536349605798818, 21511840750867383, -28381242767224453, 21560356701335110, -24481894867121014, -51541622153835035, 15615115474418625, -2096090002071123, 35400620348545137, -40015184069193401, 67732419416456086, 28793657088860982, 32156875187621283, 70030434252723229, 46690478546362952, -34544644737110555, 18216514783351877, 46437297899138693, -39318714561738055, 43561462967500388, -44286293645445818, 37468813789715845, -41604165689995002, -20165609021052340, -58004732172547204, 60108338521688824, 44965644852624140, 27724740779690788, 63806517370148827, -24401227607284212, -59342208499192137, 3985009429837725, -20304905898486591, 42863046992863169, 9312350815178465, 19183563340548196, 69085692933174974, -31625225968845892, 45873731954121183, -47921621526363751, -34464228992280523, -15520090064952928, 58264909740292131, -28484252203843892, -27265657531398920, 69774632407388285, 16145556607130824, 8057207633610516, -71175426106469921, -19060243303511213, 19453600791611113, -27326687324603197, 26472822176141651, 14496913923513144, -48031658512258598, -47027494778195101, 32774372901586659, -71346572130900964, 17104023778428714, -24262277567902319, 14580062698166376, 71090264322849081, -16350133155854988, -61160771052063125, 37961309213978733, 32324706423511017, 26918608111955832, -55628210759542961, -46701462221404771, 32224388509635878, -30301906552741998, -3333461096962605, 34548107217518774, -18516782830460601, 55007188548860957, -16515362264861127, 24555782832377490, -65302142316854711, -8631183067296509, -10988167530891683, 48766076454473521, 46745848848070679, 17198128859550593, 158470790368611, 14410658582479946, -23117206780971684, -45745771021140442, 59007849352806692, 36775252572187516, 49844907120639993, 14642197464966053, 69365204839581052, -2837202608978353, 53262707991666198, -39451872432597025, -13732260794786410, -61157109845374571, 63886754493550476, 30598206916887384, -3623332144991489, -34575582093463750, -70569345909580507, 25179164852840094, 37196980579255541, 58857551178825457, -27083559088118287, -57884253946327262, -63885462488665592, 42226115730391261, -45954963380686891, 51169753603145912, 756828553409077, -53620367957842187, -64761043217043650, -51745992118464456, 55566379855293555, -35175191133999631, 8530553727004684, -65298106441689966, 58898851227430348, 8687105787319220, -44162877935701333, 39417106453491662, 54445983549749868, 21932519488073188, 2551185638633363, 23984719438526811, -25724250397595814, 56594944973853048, 55320929241849334, -9240977016192265, 58345860953096898, 59472767017704607, -44292890853330334, -65336625843732854, -35968369616490263, -60613238432406533, 66299735988383610, 22812182800353855, 19524460611606906, 45494313003224126, 14875869292135798, 69717796941951105, -6121990958349719, -1922352860206351, 23285856528223006, -48132602888114356, 40706893602052434, -69326667293356405, -71854355407775205, 10955024551807687, -8886412562096374, -42368123088752062, -44344597741183490, -65077759835529610, -43522878033462172, 45929139240798553, -41420276472938859, 12235893633629340, -36742401789207638, -69599263565622580, 22069764568315080, 52308428681063821, -31788775312070132, -1065798890053154, -61023162823291530, -29506718130143289, -5889144562308861, 5426453063348286, -34686919663066704, -52140825826890483, -61080440957722620, 52265098423643508, 43381680531725305, -38840916041921398, -63062926988801940, -29965042831141817, -60623073297840190, -13143115488701270, 58571987359438309, 57009012523818120, 34598061192896061, 71691258236105721, -29328926073607495, -40586761752917215, 30298347617090902, -61118170661919394, 32317438963541772, -771660372901346, 70167328999574300, 52003630449403228, -62332428103167316, 56628495714480739, -7608996675900884, 71633363221733277, 18814224215986196, 68030092369691064, 49958009942241112, -62836562670086515, 35607671970443510, -27198959783655495, 28544576903253862, -46944483467788084, 10801141797012450, -65762128104947700, -19892459302895344, -55570608514196956, -34181247326809485, 25069185438663579, 39846315542595854, -36239488589289747, -9307928544891691, -53348316207355648, -68098280199274038, -3127949686988550, 86805160316608, -54273661854497653, -41006413303952371, -21871037026269534, 61852391479603680, -16474055383610042, 25787260938882784, -28305673645872888, -50200925194029054, 47295235667791188, -11918041065332965, -16913594053023905, 39131604465700612, 32825180292931488, 16053940745641458, -25184173903234232, 46888612262941736, 9002258721628368, 62840963048777754, -45681449989771402, 60248334694966270, -13411961688642419, -31924833513852717, -4015590286709695, -25405351308236935, 30195381051787790, -55206178414068914, -54284987879703978, -51388845651804622, 53763066167540964, 43383894726821662, 13843501458889621, 69928455020715058, -9159961765765505, -61218668104645797, -4453258614970765, -51531270073841902, -6097695521533080, -15167752268085120, -47242346159581692, -18927589034200102, -50671006761525868, 33398953520984595, 41318890471002122, 63640833926490585, 22345256367965587, 17416778245333787, 46609744626304420, -69588679263795347, -55364423100286902, -24730995657048482, -39278642528841123, 9355198731429493, -19962837191540646, 4365560255459355, -64225196306014992, -45502420963687877, -3119330725313402, -10153762641059150, 14333985587805108, 43874421549356388, -33273259713033983, -9214450814763580, -66119197372693610, -64789801901477756, 28235669185688097, -17345327912012278, 43525760949211253, -66806260636448757, 60951916596228368, 46845944631135758, 58770642785791013, -47510301752129704, 333099032258818, 7370922639326447, -47876032742932098, 50801840820903073, 53851949998274795, 39811742432710367, -58264370033793070, -69031542779863341, 26605302466501405, -27328425785261791, 53465383831905329, -51057607954821069, -62240349050931578, 43009558798516557, 65413961062943660, 44240067656657616, 55980479377217622, -7808600786591897, 17481329499357437, -11843375597752675, -68704100551295115, -14706697658663796, 58013979339610744, 1546034450348034, -12550645569721357, -42658710307390633, 50466397695102321, -67707960480882536, -30923486994158110, 29691886378405420, -9106343512345714, -71215081150567366, 2237926470969779, -52424702658289193, 18271231663789229, 54562728468022549, 41983876457406042, 42019091477729783, -44376504875444677, -15127925667400827, -22161575499462152, 39043166102467342, -56375927250654092, 17025693365390679, -61235178097378284, 35605069955347161, -66454102871323054, 55719362612399000, 59352382302952178, 55673267994904643, -18421857589524251, 52495443131393932, 2775244152133908, -35429312018274576, 17345931330346089, 49890550444989600, -48295215211156039, -62083018808016032, 13130925580725058, -64145537291421047, -4011085207511091, 39312179210990084, 20665439059936459, 59751026497216414, 19763700932613596, -62287724926581788, 68135573561797152, -35728477061938883, -65732533954238268, 13113015205730386]]
#A [[-839366021339463, 40433573830519354, 62575191534545696, 63206625800655349, -54501503670630077, 45040392128600736, 62213469239819049, 26914114944323573, -644847353925064, -45605917401242559, -2680478298528610, 14784712471366600, -22808323605651405, 69211132766685224, -47671539567144962, 2034597021025415, 35879140282008156, -24585538522867794, -52252119845005153, 59493873715188350, 57199252354247169, 25811514601533872, 71488488995135343, 46514349280527144, -19919063637938063, 43669884430559241, -32994902913905989, 54088364246134858, -28561073844297701, -67993483828426398, -23664237313639074, -68401126861418532, 51715787647230310, -21113099277331648, -28919560735463359, -13231823210971611, -64370587373006973, 21131366504257702, 42858383548014148, -1803357547477366, 9681605924412897, -10717966928455332, -62041892657411549, -22707210385625710, -20889461582370370, 27508618643436154, -44636535669781369, 42621810667568699, -32842408224644249, -4306963839930642, 66500410865521703, 27638220734708930, 30939529534958584, 13758319604852634, -13771150505953378, 50169998543723545, -8383682725459760, 56699353285441814, -67978362043427307, -14753038046085264, -1375101690263666, 68980016689954646, 12182655379394816, 65333969639051131, -36896025125206728, -62144313086844305, 59978778708960357, -22998919578348453, 5572476801859394, 3493082380202672, 49110305005395748, -14290605803318880, 49393765618275058, 8959491482769102, 28694282889302945, 33120341732236776, -21697805783344928, 46790474346247330, 63666155589094467, 5722455021602273, -35227871520356051, -14829217146411052, -36722836341498918, 35571367088272150, -25123430796081512, 43402758350177296, -44751655984761298, -51558395821623395, 39631034365699460, -66358354818273647, -34079634974777256, 38771329361560137, 33545510434152155, -62715426704184030, -53712226883158858, 2365339276898747, -14740270645985047, -61765165368222002, -41328416161660702, -44150577600974354, 34076441145091093, -28201467188703807, 1899897334438314, -37231957276249231, 53694239955144317, -49006881534311012, -40661909012078900, -8962816076072814, -61909894564366631, -34910377155707701, 39686989181602850, -61527446859820125, -28518861405700119, 53899001195590269, -44628554245858725, -39164540593696114, 44218162141179732, 39989181337417411, -55670668746285369, -26243958215681661, -1183417253209465, 2750747180808516, 60048097306126482, -69030004594606439, 37363716061737194, 54822285466637899, -35629941813519672, -67796469687664927, 52860978038747384, 62140156733198171, 24372040795342490, -3823737932305588, -34940769937582166, -46396596590125683, 40475358211425806, -44143799114887148, 55313881608654379, -8789714797280891, -58425670272847454, 24908768939463695, -46372051151038909, 29384315082889794, 8007206324058309, 69057034918648080, 29133403977444322, 17693926413593334, -42103536145206384, -43440705727991791, -64339304331845624, -11115189864911720, -43764919843544261, -30167174499270821, -64803691789716120, 34799946286002838, -42095051562952206, -26365635694508058, -12370885932933784, -48321971291686716, 10746779509849496, 64115997532602845, -49955561386660315, 27662310301141288, -57489626880917905, -36502185269740226, 9261123673793414, -43476109728178574, -35520872918980867, 28940215171232636, -26209506628515388, -43636949716098426, 9213344884294265, -37853151474337327, 29368775183730849, -56928072174961264, 22709776984839637, 38306681529326309, 27378559054116441, 56877556417892500, 37139462674629362, -27588513358427939, 27029143313182057, 19610062173467906, -49139080685008272, 54911063268207746, -44979817784940323, 53951286578601348, -62018447471877276, 44303115491137196, 50291902324792117, 42454972754315737, 69947297614906869, 21084423562414375, 66018826037831554, 12025025416904227, 2549030423846486, 20433892943763081, 31077147203058792, -58698300531688711, -39252051279061976, -56024224422892863, -58176970101513738, -52895639598734194, 8446950646351040, -37308654341294278, 6519762573102663, 21675056601322709, 58505872584879099, 51721128464317343, -58428222407535264, 40066540085636335, -31288902711462046, 4319522991688676, 52595426258607909, 60648925739545384, 48308535938817155, 53530381574979703, 46499876360627205, 4798822892115899, 19538664406307943, -39070968938546103, -25666675723571255, 33919461398540875, -5041208130692293, -65084744800231551, -36639677395166919, -9356860499544385, -31820981892672776, 52456572470877834, 70574620194664290, 9849593896033188, 24214943038851867, -54805077965312480, -27407767982154504, 29293509512521702, -30274720536017583, 8874949366116940, -27828521439532205, 30784886242781943, -22480536612416880, -60277818022001516, -64822123090104524, -21101341465363779, 29053455086827638, 3686617111736177, -18245369628178085, -12880981866074627, 25483058036927410, 20359251609009809, -64437221014376134, -21765842195489470, -19572843662141908, 16446843804912394, 23451640674251942, -55784327281879146, 5007381899697331, 30540317609246932, -29404523553199298, -49090838855317285, -40494185483233107, 71386944058068013, 29761433406148352, -24082097074154845, -14530937352793262, -46126697580168840, 8050408018248756, -25580930557565194, -6969317830758742, 6436325442378722, -50907739389889060, -41925335363795340, -53821229196480234, 59080265189095488, -30116653855814070, -11622420288737447, -12422967100963158, 33931655127737464, 25171338004044292, -27277782917017908, 39610355583846875, 44541828669761957, 69654808580905261, 1614567095701742, 31150481912362697, -23250727567462065, 64743727952380527, 26872485651735360, 7186677316844674, -6718386454101776, -3899166733944510, 58544732170296226, -37826759386256810, -53043250543385624, -5461230480488432, 5663651750686459, -63045583133441173, -18834989434057081, -63220780234224498, 5883548879941275, 11118702476873305, 12580733358156363, -14406108275098111, -69380301024337797, 1263949161568825, -34587474168398953, 13255631927394122, -36730359012979833, 71461051089925817, -3526830957101595, 56898480698657553, 64292042962934484, 33800673443610500, -14427468189715803, -67759609144737989, 40908667284260516, -51235592325231150, 40854352049520548, 32944470800325037, -40408490698590829, -15617704461609572, 46888970073685569, 18345628861135926, 54229718319053195, 54169203841104107, -52474335273740080, -11113258460095265, -49444630535618784, -55090441338161060, 64838754192598777, 58479951954791168, 36783437594373545, -8485077266580477, -11186569763496365, 32613176778459696, 22722895704733735, 33030696000969431, -22347659953606115, 14571335664899616, 64083715713968306, 55398317002470054, -46997149016821622, -26853256522221795, 67848999376018192, 71218744248974951, -4169744623753545, 15178589899919440, 63073643948180472, -70549772654942878, -42827683154946487, 69164527482940906, 68325305092366211, -49210824771687171, 12911998401579796, 320539244022542, 30924596530968009, 40958390312443023, 1922983557848410, 32557091091073102, -63323340602244311, -66066806005708635, 23185457236915466, -18166245995716507, 46253740528447641, -7563536322504949, 27409114786797590, 69929315407635547, -11005556356510332, 25784010940538043, 25132088571011641, 1518265694433006, -45182063721362997, 4584192560280972, -38086773295556629, 66274897265018642, -14015367884345563, 20555933481032063, 43917080383127973, -38863890310003624, 66837553987555445, 63841001440398325, -13809641333185840, -33655476497344122, -43065032659132608, 42786138758655878, -32184225442058291, -53235172875430065, -17598765497627160, -39072220151386571, 4778186280985543, -6616648335363518, 20989952322474964, -61636057524262440, -44629201050860855, 48952741736276191, 59240521887091161, -17024714433681439, 62476239771264116, 41054780054699483, -67505645642962102, -64926904310013611, 18011750939564710, 18386583344082981, -19772023128084923, 44005689998895452, -47131568886830010, 12191515162747334, -41616936172398667, 2097418270810669, -37811596543188607, 69024797025509886, 24536518193067704, 38221948628821826, 21648393833329732, 10624589496364594, -53893328141048761, -10598798099341112, -1472930609274145, 35149246322449550, 56597350773144619, -68589218937321372, -37128708961800484, 4572010976182615, 22227322019673551, 60284221428726115, -25467034699025732, -35043807392609313, 31911706274149250, 14423047960528058, 63517769119023197, 70054038100519592, 60469382085470649, -15800523218477452, -43731772620062620, 55618431376057041, 45765583216970257, 69921266344197379, -27822112640408534, -59265934601955075, -29294409309114293, -35926761251396949, 51636142319153141, 20918575490760029, 35931778742118436, 15621509521543943, -50372748696797676, 45497783256533976, -58792203845308276, 40687740926197700, 25505195848895306, -63740071161319527, -27770871269330364, -41219012491140259, 23703486632588449, -65906899221944160, 37271054845479314, -51534499708906667, -50219554376750318, -28622731662091984, 68911317130284077, 50291110537269355, 15750920630531137, -3735356680844461, 9282836621184057, 35940331411484729, -47032615748103716, 30487850198491171, -40229675274957759, -66972111054778289, -8140328335479310, -3764884794760008, 47824221024672776, -32988216813489715, 56946493522125121, -52804648329904939, 7893271121416930, 40259643710034219, 52015069682468633, 55541003217703442, -46721683631318014, -24323005848609172, -39960447059241494, 31667908986462446, 63699472049073060, 62427867598378907, 67481175645791975, -53235758075222404, 4379791554296295, 41200908967791498, -17997826582875895, -61221698552057670, -36698815094903810, 66796724730183635, -71564399946074779, -28568363241686903, -43792575800588346, 43384701469791014, 54022095646130128, -16740500647909539, -24653465292644062, -48581385611943405, -11769453659231638, 25826665175808211, -10625326671429416, 30582892275893420, 32298419657675626, -17993854309378860, -28301744889312736, 65062862112091723, -2567209115753292, -26721703745389444, -65627563172643047, 45301224709357246, -62503786424747938, 5494813035473997, -19330466666213212, 36240851560323124, -53015852529684430, -29328576986117867, -61220863496472320, 68031470215476656, -22644543677255146, -24314857599159680, -41622795431522172, -5739826296825328, 41480596866755110, -1635598365332511, -48498442194376690, 70916480099568382, 47610952943953196, 58428062370451806, 6825960797732958, 3988769800503409, -19319376058223415, -6351303961973943, -43614639642530179, 23161335302148725, -64971136730554930, -15271817329521061, 10782704170112700, 14531492037218539, 29920733632859709, -56455808717147553, 71352870661586170, 64094760425790096, 59132067720706110, -48887459310705072, -24481161055435111, 1246250744877971, 54961089160004665, -36899593627359209, -50515937275953282, -29366948311185513, -27154203505524053, 10320076260079539, -47062703241326635, 33742062826918611, -58015822824990923, -27093456455550295, -47986689759858566, -18590680207531089, -69044538063595270, 28604738477462480, 28296260434700766, -63130375915060381, -1525509130954383, -46638011384839447, -34694030978452350, 52495129514174994, -31897026746072269, -20330401137341638, -60776428180669401, -23584342109237972, 68753596183553129, 33349291472978953, -52338877666768550, -24022615232790074, -63755320996803448, -32108328608844151, 61466511459404683, 30800002802977741, 64002638436529314, 25445966438559814, -50684994468092380, -60281545248004178, -6800918579799581, 54634104554266441, -55526599276913466, -13760910118650179, -35171276474781230, 37514884548003467, -53134458835103907, 54495728649403686, -9135832226946782, 51631494677459944, -25391789079555195, 47051686471223160, 57943846361685030, 42879259671350865, 43457895831340889, -69951210015458234, -27553127348667164, 36571553053443438, -7443729667797405, 4939827394080389, 231821319051236, -54469842313432092, 15011575636051651, -36332139531603886, 52828433361318595, -23662655277366264, -47413953665299285, -71610099499568861, -41280051628477194, -35754248190457901, 68818894479155941, 9762204995897662, 6731793507181275, 54761214051925505, -32626818010373774, -60004299327876073, -47519596215198326, -53973343568892911, -9684249518291718, 15587637350374864, -27522398762313220, -11983467142061866, 71449208459939371, -60229613572687576, 14634549569295203, -41764299698369145, -30029913112788467, -43670667465172042, 61637615172475009, -29925024649909500, 63900813408958640, 30623328403442096, 66430716720294383, -11612848115845757, 63268804059876242, -63948622002054963, 60638152013328472, -60463761621613619, 4428008533493000, -47755709507569170, -30265132475973581, -8125251111823476, -50024493101921753, -66507939281675636, -25438845152659916, 17330951460793508, 20397279249725123, -762668930417482, -58176800921216319, 53431845474863073, -47051813468794155, 23780305024697626, -42780531014513780, -28081255210093845, 19795779105062302, -59700959221841421, -67708449167037893, 58274795847108006, -10133415198629552, 57026129862662751, 69424734202919251, -5309891381563977, 26655736755296955, 46842294596140828, 62885520917640771, 67447291228063053, -68271470466905909, -28103179612494337, -40667526497753321, -27176363598173401, -37074006939724562, 46017164804111688, -39220993076876530, -7498610617042025, -28133083380401295, -31882807547311432, -34802400481262584, -27324715633810834, -54893670318452917, -28292142216246964, -27860321078563258, -49816129796870000, 61980435856841247, -47370115732673197, -63693888728584278, 17151626256338411, -22555817309152952, -64869733493037547, -70917401601260549, 66848856825295086, -23130755810916076, -53746399875501023, -11066690330162533, 34298719399474594, -10652950157501682, 52358381137732971, -64688914308391308, 22170995473075407, 21579893530152632, -26174398401600727, -42972634153353740, -8611148885934603, 58324529823240344, 46830403220997598, 48206323593267076, -61328879726604960, 19432542338557820, -68833972557028817, 34094211331216507, -21188221356894981, 69956560159321590, 11353688119610860, -7012657548276873, 56488762139107214, 48764940755268363, -996138379219493, -19499930760056006, -10856294827543792, -16869817085037924, 51025226083957393, -63249507511313080, 34862112150282976, -41636213868329945, 45915155461627622, -10099949311608359, 56346681113413473, 67328300500663299, 15839571695483991, 64108914681285977, 62601080750601422, -45654391848437796, 21010230860186172, 42770023332511047, -40164703797552292, 40270255876075541, -15851281103952233, 71735183952276711, 5475248224535298, 61543695655147470, -33459296409703044, 51397403413002068, -2663083622731042, 27378651546604858, 46269474370042438, 42092592278302078, 55446167387962048, 55757190426087872, -71562281931798966, 55018528338363549, -23020125284875489, 4793719848125831, 64666308759963605, 65989044674392361, -42228757166797204, -2349352025200987, -26877456376179242, 52402661877738160, -57751810114489680, -14652294218253076, -70303800610328127, 71675731443920674, 54114089033694275, 219787724671809, 33553320173221622, -2094202810567380, -53929913277622919, 44183136368876479, -9884723243024512, 27717259557921991, 69407597264818060, -25246797041584069, 42604769250952238, -28522601236283135, 33149472146982455, 46690766031520072, 52386319522608954, 59988482824138865, -47406831864893148, -69907179319392403, -20300198397969318, -41287886538857870, -11125895978497424, -47577488338806945, 7558516410475775, -60507244924530243, -70974882337075149, 53368427598267842, -50274361801010952, -59188860136238611, 41032068673762106, -14657037952549538, 26919666353664207, 9206152456752079, -24591704160216989, 19668148778167258, 5292154190449092, 15396060732157768, -8490532297418443, -39761264972865003, -47351644699475308, 12213703622359554, 48560731658587264, -51230638807213431, 6574117654738153, 39337494014064820, -52986453751378615, 69447233048474682, 62531314219150960, 63255788493701596, -66977999068141863, -70978532941967223, 2030353895072525, 51839338031669190, 53829740376489914, 59094074284454573, 24175083470124037, -68434808044256667, -35715642550668029, -60480907179205112, -6158747449332172, -18632042068910693, 30977489443028956, -7543669112494398, -55829046592929853, 51141200501364931, -39401471333094835, 11878166067028499, -56915951539969209, -48452542324542170, 16699472735727924, -26510347485504108, -55991446835439182, 34798426689787177, 32393292380756614, 68635296744257596, 35762199203324996, 8359823488096839, -57600717111274183, -2754143495018159, -34317205425460519, -1253536554755624, 40930429384570448, -23046177112622738, -45591767797659046, -14384241465733187, -3772186612434788, 16481851470252312, 27446740757438771, -47239517719007614, 50112512448188612, -61878113578143771, -10494266101981789, 48295114415489913, 67625971920226041, -57403406202206908, -67414037626934198, 57336187742262003, 52067863467292150, -13690948841359454, -1171463078459384, 22732308134684871, 61854319664156341, -26308815688005363, 3247886150024257, -47059399983417078, -38291108052751434, 42905554996166410, -16538399074837768, -23463080104515036, -38968478126194062, -9649177688026748, -52423478735570233, 23297850567095405, -15446844024357389, 47652508353737941, -69726842588230020, 32380807814176563, 33085943145065646, 34839437669415827, -60571850711237921, -46109177299613857, 21162535260111836, -47722440409387430, 56704691865997882, 42770662832877693, 6266423988367439, -162630918341676, 9383510640205055, -12892367225584226, 39895368053518934, 30958292792335552, -57983954596575840, 11400086399806696, -39246233101407541, -56775134158426559, 57992666152129376, 42984797500037065, -68364844476152160, 11150451691353219, 53849286424879720, -40245587258492789, -64377919574395812, 35196663044160224, -36357720529027978, 64683758982671717, -43590403370339314, 11523650445239415, -16167564208258863, -40561192836514382, -49820565850732806, 60274607526848848, 26817429886169476, 30569241371722230, -49702191955168632, 60825580210929129, -12713725691409584, -60911701715780653, 33097317145655070, -11547737487341285, 31252055757706974, 1415264384983723, -62331417446769635, -45686295820606949, -63196886811164986, -26904257487187169, -67448913293888548, 34112262758333370, 71229013733670212, -17907095228143914, 22920525542239587, 45244886085727011, -19173078491098678, -26875842471002089, 27818226088683713, 54891459245332410, -11791946015360999, -13924957787234942, 2314901425687335, 63897046970575549, -39912257734234499, 71651695913947594, -62347598601247819, 9015539226459679, -30205655628089942, 65250708062163672, -47870706687639765, 6044416700222829, -36478031691571676, 4303262552879678, 25752105355796662, -46482190654398224, 61609043758415833, -20197336968247256, -18181315425416285, -10633535665722318, -67823259143973565, -66097816493226856, 45935842613750177, -42759047033731488, 7929613689784461, -62438961311249900, -22737849175799068, 14167734535869149, 53958281278240522, 67225797610144366, -55231568786520835, -42793101453080700, 50713784881097333, 44604287542548012, 39159685406848792, 50538826557231815, 40952257222760901, -3136401737326338, -60325229627937837, -57918350500493174, -10311268160438584, 50820353436594445, -6443022423071510, 63671294637573961, -32761738520227291, 66465326727970567, 51813619721914475, 3616175941836188, -55605275987295073, -59135659097101919, 59836247229769679, -13284938365056380, -70918904268804159, 48084854953487073, -23143974342877936, 40634471115966635, -26776016628998684, -58296925180689646, -28122506946506040, -13457802411171023, -53872000280348000, -55374560304844771, -42260122791008777, -34140937433299435, 22738832622209217, -11912666347167955, 26805320690906758, 7602479569856763, -29435841761429108, 43025047481909569, -64566867721746881, -65345084528116537, -5129020588424276, -9774940555855934, -1303726647389176, 65554842952436856, 69034826801215767, 66008224567270719, 16296533229584082, 46123954609080348, -53592081182718534, 71036489802782018, -48795752110543338, -71076241131457670, -39834371054750066, -34083807111618681], [69465320453906759, -53803238909884064, -41352215986303450, -57543951766019825, 43030588384066346, 66722228954897279, -45756534554715472, -30254903431448260, 58932163743116682, 68659533898430006, 1074072476584077, 8629874552349705, -24580521904163647, 48864409924553565, -22876454054931619, 53346199496294548, -42731708123351276, 57676239350151477, 62686136358291345, 27717143744304146, -31963249303938838, 68002230133691866, -70255708672527554, -4480137746353581, -45504693649449312, -32993878264118390, -62254577348662481, -31889264504221994, 5826750511430241, -29873460820871023, -3234287177272970, -69808657346509794, 60131496530187372, 46182439344474907, 27716144363550133, -62957273403155273, 44712736511204056, 38786407428076334, -27964142116188924, -65883210173529410, -57915779000998965, 60441117733182082, 2695368822643221, -53790289901137554, 25985227633753305, -52201323743876156, 71425999952873721, -45655818737643379, 64787516557682349, 37171971633611554, -6908416990193989, -61842858259222096, 71588561442256670, 14719700848748875, -39875866859874054, 18273662655205640, -31893483217276205, 11351734534429650, -14541351765717467, -21533534560665053, 26887865948145819, 71840004619508338, -12853223870079970, -67070198979225444, -47186101438839128, -4615998396493489, 68724942457028396, 35428772119913778, 65498662475282470, -48159658702985428, -7478955612022251, 63279460589487900, 61118555312831556, -57132528594984958, -8140183591787095, 21493900348689912, 42895383128358145, 3263379628664057, -29866971417167717, -30710861251027457, -57530851175513680, -7666326129380818, 47383698842211524, 24457444317305200, -54308844985229020, -25520296722788863, 22160841739300509, -28846026090209862, -37090024525567287, 16851491083931846, -5565734353541977, -50004693762540557, 1825085393981165, 51332860562291885, -65762032455804395, -17502872658241876, 59576725308666900, -68429331862968423, 24505617364693759, -39004058158413826, -2451275503782426, -12421874047668300, 39707944388103349, 50840563339593537, 35504277527964931, 44042718793408548, -2600572679455160, -29466641660802036, -11647265878834338, 31923161644315077, -31561384821328291, -23389359908993341, -41647229725132176, 2133918673886337, 57537293485292505, 70808802189588064, 25360094743440263, -12422050287878623, 35739149503254702, -11880337361232584, 59213505649499291, 49175708289024142, -28518506161160678, 2297549960899897, -5004289933295792, -8595317217681749, 25421356500471523, 25035235903915461, 69312032284409414, 14121079460411337, -37247871594042267, -70266219007492230, 26129440717501251, 5771287694482810, -61122516502370912, 65094671764380644, 39368891907675508, 67190528170368632, -28171651659974095, 40574416127627042, -33180210955882847, 16122981955002604, 52452037038219230, 9672700695423670, 3286452414076908, -67148171568715843, -21454485882445578, -54883302913964251, -48830359832437843, -41512152706748728, 46662309270703505, 53103552155167693, 65970118665841260, 44948152969049071, 47842902078244144, 68479039098952264, 16926485352281094, -71226752809416443, 70387312920299818, -38133446929652359, -20758502817453527, -10251477035384452, -53620905098953137, -68392584381062479, -23180040138796794, 40642755439561822, -57473774638212197, 40469659198748546, 40606860623360042, 31377049323751088, -23148488100539126, 68486357300086231, -41218446546908729, -50664002693614097, -48822537316933323, 20047637420962740, 4248016693523984, 27138627905021896, -71310985092570740, -42656609640951022, -51245230973637772, -13067499553817322, -8762496197672077, -70901041883042970, 1220075039319640, -2050375181657407, 21383330593516153, -17697255965906348, 4387963675435189, 4992704041232206, 22737484184475144, 32123247594589552, 40449311897895688, -3361777317552086, -37659026332971850, 60841353810793093, -67648513139085962, -54931166384955750, -50706195884886990, -56832242916247279, 50109468174653338, -29518660635498502, 61859134758012196, 23216972868799220, 52259182737195282, -65411581368179642, -32002403672874074, 53412964998121576, 46151336077353354, -37167020420630671, -51974179962319204, 1695260427004665, -9197449974888219, 13845820861957534, 70475971142757299, -28664157484365031, 21432814248492635, -35120483916981855, 35863782006742156, 52487430181246529, 58062896332213452, 7579695251636771, 41641909039594129, 7443280548480405, -13754834909148688, -53963730399348664, -62756909214967717, 8031967666377463, 33181747362462026, 18456675565864903, -22007553068180038, 16090270880121798, -60394905645227322, 35113666351196063, 33496961195288935, -63609169273136592, -69170118036028523, -44638846173211473, 11606922439045340, -60641203686451028, 37655484384023063, 69676555852480776, -11310072221493000, 30774610081804389, 23140247068508559, -16278922467818570, -40632818918253655, -65622324078590107, 25642065949203865, -42249968896652987, -9319266599689641, -6766282513442018, 3062570545906647, -10466301445681937, -66879153917273910, 50587456116719845, -8549603290968411, -18066295537379831, 37697200005698661, 70524946556391353, 67896003290694070, -34891926939064086, -59901322032026238, -48671617606882007, -14152648390291794, -59633549484221794, 48609699321525016, -14930972767388013, -60964005743765976, -47554459636540991, 26419534668724112, 68493819755653443, 36536866289220058, 62144385564375686, -47905279814052487, 49896658553152991, 66564387546387528, 12535011828258327, -27577240479106022, -26685493357498622, 12832793941300293, -9528258161245555, 41469306381353104, 32919346595962581, 53499389895767953, 57484989737154418, 43513756644134342, 732280779102321, -14957728166275402, -50190989870820196, 22089581272713532, -2480294482753622, -51421351623222069, 56000364403477974, 10784654935961688, -52501510159513381, 16553215693505594, 34522915643223463, 15152538012531308, -34777203630241863, -38490553731154517, 60680482181269371, -4388933278873409, 41808942624492252, -921858256774014, -26722643151563968, 8440803814101102, 50923044542271019, 15270081785020061, 12579927252104091, -57803311648672213, -12901252809773048, -31436680089858323, 12758485089463564, 16219265650984775, 17303196208561934, 70514219800316900, -26855412074204723, 9861874450248559, -27317500530586738, 20596530746183480, 31739429184712439, 32367125863075721, -10752949596748732, 41151625982910443, 33850030543197255, 15213804290862406, -26823601413831212, 38628709375484805, -35547584010052937, -12735059089885117, 9590533478537313, -3127774438334545, 49857189866704625, -61963883793419939, 38431140987664999, -64361492901353642, -24330853954490767, -50103163558663007, 19448724132805341, 16726533121081537, 24130843340450428, 51551888474019436, -28953487485213473, 45903763380936638, -52411907070241516, -65408224723288538, -4819365105414661, -61038309566750173, 43784949931089915, -32888917170734805, -2630214694709954, 27559390718057422, -14289762212877938, 69932564563449144, -64345849849803742, 35799254513558413, 28807366731724142, 47245915266535933, -58452360486451202, 21933453223077984, -42401607318184987, -35694326812847286, -6687230386873355, 52593349993221075, 67247395763558596, 8768644170324156, 46382105269377570, -18641471322059048, 23958429617436551, -36313869773317161, -65989413187373736, -46864726131074009, 64524627089299714, -55638734848665493, -31428486796407826, 43596285872307044, -30119156023368075, 19386436869991298, 9879105028609193, -68511993660251117, 11099502612656374, 22786852947620195, -11858473192588341, 29915716905676786, 58289938314046246, -9271782784765988, -47376714836848859, -24596057172561828, 26517535674782315, -54052483351526096, 21934497825352258, -41586856414788302, -63646708969774844, 60769642897936421, 57468692889548226, -15077119010913354, 24624489323376087, 59766567882515328, 62452255190535348, -38192034419723639, 49652898674774370, -66427634797639400, 47184910896341395, 6434069487929844, -36085864177648225, 3699382505859880, 40729282141821225, -70315567984380133, 23737366556444899, 43020128572474355, -71666858935798973, 25052616596228092, 13005841415213301, 24226406768898473, -5330215732872199, -9407794729896272, -11408074953145799, 23458763622384355, 19728861450478273, -56934895651649773, -11010594664941858, -49729090906150763, 11883549438493816, 70087985387287058, -64382617089789491, -34350874317619147, 38256777299628412, -31700020498808085, -15315978419073463, 45102010010126992, -32379571611147010, 8938332229752288, -3097320517448808, 28712777586417693, -2780837130641148, -46940876632266322, -35479330438939470, -55371033339690567, 41913859813117072, 7952571487674511, 46253291633829729, 11901824974995442, -23522241433392549, 12033053394600478, -14018001525234279, 19068669863287935, 54138151520481934, 8533963840308953, 55268647518737571, -58475132693064748, -44354542538675931, -51177623405061295, -5053748828434733, -42090164249579729, -9689008381111031, 42183273437349086, -21873420007099351, 55080183565264337, -66993463374944747, 40577026712580529, 43255880481968010, -34170338898148550, -10116193499152357, -23188398335246344, -67995927738598631, -24174167075394724, -70121143349030785, 56984883863671075, 3394830374273993, 27417080542124148, 22535970369166656, -10997540548374717, -29439754005300246, 2351483213845321, 990760639933044, 51212161910533881, -71396253508593810, -1178959076404968, -7348046357333231, 10938070845316477, -13130836932097326, -48576115095634298, 13008633836281020, 21757883076237839, -33562918865779728, 5588678591163879, 29959306002204086, 47673091849496799, -64332099500755792, 7027956427406209, 2841252904788716, -1091628506371305, -68932288864105123, 4390180435633215, 31231231854762939, 10205638019993291, -13325223279639861, -44480609779511628, 42688655727934653, 66673388312789386, -23671257363654317, -18052743146151484, -47530064392616147, -48091933967226445, 3121140802615258, -44489198053744584, 38522678257912023, 48284005207997666, -8376296593028992, -48438755934777938, -18630984914150453, -24117776728106727, -1355612037838175, 56338819504222451, -38729925093080717, -18640805531281866, -4450176829423088, 61003979481998665, 35318464764153006, -23273941700811011, -59543445085308198, -24902318601221062, 40557143631993366, -44556831764879166, 41074770837621927, -53656359212145880, -27300009418099260, -38691797305753135, -39093641395153046, 22730323170934769, 38464141513883508, 2530181643656658, 36983466911545192, -7410317706545173, -41787944893408438, 18713841699570601, 14079255503773045, 12230930446400220, -54943284706293286, 22096734480452000, -39247339673709390, -49481329196138470, 56540871196875485, -5469177059655472, 44092469415533014, -45010822182301379, -49024389090818766, -63778555519175031, 18051527216927876, 10375963957520629, -55748450069045807, -53358501680658218, 16244363820505877, 55788254649512519, -37789734054973971, -54817272733622962, -26948473414394359, -12602919778637798, 4328489698298954, -64244998221501237, -4229827451395543, 16234147273099904, -36889424320088911, 9812062507633305, 9827892713493408, -59626202710654689, -5175409281921677, -70610657799843961, 39665117169343534, -50900682797217793, -25818785147235017, 1795125654878582, 47925431225807279, -21282571789071410, 52346220577491422, -41966131231181992, -10016770636071909, 59601978280050629, -21234698587044925, 47913419059224815, -69424172160086714, 18059185859087583, 43318835301321776, -33892699080658583, 51589287710362125, 45627609826348335, 15447109191230285, -45533937165768293, -22739088942875959, -47724424778029874, 21010123785278834, -63135229248365900, -34343544180130325, -6950938575120315, 59279685579389045, -8599036919087683, -22744197299274098, 798941724376649, 24672737439801739, 63948670059366672, -6153717941789436, -71596540002721184, -3070833449359969, -23617467779186498, 15735758826247496, 30793684994382189, 30720562664925778, 13948472816425402, 48330016843783751, -9567888010566763, -31197036886604745, -34745171116823814, -69907192426637560, 56282571600737562, 29940464996778257, 21369497028489164, 22487407568663965, 58293483208968926, -47598305206716464, 26322780670817076, 7263015272446548, -32509583543049899, 14936681749003114, 7228129208628366, -38702867234703452, 43319996517289633, 25369561644039595, 18166806920273366, -26105952801697364, 37981578850539880, -68057841230384961, 60929510810930349, 50633247960429681, -22810841962381692, 14613563677010583, -11145186761280676, 42554303702786078, -59971340426761974, -52524220684205221, 19325177182576100, 50338761273578148, 6486614420582879, -30454218788905809, 19330231741838425, 45653726496682475, -50223108921734200, -1461381547761694, -20276807032774518, 40546412151154573, 26512999779321482, 54729793218596261, -47175744097961688, 23777380065014114, 71863597753572232, 17117253607603859, 44013096565111642, -18657238009962803, -20441065124529764, 10977183000110241, 49979164790906335, -37351691177059911, -4695854964088734, -26188989944639760, -12898956870222749, -33906885919561307, 44333783263419719, -20842074741613731, -71608546306159267, 1741787662922660, -69714732053949940, -19644058191641370, 60560754610977773, -14357071032325727, -33839756503488441, 66408844993505165, 37566894172529228, -10777444338545317, -44333084999980701, -63584554424292957, 681244675650333, -43166862700216522, 70809998834458142, -31444048617221864, 11137121728678420, -27230643132201743, 25887908832622088, -8053216872275382, 46527231473965185, 28713449739620703, -3766773690182907, -60574581882545280, 26273797757989558, 35492147030414109, 19843073067299737, 60431361346156132, -28625736929508062, -58104616224267528, -7159652489106819, 10763983378097185, -9174514249296652, -66346231689594550, -27212058673992161, -12817645444869781, 51971061030679908, -69310827738092429, 7352451711957414, -67117285103218875, -52309402862530639, -31209026854951833, -50965586252928695, -66540322408060742, 47277681263005527, 19964399323190233, -61681194918016555, -11115420893120620, 61356315601655091, -41216885448373030, -14333965147671047, -57293101997061021, -61853983369465062, -61524583886223779, -60810977743562402, 33193824096103821, -18399407749431395, -43545584133656697, -36635677824728387, 59109422048675275, 25888534542747377, -59177265249081475, 70248555659835001, 44192956990712716, -68499380533858314, 26360892768115106, -23159912479698504, -39578562265657304, -36840958074872707, 27240065380716428, -18421065483544237, 4137308260910127, -26194993412245820, 61845169882282993, -1984750089502519, -19552335124160542, 594125561914319, 340075873228679, -33970966118464272, -20060361254906303, 37877615471353516, 15199382752908638, -19477298943808606, -22675761892387822, 17073376680598804, -67805929773017948, 6462110862077607, -27758610663010871, 36529375289964985, -27817665272413074, -36677332251009105, -28024993923673024, -20311357474597134, 42850361854077004, 25096885790668487, -64822950042803200, 38599455491385865, 35923489480976519, -13436668785144879, 56843205470736582, 37080673344323315, -7370263031410874, -57033359240210703, 56966667747221395, -19313082255872698, 52083101997584450, -40077173697823799, -1795234810013332, -24255921250233934, 57556571990545275, 14075296380087246, 34533269149041582, -31317108611532908, 34906511792884713, 58451794625122842, -43541572952080435, 64218372723403528, -36179817164364865, 33703047704326115, -46987448278700835, -57199755318637417, 21122584559042986, -43395836687304665, -46762660292122330, 65093298950695949, 59180308672374250, -10241585384820485, 31312561723110441, 23460783812469139, -11249228173889579, -25020104534803974, 22829674721784998, -67584323706487885, 42155449511728479, 30314333947547422, 18506402327739230, -19659829968925834, -58460129079950037, -46734286588843926, -65732992646523484, 32787452365707181, -23064122612975812, -13241923155884352, 44084844718363830, 36033993728753555, -59251881133579336, 156210091796630, -36092682384756240, 37151355626503666, -50477137829598132, -24112923304586105, -52617650421755388, -44689154559848535, -41254104643858082, 46492963550348275, -29519574804632918, -31498088017145785, -49258794654325870, 5671401509625069, -17432753172100810, -23758058971803246, -13993892367788724, 25991487873461797, -65152008888916408, 30177118285761029, -62454843115656638, 18819740556315729, 67637239625780395, -10414540562415368, -36988488790688256, 40449372439011794, -65305424214555645, -5546267614185690, -69095924100645744, -54435385630045597, -46447982598832209, 32779249048765502, 14122488545351888, 63193911723353797, -35277960452791769, 51892316826701576, -52617451119900845, -41086484718726764, 13679768303432082, 63409662282226190, 17180827083064351, 52600590690910930, -5231663700045124, 11266111000853060, 45114082382230951, 19572772491092468, -31465366904421840, 70452669721263997, 65532718038470661, -20368737472113735, 5383494450690237, 55248769525642672, -26761691474548362, 20158150017065062, -2754763573264337, 1876282004792726, 51965935776501674, 15010633550612288, 34023662840636300, 17418058197031172, 53599899133103398, -12052117124742731, -68007316989690173, 6101961925329046, 24804623930530029, -57980116953383782, 15287774566009139, -1175886783825093, 32659043750652774, -30094313813055898, -23497585177120022, 15695150163644480, 28370737295197574, -23996342167171597, 49145303139556482, 10265826808010715, 8138614623785412, 70909837991664769, 16166687819579173, 48325242752615265, -39467325228039271, 10895284300846855, -4657985238388919, -21894292933513970, 12387134483699257, -5913237638508459, -63303425682498856, -53573346914838884, -21250256356068162, -3717402294225408, 59694504424458132, -28761945646255045, -25395288259823026, 20959053912292017, 25098233119313305, -71411168403936433, 60539323766656599, 60966985493558641, -5412623625326494, 50515873291838060, -21866511201567823, -53211905158188896, -58739354552570195, -33557357375853443, 54684740524991588, -2887442432683272, -500182269449723, 62716422710518523, -6703096202244251, -55578467472241451, -5262028581943637, -50296206587746939, -63642911783081196, -24634659559507287, 61910239881206369, -5650893835012662, 52793015510034786, -60746465735860717, 70455676465744667, -430450927252211, -44515478220076842, 26371343462318024, -58897895916602903, -67404628655677058, -46643561288746262, 64398841244134329, -41205376615546456, 55470665157783118, -3828371997177713, -67107184489946793, 70348786290728933, -13671004823805805, 43257891486098946, 4606537635646474, -41891838038804969, 1370396551195662, -38588269525967427, 54977117816634453, -51969221205654269, 34563091117131351, 34377101171228776, -44432455830105296, 67078170584091078, -14389754665365865, -46822967943798872, -47678149682083608, -35868770050744251, 30325294276428738, 65385862374027308, -6182323894472186, 67934985088752069, -21069669187996024, -9114078018200530, -42707542736748594, -57444434950812535, 66663529647915037, -4023657963626082, -20027494456361617, -33820782851995445, -16765179482780029, 61861938762572612, 49023744655956167, 7992859169345663, -3773253897501354, -59527187111815872, -70660206115338785, -44464431469953836, -62397651296988438, 68560486508923179, -1762015172906314, 46770489867598235, 51546030787911513, 61245508036077646, -40187215326046690, 40546374144573179, -27799304405683128, 5893617463525949, -43944143470560958, -58658137843218017, 67371890313424315, -38840687311238671, 44494141141072015, 15716772958824795, -44517674882303050, 11898233028057305, -54132928217052748, -66177055403606387, -47606347609569137, -53589602553523253, 52048734512539475, 59500483698439481, -22308859952956244, -33152550174692462, -54382048672371064, -132727124818735, 50124501975770567, 60046943220059427, -51088154402687601, -37784347815861707, 45111799479065801, -45779790506592260, -61427349557960829, -20572794457797555, -2490140801140708, 26799518931348769, 40924929798964835, -65944323337585045, -23010192562213907, -229681612929150, 34106998314827434, 65762549379445778]]
#m= [8, 0, 4, 4, 8, 2, 0, 1, 8, 8, 5, 8, 8, 0, 0, 2, 1, 4, 6, 4, 9, 6, 4, 8, 9, 4, 1, 9, 2, 5, 9, 3, 8, 1, 1, 2, 4, 1, 5, 7, 5, 0, 2, 1, 8, 4, 6, 6, 8, 0, 7, 5, 7, 7, 4, 1, 1, 3, 3, 4, 1, 3, 9, 0, 3, 3, 5, 6, 2, 9, 6, 0, 0, 2, 0, 6, 2, 5, 8, 7, 1, 4, 8, 9, 7, 5, 1, 7, 7, 9, 1, 0, 8, 1, 2, 1, 0, 6, 2, 8, 7, 1, 2, 2, 3, 9, 1, 7, 2, 2, 7, 1, 7, 5, 3, 3, 3, 3, 6, 7, 3, 1, 6, 3, 9, 9, 0, 6, 1, 1, 4, 0, 7, 5, 9, 4, 2, 9, 7, 8, 7, 0, 7, 0, 2, 8, 5, 7, 2, 6, 8, 5, 4, 6, 6, 1, 2, 8, 8, 3, 3, 6, 6, 4, 6, 5, 5, 6, 2, 2, 3, 8, 7, 1, 8, 6, 1, 1, 3, 5, 5, 6, 7, 8, 6, 9, 6, 0, 5, 8, 3, 4, 8, 1, 9, 2, 5, 1, 2, 8, 4, 1, 0, 3, 4, 2, 7, 8, 4, 6, 7, 9, 6, 1, 1, 5, 1, 1, 1, 3, 1, 5, 8, 2, 3, 6, 4, 2, 5, 6, 9, 6, 5, 6, 6, 0, 9, 5, 9, 8, 4, 9, 4, 6, 0, 1, 0, 5, 4, 8, 8, 8, 1, 0, 3, 0, 5, 5, 6, 9, 3, 8, 3, 4, 9, 6, 0, 4, 9, 5, 0, 4, 5, 8, 6, 0, 9, 8, 5, 8, 1, 3, 9, 8, 0, 4, 9, 9, 7, 3, 8, 4, 2, 8, 6, 4, 8, 5, 0, 3, 9, 5, 5, 1, 5, 5, 4, 5, 4, 9, 1, 3, 1, 8, 5, 2, 3, 8, 8, 4, 2, 2, 2, 6, 5, 9, 1, 5, 5, 4, 9, 1, 9, 2, 3, 8, 4, 6, 7, 5, 2, 1, 5, 7, 8, 7, 9, 4, 1, 6, 8, 6, 2, 3, 0, 7, 4, 3, 0, 0, 3, 9, 6, 0, 0, 6, 0, 2, 7, 4, 3, 9, 2, 7, 7, 3, 4, 6, 2, 9, 0, 1, 6, 0, 9, 9, 5, 6, 4, 2, 1, 5, 0, 2, 9, 7, 6, 5, 5, 5, 2, 0, 2, 1, 5, 3, 5, 1, 8, 6, 7, 6, 7, 9, 0, 8, 6, 0, 6, 6, 5, 0, 8, 2, 7, 0, 1, 2, 1, 6, 7, 0, 9, 0, 9, 0, 0, 0, 8, 3, 7, 6, 1, 6, 6, 4, 7, 4, 2, 9, 8, 2, 8, 2, 4, 5, 4, 5, 5, 0, 9, 6, 3, 1, 1, 9, 3, 9, 3, 0, 4, 6, 3, 2, 8, 7, 6, 0, 3, 5, 8, 2, 5, 2, 3, 1, 4, 0, 8, 5, 7, 4, 1, 8, 8, 4, 7, 8, 2, 9, 6, 8, 1, 9, 8, 2, 4, 1, 0, 7, 4, 7, 8, 6, 9, 1, 8, 5, 4, 7, 8, 9, 1, 9, 3, 9, 1, 7, 4, 5, 0, 7, 5, 8, 6, 2, 0, 8, 1, 4, 1, 6, 0, 3, 5, 2, 1, 7, 6, 6, 1, 8, 7, 4, 8, 1, 6, 5, 7, 3, 3, 1, 9, 8, 7, 6, 1, 2, 4, 2, 1, 0, 4, 5, 5, 8, 5, 6, 5, 7, 7, 9, 9, 9, 6, 7, 2, 4, 1, 8, 3, 5, 9, 8, 5, 3, 5, 3, 3, 6, 4, 1, 7, 7, 6, 1, 2, 0, 0, 6, 7, 8, 9, 1, 1, 6, 3, 3, 2, 5, 8, 6, 3, 9, 3, 1, 1, 3, 0, 3, 6, 9, 6, 8, 0, 9, 6, 7, 3, 8, 5, 2, 9, 0, 8, 1, 1, 1, 9, 9, 8, 6, 7, 4, 1, 1, 1, 2, 7, 6, 4, 5, 0, 3, 8, 0, 9, 3, 6, 3, 6, 0, 4, 4, 6, 5, 8, 2, 0, 5, 6, 3, 5, 9, 9, 0, 6, 4, 8, 5, 9, 0, 9, 5, 8, 4, 5, 0, 5, 7, 8, 8, 9, 1, 8, 4, 3, 4, 9, 2, 6, 6, 5, 8, 8, 4, 4, 7, 9, 5, 1, 0, 7, 8, 1, 0, 9, 2, 9, 3, 0, 7, 2, 4, 4, 5, 3, 5, 7, 6, 2, 7, 4, 3, 7, 6, 1, 5, 8, 9, 3, 2, 6, 9, 2, 7, 4, 7, 2, 1, 2, 8, 6, 1, 9, 5, 0, 3, 5, 9, 5, 7, 3, 5, 3, 0, 7, 7, 9, 8, 4, 3, 2, 3, 9, 6, 8, 1, 4, 1, 5, 9, 2, 7, 2, 6, 2, 6, 8, 3, 2, 8, 1, 2, 8, 4, 5, 1, 9, 7, 6, 7, 8, 2, 1, 5, 7, 0, 6, 5, 5, 8, 5, 9, 6, 6, 2, 1, 6, 2, 3, 3, 6, 1, 5, 0, 4, 1, 5, 2, 8, 5, 3, 9, 9, 7, 1, 4, 6, 0, 4, 5, 1, 5, 5, 4, 5, 8, 0, 9, 5, 3, 4, 1, 9, 7, 2, 1, 6, 4, 8, 1, 8, 5, 6, 3, 6, 1, 6, 4, 6, 1, 2, 5, 5, 9, 6, 9, 1, 4, 1, 2, 2, 0, 0, 0, 0, 4, 3, 8, 1, 5, 1, 7, 3, 2, 2, 3, 3, 8, 3, 5, 0, 8, 8, 5, 2, 0, 6, 7, 2, 1, 7, 2, 5, 0, 8, 1, 3, 2, 1, 6, 9, 8, 5, 2, 2, 4, 4, 5, 4, 3, 2, 1, 2, 0, 1, 5, 0, 5, 7, 5, 8, 5, 5, 5, 3, 4, 3, 4, 8, 6, 7, 5, 8, 9, 2, 5, 1, 3, 9, 8, 9, 7, 8, 0, 2, 0, 1, 7, 4, 7, 4, 4, 2, 0, 7, 5, 1, 2, 7, 6, 7, 8, 7, 2, 9, 0, 8, 0, 1, 5, 2, 5, 6, 1, 1, 6, 3, 0, 0, 8, 1, 2, 5, 5, 4, 5, 7, 8, 0, 4, 7, 3]
#s1= [-13050489207666472, -9556036188972064, -4179162532145459, -9184992799613515, -70583699527867344, 47913284109821845, -3256030134006019, 64673602292555156, 14676442396673738, -29209988800670998, -9360936448027361, 29061240223506387, 20559832094793040, -16712077582493088, 71069353735653646, -4124241428120350, -18012624124096378, 22978625382454570, 927594348504817, 45122835545618132, -31116353270297151, -47875472189309821, -45256943348078102, 42155606030624109, -40053318129479828, -31324723765546719, -260045495243101, 5318576170981815, 32453685698800868, -64593201221304168, 55837067447497182, -61745651173377633, 46282055642323428, 39894820695472574, -3938043113592197, -40491997876204889, -13532047996168052, 20990670870791758, -71830348114183533, -26046015312116757, -10124908598467150, -33913651987964964, -1662403410819272, -51992161799380655, -34131430071070719, -9114685971128424, -58622216640685511, -26480238847463572, 33216532580908149, -45217603002250412, -40709356678520970, 15889975263860687, -29913855131741892, -11240649466048035, 66963361609270547, 43957242279583418, 9409357757609791, -46213032497565046, -55719466573677462, 64855575702689622, -50550767466793903, 70765787295860574, -71658982494234918, -63865737858432086, -4639084988391753, 25460694630711103, 16220098122204625, 52211257096781726, 5822963360957353, -47245710183015298, 34891627461345758, -71993329304689359, 57643158634620057, -21120777252991309, 51983529605709751, 41665753882631104, -50789992560139600, 32241797004818967, 64558141862701254, -18026850017244257, -390306439092173, -23861512759377831, 1481077761719298, 65900340595398328, -62959345521095176, 40361445089797620, 65514095721005612, -25552413813922902, 16544160622400462, 58044710462345234, 65949303905766321, 18798757313432405, 39789765608801476, -60085467289173603, -53696943064671935, -29468353488924897, 12514131222814703, 52832213627663460, -671332935826633, -10290954789520259, 21313404454621369, -31730059963821385, -5622583361092193, 62771686057848228, 9133731950043551, 28593102905723154, 29093113384805855, -10800761936361485, 35210435701940434, 23880469003128688, -61185645275912708, -41371736162817969, -9287571860216962, -67536844999781529, 8168517327590510, -3126496477294919, -43962151742941933, -56796900094683698, 52682162042162296, -48947149202552905, 57466433877796124, -43043500448519872, -32689673501619761, -47878076408959246, -67279627895244621, 13231766115291114, 52090253036726321, 47056908174040008, -33106899390192187, -26726586167748205, -28346430640411053, -66956310575440727, 58143304193726167, 3218357749555409, -29751903601494352, 56388831142050112, -29101651671514296, -1951753994552997, -37966380415685366, 17545726614083101, 61342652652244759, 68979518372876477, 1599873937009315, 70389477196248413, 26934369053292449, 59798731822011995, -62308658556993859, 17656095246289147, -47877186619993066, -56172477582061708, 68456577134315108, -15952743788517511, 45365354843101099, -41915356601402779, 24316493131842889, -70973805094163260, 19151198038079625, 24958865207148324, 52054730544034928, 64939849555951396, -62142037145085011, 56338264198770698, -35363539399358305, 16511168964245499, -15269775732250237, 34202494319709599, -66216250371571629, 17408873507482787, 69823623800521062, -24734437900327651, 34165832653964357, 55775822263096668, 4616812211046794, 31098618528557775, -59294587112088131, 26759394640356064, -3316835972728680, 17026139847138212, 56770114053652526, 49819768574374794, -33654700429263741, 40245616527469202, 30630476992330435, 30862177462352636, -761948013275703, 44989354047632843, 70945891304785942, -4384894468652153, 5070994405495921, 346268687488809, 44213994139722875, -69710112065517091, 32886572547755091, 49986645777429989, -35364075023976172, -56791525396919538, 7628868321443261, -35508108859590682, 58440677894926399, 64554892234071219, -6443137466355622, 54997216670251435, 7517735373146051, -43580771562108466, 48338996495218489, 4982738542441459, 52326515432886240, 59714727944139993, 22016341609355267, 66557689819737078, -33159676629274329, 695545276550516, 7452291377811744, 43331664341251826, 9425358062916854, 55927039784210834, -26968097028594845, -34823241210132323, -71591553798231357, -60917772764130403, 65893982390233328, -54695098205605475, -54432830854445092, 21546810702011418, 68544739909730114, 15790001471283972, 15384928505858136, 10252436160263429, 28499399863856147, 6139486018321998, -49942139015778564, -34445120319675062, -59022959644544351, -69519294096033640, -6648311264008882, -33613060412053717, -43044059652524139, 6277505550692454, -49647487260375079, -22332325102940470, -26321393564644824, -70056430520242666, -48953921512660588, 66331700951165571, -68551821389376856, 8466407259326945, -1104863346389750, -31216972900808254, 12942675695075027, -41910210019367456, 49205853801940211, -26663587773267634, -70847628159903532, -35386391732915881, 28596761396694343, -37741788783453038, 20387773100537951, 51331641444078778, 42393723183223726, 8038175594168333, -23185054350297063, -12873276341009443, -11251770990650016, 59813809541062429, -36065627544467127, -63521273834599633, -18127117190629719, 11951343186462412, -3592183700208682, -46800757576804777, -42144321725440215, -55970898495994790, 19636713212663712, -53481696535209619, 59204710037876268, 6232827530328418, -70133152245365599, 63426890421748275, -39199175238064091, -14163426122481730, 40452154386125655, -138168533933043, -47778897372456399, 13422486765414798, -43930436332386023, 52505138386105906, -23963685878947273, -20440010335773220, -11544905267132922, -2837618148966439, -15188313468251322, -5424382061397233, -34558625220065377, -5112189293567501, 62149895385781782, -37709823787864367, 29340840511764037, -28087231743003328, 29911731737702497, -36805523522805177, 14509534352622396, 64012273013124854, 54154802994203600, -18956000724608843, 52601850072695723, -34903392642543244, -41816065137120349, 31134013944517719, 10856566026429176, 6101163171647192, -44226595008105757, -59964656214025855, 69238589649408241, -16953335496201057, -14294221265697487, 46732050080948845, 18345352379789489, -70535027265160806, 24698758540545629, -5378210895791854, -18443458976182457, 41449969275488569, -13616941017147938, 59883818167802155, 29072466236645610, 58968679481030378, -10642805965959832, 55058722951954158, -3158386095755857, 3359124841804057, 12581511762455304, 2988351412709079, 17377693010722984, 38172549019365928, 69282077723099592, -35691223030381346, 33974299718841025, -54402134207474734, -19599398180276746, -57680279590902184, 3907150970799157, -66580366404593645, -32860049611106187, 38522130073127554, -63604087961024397, -17230824320931825, 68236139067117195, 70292467085794070, -46322280237493901, 56128270284980305, -54742541316739870, -4916086327216812, -52284893316875047, -55905546213877835, -5899499018012127, 39076646945899830, -39846242830143160, -42987978092216230, -31526168258992036, -53991456315018856, 37657513794509696, -68726787544332563, 49122267226932330, 46843140355859545, -36205940402972288, 67713682732171993, -24563564035113327, 35435187278095016, 36452722847543132, 49265051810398404, -51392021298026149, 3357202696988091, 11147649964550236, 64626836061346977, 43407123902416404, 57472695252027673, 4704035768302811, -67300141464137637, -9197039976252406, 29868707210238351, 8912710647603295, -27690612920242850, 46443036815337078, 7453102194030942, -57452486640709194, 65641941185206647, -70914081746209363, -25001592266694290, 49735371411483724, -54290638056683503, 57325894657576438, 10717289149123218, -66917419417362674, 32681261244586577, 30506165566277607, 34165241324330395, -33532289271132125, -37186392532989326, 22340227250929511, -14304381557485686, -23371119424038530, -69867082351792447, 47974147284572594, -1577306010948127, -43496448092486012, -5439926845294823, 10050928922707033, -55822913425214263, 803188808389617, 1029131375152929, 71568971509949788, -53866653410069996, 14746359753656895, 70625270042935515, 61147344301603521, 66077972545421040, 71600796619776329, -43612677103319397, 4701797285651880, 47809312388982682, -51176184103393597, 52313112578558629, 66344526480022552, -3779300436673539, 63405349580119165, 57120189352536966, -63126375425634471, -10694763821080800, -35429320640200269, -5340345336909674, -40314538761573445, 62687996456063675, -33950068845429176, -62968978410557563, -57364846160269917, -65256933377794658, -9570969664321153, 6807059981478617, -39020638261997657, -22536148607290525, 24831659858363329, -10105677529578711, -55813949059124148, -20871844432168395, 35867722329707354, 64646985827544994, 10216897121882919, -35051656050466730, -16967681592123445, -57354105075349607, 12896758530275070, 4596107040826075, 18319782608521969, 66919705976583561, -56816316250651626, -12945821379938424, -8546496421380435, 57207241974019989, -40915435498352560, -69414248430842730, -17267698520383171, -35651265719698758, -3397639732028517, -55665237892495921, 58142688127938509, 71047923758657183, 63385217835533616, -22712353954522247, 4568323680207728, 13125718844870459, 68903128947996394, 43468977628542153, -53461739967254516, 62517328033499167, 71471266971132725, -32283397452047433, -40333464554744884, 49595062266180071, -48196186764840542, -54824615929782661, 30924202615371512, -44056700796355371, -65281357922841236, 12879931845335353, -70991537666922446, -18162700785429522, 59730166239314175, 44055042612379700, -20879672987432198, 69003149527875819, -25866278480754301, 59112292312872484, 42441985466222577, 39378996856733356, 32720300779554706, 70439317642311216, 27147676933713687, 69770334798190054, -42016346614919889, -23518648192838513, 47298773658016101, 21214867351418033, 59436283471485939, -5559042442067071, 53443936222705016, -55235301351926232, -30832488705804457, 24448461331735778, 45583818568464331, -21713638127473554, -22954525958063090, 49794043322083784, 66993925182313495, 66899964378742823, 71096976162086049, 50284247378329137, 58215274750205940, 69806921257424887, -59209175655004091, 55472478542769995, -32703370422607462, 16937112806999938, -36655575861311677, -55352526222422063, 65656678328480559, -57483030179580764, -54224903344750809, 35281192556355060, -41342436741291825, 2988240561336341, 58320204928451304, 140993600958981, 10186711232897608, 7446406399010652, -67964683461185235, 25518982948297584, 60532411725213523, 33517369551542508, 46765312393216107, 43589666452891207, -10451330492679037, -54055754038541749, -3378424509846695, 26474375577437484, 42009345890249138, 43330095601520120, 23514702470872086, 50276522932376170, -30311549011251509, 69149185253836372, -58483285985051712, -8302293213196164, 16280613833977852, -9302513981841308, 46006316449009027, -23490503901452593, -59262108065941860, 70366702599992161, -38277903025081412, -239108037115651, -69850298131728590, -44577097346536624, -9356981522410900, -53354522431048312, -38323336359464151, 67770610535756519, -9240167133325697, 16882666151842301, -29256076105318906, -53625366220568219, -13606725818502824, 15528673452985128, 52112057558093018, 50781140895888065, -27140738384903909, 3103538609191674, -28199420806520128, 15529296736719119, -46671476100684905, 52795276755449254, 50472440294926791, 61121779246385819, 63727540340747418, -10717138927685395, -10562780016062156, -46052792902193176, 5493369569843086, 45825143079166821, -60735163923004122, -39125789727237226, 35100035799432002, -12974575909823530, 24078771476274036, 40717126238908239, 17660300009444832, 52013348156556902, -5111974755228291, -49679243512583188, 50734551916361453, 20547589795821425, -36216211590554405, 29503585844834078, -65604802254963556, -46938489474428617, -9157608487416098, -33238943063532785, -65607286623790763, 2053275045173306, -15096252134382034, 52530816050760034, -42983994097919205, 52625012825797051, -17471295690330108, -52208898689807274, 51541801739685133, -28995574415436978, 70043544145708995, -24544680946312950, -8352283168619299, -56855186387172803, -55951514543454182, -66306379074993736, -58465748205722848, -70618197402484541, 21881085273447525, -39664057054640543, -43936318986950001, 34666399224772261, 40977342934114783, -69604473301136362, -9514993693175980, -66385733519055868, -20877175686894941, 43434846145225896, -58234695628594507, -62515019351174265, 60473593599606235, 12745710554437454, -2145573158465738, 65597505753879566, -27496748446758944, 66968849721194119, 47051390252866913, -69341241775610726, 40932952994168527, 66426494042412341, 1034053831145988, 23645412223059657, 14945714567790578, 26109043161296273, -45291902979759155, -2460918447404628, 23636276587917419, -46862128315194233, -8730826685014611, 12412523315766240, -44352518739705491, 56140061897335191, -43004629808268514, -28913436093795550, -43341156625223042, 52356957285669268, 44590267737339159, -66229593412247325, -22572605259509448, -70927537229287107, -11536349605798818, 21511840750867383, -28381242767224453, 21560356701335110, -24481894867121014, -51541622153835035, 15615115474418625, -2096090002071123, 35400620348545137, -40015184069193401, 67732419416456086, 28793657088860982, 32156875187621283, 70030434252723229, 46690478546362952, -34544644737110555, 18216514783351877, 46437297899138693, -39318714561738055, 43561462967500388, -44286293645445818, 37468813789715845, -41604165689995002, -20165609021052340, -58004732172547204, 60108338521688824, 44965644852624140, 27724740779690788, 63806517370148827, -24401227607284212, -59342208499192137, 3985009429837725, -20304905898486591, 42863046992863169, 9312350815178465, 19183563340548196, 69085692933174974, -31625225968845892, 45873731954121183, -47921621526363751, -34464228992280523, -15520090064952928, 58264909740292131, -28484252203843892, -27265657531398920, 69774632407388285, 16145556607130824, 8057207633610516, -71175426106469921, -19060243303511213, 19453600791611113, -27326687324603197, 26472822176141651, 14496913923513144, -48031658512258598, -47027494778195101, 32774372901586659, -71346572130900964, 17104023778428714, -24262277567902319, 14580062698166376, 71090264322849081, -16350133155854988, -61160771052063125, 37961309213978733, 32324706423511017, 26918608111955832, -55628210759542961, -46701462221404771, 32224388509635878, -30301906552741998, -3333461096962605, 34548107217518774, -18516782830460601, 55007188548860957, -16515362264861127, 24555782832377490, -65302142316854711, -8631183067296509, -10988167530891683, 48766076454473521, 46745848848070679, 17198128859550593, 158470790368611, 14410658582479946, -23117206780971684, -45745771021140442, 59007849352806692, 36775252572187516, 49844907120639993, 14642197464966053, 69365204839581052, -2837202608978353, 53262707991666198, -39451872432597025, -13732260794786410, -61157109845374571, 63886754493550476, 30598206916887384, -3623332144991489, -34575582093463750, -70569345909580507, 25179164852840094, 37196980579255541, 58857551178825457, -27083559088118287, -57884253946327262, -63885462488665592, 42226115730391261, -45954963380686891, 51169753603145912, 756828553409077, -53620367957842187, -64761043217043650, -51745992118464456, 55566379855293555, -35175191133999631, 8530553727004684, -65298106441689966, 58898851227430348, 8687105787319220, -44162877935701333, 39417106453491662, 54445983549749868, 21932519488073188, 2551185638633363, 23984719438526811, -25724250397595814, 56594944973853048, 55320929241849334, -9240977016192265, 58345860953096898, 59472767017704607, -44292890853330334, -65336625843732854, -35968369616490263, -60613238432406533, 66299735988383610, 22812182800353855, 19524460611606906, 45494313003224126, 14875869292135798, 69717796941951105, -6121990958349719, -1922352860206351, 23285856528223006, -48132602888114356, 40706893602052434, -69326667293356405, -71854355407775205, 10955024551807687, -8886412562096374, -42368123088752062, -44344597741183490, -65077759835529610, -43522878033462172, 45929139240798553, -41420276472938859, 12235893633629340, -36742401789207638, -69599263565622580, 22069764568315080, 52308428681063821, -31788775312070132, -1065798890053154, -61023162823291530, -29506718130143289, -5889144562308861, 5426453063348286, -34686919663066704, -52140825826890483, -61080440957722620, 52265098423643508, 43381680531725305, -38840916041921398, -63062926988801940, -29965042831141817, -60623073297840190, -13143115488701270, 58571987359438309, 57009012523818120, 34598061192896061, 71691258236105721, -29328926073607495, -40586761752917215, 30298347617090902, -61118170661919394, 32317438963541772, -771660372901346, 70167328999574300, 52003630449403228, -62332428103167316, 56628495714480739, -7608996675900884, 71633363221733277, 18814224215986196, 68030092369691064, 49958009942241112, -62836562670086515, 35607671970443510, -27198959783655495, 28544576903253862, -46944483467788084, 10801141797012450, -65762128104947700, -19892459302895344, -55570608514196956, -34181247326809485, 25069185438663579, 39846315542595854, -36239488589289747, -9307928544891691, -53348316207355648, -68098280199274038, -3127949686988550, 86805160316608, -54273661854497653, -41006413303952371, -21871037026269534, 61852391479603680, -16474055383610042, 25787260938882784, -28305673645872888, -50200925194029054, 47295235667791188, -11918041065332965, -16913594053023905, 39131604465700612, 32825180292931488, 16053940745641458, -25184173903234232, 46888612262941736, 9002258721628368, 62840963048777754, -45681449989771402, 60248334694966270, -13411961688642419, -31924833513852717, -4015590286709695, -25405351308236935, 30195381051787790, -55206178414068914, -54284987879703978, -51388845651804622, 53763066167540964, 43383894726821662, 13843501458889621, 69928455020715058, -9159961765765505, -61218668104645797, -4453258614970765, -51531270073841902, -6097695521533080, -15167752268085120, -47242346159581692, -18927589034200102, -50671006761525868, 33398953520984595, 41318890471002122, 63640833926490585, 22345256367965587, 17416778245333787, 46609744626304420, -69588679263795347, -55364423100286902, -24730995657048482, -39278642528841123, 9355198731429493, -19962837191540646, 4365560255459355, -64225196306014992, -45502420963687877, -3119330725313402, -10153762641059150, 14333985587805108, 43874421549356388, -33273259713033983, -9214450814763580, -66119197372693610, -64789801901477756, 28235669185688097, -17345327912012278, 43525760949211253, -66806260636448757, 60951916596228368, 46845944631135758, 58770642785791013, -47510301752129704, 333099032258818, 7370922639326447, -47876032742932098, 50801840820903073, 53851949998274795, 39811742432710367, -58264370033793070, -69031542779863341, 26605302466501405, -27328425785261791, 53465383831905329, -51057607954821069, -62240349050931578, 43009558798516557, 65413961062943660, 44240067656657616, 55980479377217622, -7808600786591897, 17481329499357437, -11843375597752675, -68704100551295115, -14706697658663796, 58013979339610744, 1546034450348034, -12550645569721357, -42658710307390633, 50466397695102321, -67707960480882536, -30923486994158110, 29691886378405420, -9106343512345714, -71215081150567366, 2237926470969779, -52424702658289193, 18271231663789229, 54562728468022549, 41983876457406042, 42019091477729783, -44376504875444677, -15127925667400827, -22161575499462152, 39043166102467342, -56375927250654092, 17025693365390679, -61235178097378284, 35605069955347161, -66454102871323054, 55719362612399000, 59352382302952178, 55673267994904643, -18421857589524251, 52495443131393932, 2775244152133908, -35429312018274576, 17345931330346089, 49890550444989600, -48295215211156039, -62083018808016032, 13130925580725058, -64145537291421047, -4011085207511091, 39312179210990084, 20665439059936459, 59751026497216414, 19763700932613596, -62287724926581788, 68135573561797152, -35728477061938883, -65732533954238268, 13113015205730386]
#a= [-69465320453906759, 53803238909884064, 41352215986303450, 57543951766019825, -43030588384066346, -66722228954897279, 45756534554715472, 30254903431448260, -58932163743116682, -68659533898430006, -1074072476584077, -8629874552349705, 24580521904163647, -48864409924553565, 22876454054931619, -53346199496294548, 42731708123351276, -57676239350151477, -62686136358291345, -27717143744304146, 31963249303938838, -68002230133691866, 70255708672527554, 4480137746353581, 45504693649449312, 32993878264118390, 62254577348662481, 31889264504221994, -5826750511430241, 29873460820871023, 3234287177272970, 69808657346509794, -60131496530187372, -46182439344474907, -27716144363550133, 62957273403155273, -44712736511204056, -38786407428076334, 27964142116188924, 65883210173529410, 57915779000998965, -60441117733182082, -2695368822643221, 53790289901137554, -25985227633753305, 52201323743876156, -71425999952873721, 45655818737643379, -64787516557682349, -37171971633611554, 6908416990193989, 61842858259222096, -71588561442256670, -14719700848748875, 39875866859874054, -18273662655205640, 31893483217276205, -11351734534429650, 14541351765717467, 21533534560665053, -26887865948145819, -71840004619508338, 12853223870079970, 67070198979225444, 47186101438839128, 4615998396493489, -68724942457028396, -35428772119913778, -65498662475282470, 48159658702985428, 7478955612022251, -63279460589487900, -61118555312831556, 57132528594984958, 8140183591787095, -21493900348689912, -42895383128358145, -3263379628664057, 29866971417167717, 30710861251027457, 57530851175513680, 7666326129380818, -47383698842211524, -24457444317305200, 54308844985229020, 25520296722788863, -22160841739300509, 28846026090209862, 37090024525567287, -16851491083931846, 5565734353541977, 50004693762540557, -1825085393981165, -51332860562291885, 65762032455804395, 17502872658241876, -59576725308666900, 68429331862968423, -24505617364693759, 39004058158413826, 2451275503782426, 12421874047668300, -39707944388103349, -50840563339593537, -35504277527964931, -44042718793408548, 2600572679455160, 29466641660802036, 11647265878834338, -31923161644315077, 31561384821328291, 23389359908993341, 41647229725132176, -2133918673886337, -57537293485292505, -70808802189588064, -25360094743440263, 12422050287878623, -35739149503254702, 11880337361232584, -59213505649499291, -49175708289024142, 28518506161160678, -2297549960899897, 5004289933295792, 8595317217681749, -25421356500471523, -25035235903915461, -69312032284409414, -14121079460411337, 37247871594042267, 70266219007492230, -26129440717501251, -5771287694482810, 61122516502370912, -65094671764380644, -39368891907675508, -67190528170368632, 28171651659974095, -40574416127627042, 33180210955882847, -16122981955002604, -52452037038219230, -9672700695423670, -3286452414076908, 67148171568715843, 21454485882445578, 54883302913964251, 48830359832437843, 41512152706748728, -46662309270703505, -53103552155167693, -65970118665841260, -44948152969049071, -47842902078244144, -68479039098952264, -16926485352281094, 71226752809416443, -70387312920299818, 38133446929652359, 20758502817453527, 10251477035384452, 53620905098953137, 68392584381062479, 23180040138796794, -40642755439561822, 57473774638212197, -40469659198748546, -40606860623360042, -31377049323751088, 23148488100539126, -68486357300086231, 41218446546908729, 50664002693614097, 48822537316933323, -20047637420962740, -4248016693523984, -27138627905021896, 71310985092570740, 42656609640951022, 51245230973637772, 13067499553817322, 8762496197672077, 70901041883042970, -1220075039319640, 2050375181657407, -21383330593516153, 17697255965906348, -4387963675435189, -4992704041232206, -22737484184475144, -32123247594589552, -40449311897895688, 3361777317552086, 37659026332971850, -60841353810793093, 67648513139085962, 54931166384955750, 50706195884886990, 56832242916247279, -50109468174653338, 29518660635498502, -61859134758012196, -23216972868799220, -52259182737195282, 65411581368179642, 32002403672874074, -53412964998121576, -46151336077353354, 37167020420630671, 51974179962319204, -1695260427004665, 9197449974888219, -13845820861957534, -70475971142757299, 28664157484365031, -21432814248492635, 35120483916981855, -35863782006742156, -52487430181246529, -58062896332213452, -7579695251636771, -41641909039594129, -7443280548480405, 13754834909148688, 53963730399348664, 62756909214967717, -8031967666377463, -33181747362462026, -18456675565864903, 22007553068180038, -16090270880121798, 60394905645227322, -35113666351196063, -33496961195288935, 63609169273136592, 69170118036028523, 44638846173211473, -11606922439045340, 60641203686451028, -37655484384023063, -69676555852480776, 11310072221493000, -30774610081804389, -23140247068508559, 16278922467818570, 40632818918253655, 65622324078590107, -25642065949203865, 42249968896652987, 9319266599689641, 6766282513442018, -3062570545906647, 10466301445681937, 66879153917273910, -50587456116719845, 8549603290968411, 18066295537379831, -37697200005698661, -70524946556391353, -67896003290694070, 34891926939064086, 59901322032026238, 48671617606882007, 14152648390291794, 59633549484221794, -48609699321525016, 14930972767388013, 60964005743765976, 47554459636540991, -26419534668724112, -68493819755653443, -36536866289220058, -62144385564375686, 47905279814052487, -49896658553152991, -66564387546387528, -12535011828258327, 27577240479106022, 26685493357498622, -12832793941300293, 9528258161245555, -41469306381353104, -32919346595962581, -53499389895767953, -57484989737154418, -43513756644134342, -732280779102321, 14957728166275402, 50190989870820196, -22089581272713532, 2480294482753622, 51421351623222069, -56000364403477974, -10784654935961688, 52501510159513381, -16553215693505594, -34522915643223463, -15152538012531308, 34777203630241863, 38490553731154517, -60680482181269371, 4388933278873409, -41808942624492252, 921858256774014, 26722643151563968, -8440803814101102, -50923044542271019, -15270081785020061, -12579927252104091, 57803311648672213, 12901252809773048, 31436680089858323, -12758485089463564, -16219265650984775, -17303196208561934, -70514219800316900, 26855412074204723, -9861874450248559, 27317500530586738, -20596530746183480, -31739429184712439, -32367125863075721, 10752949596748732, -41151625982910443, -33850030543197255, -15213804290862406, 26823601413831212, -38628709375484805, 35547584010052937, 12735059089885117, -9590533478537313, 3127774438334545, -49857189866704625, 61963883793419939, -38431140987664999, 64361492901353642, 24330853954490767, 50103163558663007, -19448724132805341, -16726533121081537, -24130843340450428, -51551888474019436, 28953487485213473, -45903763380936638, 52411907070241516, 65408224723288538, 4819365105414661, 61038309566750173, -43784949931089915, 32888917170734805, 2630214694709954, -27559390718057422, 14289762212877938, -69932564563449144, 64345849849803742, -35799254513558413, -28807366731724142, -47245915266535933, 58452360486451202, -21933453223077984, 42401607318184987, 35694326812847286, 6687230386873355, -52593349993221075, -67247395763558596, -8768644170324156, -46382105269377570, 18641471322059048, -23958429617436551, 36313869773317161, 65989413187373736, 46864726131074009, -64524627089299714, 55638734848665493, 31428486796407826, -43596285872307044, 30119156023368075, -19386436869991298, -9879105028609193, 68511993660251117, -11099502612656374, -22786852947620195, 11858473192588341, -29915716905676786, -58289938314046246, 9271782784765988, 47376714836848859, 24596057172561828, -26517535674782315, 54052483351526096, -21934497825352258, 41586856414788302, 63646708969774844, -60769642897936421, -57468692889548226, 15077119010913354, -24624489323376087, -59766567882515328, -62452255190535348, 38192034419723639, -49652898674774370, 66427634797639400, -47184910896341395, -6434069487929844, 36085864177648225, -3699382505859880, -40729282141821225, 70315567984380133, -23737366556444899, -43020128572474355, 71666858935798973, -25052616596228092, -13005841415213301, -24226406768898473, 5330215732872199, 9407794729896272, 11408074953145799, -23458763622384355, -19728861450478273, 56934895651649773, 11010594664941858, 49729090906150763, -11883549438493816, -70087985387287058, 64382617089789491, 34350874317619147, -38256777299628412, 31700020498808085, 15315978419073463, -45102010010126992, 32379571611147010, -8938332229752288, 3097320517448808, -28712777586417693, 2780837130641148, 46940876632266322, 35479330438939470, 55371033339690567, -41913859813117072, -7952571487674511, -46253291633829729, -11901824974995442, 23522241433392549, -12033053394600478, 14018001525234279, -19068669863287935, -54138151520481934, -8533963840308953, -55268647518737571, 58475132693064748, 44354542538675931, 51177623405061295, 5053748828434733, 42090164249579729, 9689008381111031, -42183273437349086, 21873420007099351, -55080183565264337, 66993463374944747, -40577026712580529, -43255880481968010, 34170338898148550, 10116193499152357, 23188398335246344, 67995927738598631, 24174167075394724, 70121143349030785, -56984883863671075, -3394830374273993, -27417080542124148, -22535970369166656, 10997540548374717, 29439754005300246, -2351483213845321, -990760639933044, -51212161910533881, 71396253508593810, 1178959076404968, 7348046357333231, -10938070845316477, 13130836932097326, 48576115095634298, -13008633836281020, -21757883076237839, 33562918865779728, -5588678591163879, -29959306002204086, -47673091849496799, 64332099500755792, -7027956427406209, -2841252904788716, 1091628506371305, 68932288864105123, -4390180435633215, -31231231854762939, -10205638019993291, 13325223279639861, 44480609779511628, -42688655727934653, -66673388312789386, 23671257363654317, 18052743146151484, 47530064392616147, 48091933967226445, -3121140802615258, 44489198053744584, -38522678257912023, -48284005207997666, 8376296593028992, 48438755934777938, 18630984914150453, 24117776728106727, 1355612037838175, -56338819504222451, 38729925093080717, 18640805531281866, 4450176829423088, -61003979481998665, -35318464764153006, 23273941700811011, 59543445085308198, 24902318601221062, -40557143631993366, 44556831764879166, -41074770837621927, 53656359212145880, 27300009418099260, 38691797305753135, 39093641395153046, -22730323170934769, -38464141513883508, -2530181643656658, -36983466911545192, 7410317706545173, 41787944893408438, -18713841699570601, -14079255503773045, -12230930446400220, 54943284706293286, -22096734480452000, 39247339673709390, 49481329196138470, -56540871196875485, 5469177059655472, -44092469415533014, 45010822182301379, 49024389090818766, 63778555519175031, -18051527216927876, -10375963957520629, 55748450069045807, 53358501680658218, -16244363820505877, -55788254649512519, 37789734054973971, 54817272733622962, 26948473414394359, 12602919778637798, -4328489698298954, 64244998221501237, 4229827451395543, -16234147273099904, 36889424320088911, -9812062507633305, -9827892713493408, 59626202710654689, 5175409281921677, 70610657799843961, -39665117169343534, 50900682797217793, 25818785147235017, -1795125654878582, -47925431225807279, 21282571789071410, -52346220577491422, 41966131231181992, 10016770636071909, -59601978280050629, 21234698587044925, -47913419059224815, 69424172160086714, -18059185859087583, -43318835301321776, 33892699080658583, -51589287710362125, -45627609826348335, -15447109191230285, 45533937165768293, 22739088942875959, 47724424778029874, -21010123785278834, 63135229248365900, 34343544180130325, 6950938575120315, -59279685579389045, 8599036919087683, 22744197299274098, -798941724376649, -24672737439801739, -63948670059366672, 6153717941789436, 71596540002721184, 3070833449359969, 23617467779186498, -15735758826247496, -30793684994382189, -30720562664925778, -13948472816425402, -48330016843783751, 9567888010566763, 31197036886604745, 34745171116823814, 69907192426637560, -56282571600737562, -29940464996778257, -21369497028489164, -22487407568663965, -58293483208968926, 47598305206716464, -26322780670817076, -7263015272446548, 32509583543049899, -14936681749003114, -7228129208628366, 38702867234703452, -43319996517289633, -25369561644039595, -18166806920273366, 26105952801697364, -37981578850539880, 68057841230384961, -60929510810930349, -50633247960429681, 22810841962381692, -14613563677010583, 11145186761280676, -42554303702786078, 59971340426761974, 52524220684205221, -19325177182576100, -50338761273578148, -6486614420582879, 30454218788905809, -19330231741838425, -45653726496682475, 50223108921734200, 1461381547761694, 20276807032774518, -40546412151154573, -26512999779321482, -54729793218596261, 47175744097961688, -23777380065014114, -71863597753572232, -17117253607603859, -44013096565111642, 18657238009962803, 20441065124529764, -10977183000110241, -49979164790906335, 37351691177059911, 4695854964088734, 26188989944639760, 12898956870222749, 33906885919561307, -44333783263419719, 20842074741613731, 71608546306159267, -1741787662922660, 69714732053949940, 19644058191641370, -60560754610977773, 14357071032325727, 33839756503488441, -66408844993505165, -37566894172529228, 10777444338545317, 44333084999980701, 63584554424292957, -681244675650333, 43166862700216522, -70809998834458142, 31444048617221864, -11137121728678420, 27230643132201743, -25887908832622088, 8053216872275382, -46527231473965185, -28713449739620703, 3766773690182907, 60574581882545280, -26273797757989558, -35492147030414109, -19843073067299737, -60431361346156132, 28625736929508062, 58104616224267528, 7159652489106819, -10763983378097185, 9174514249296652, 66346231689594550, 27212058673992161, 12817645444869781, -51971061030679908, 69310827738092429, -7352451711957414, 67117285103218875, 52309402862530639, 31209026854951833, 50965586252928695, 66540322408060742, -47277681263005527, -19964399323190233, 61681194918016555, 11115420893120620, -61356315601655091, 41216885448373030, 14333965147671047, 57293101997061021, 61853983369465062, 61524583886223779, 60810977743562402, -33193824096103821, 18399407749431395, 43545584133656697, 36635677824728387, -59109422048675275, -25888534542747377, 59177265249081475, -70248555659835001, -44192956990712716, 68499380533858314, -26360892768115106, 23159912479698504, 39578562265657304, 36840958074872707, -27240065380716428, 18421065483544237, -4137308260910127, 26194993412245820, -61845169882282993, 1984750089502519, 19552335124160542, -594125561914319, -340075873228679, 33970966118464272, 20060361254906303, -37877615471353516, -15199382752908638, 19477298943808606, 22675761892387822, -17073376680598804, 67805929773017948, -6462110862077607, 27758610663010871, -36529375289964985, 27817665272413074, 36677332251009105, 28024993923673024, 20311357474597134, -42850361854077004, -25096885790668487, 64822950042803200, -38599455491385865, -35923489480976519, 13436668785144879, -56843205470736582, -37080673344323315, 7370263031410874, 57033359240210703, -56966667747221395, 19313082255872698, -52083101997584450, 40077173697823799, 1795234810013332, 24255921250233934, -57556571990545275, -14075296380087246, -34533269149041582, 31317108611532908, -34906511792884713, -58451794625122842, 43541572952080435, -64218372723403528, 36179817164364865, -33703047704326115, 46987448278700835, 57199755318637417, -21122584559042986, 43395836687304665, 46762660292122330, -65093298950695949, -59180308672374250, 10241585384820485, -31312561723110441, -23460783812469139, 11249228173889579, 25020104534803974, -22829674721784998, 67584323706487885, -42155449511728479, -30314333947547422, -18506402327739230, 19659829968925834, 58460129079950037, 46734286588843926, 65732992646523484, -32787452365707181, 23064122612975812, 13241923155884352, -44084844718363830, -36033993728753555, 59251881133579336, -156210091796630, 36092682384756240, -37151355626503666, 50477137829598132, 24112923304586105, 52617650421755388, 44689154559848535, 41254104643858082, -46492963550348275, 29519574804632918, 31498088017145785, 49258794654325870, -5671401509625069, 17432753172100810, 23758058971803246, 13993892367788724, -25991487873461797, 65152008888916408, -30177118285761029, 62454843115656638, -18819740556315729, -67637239625780395, 10414540562415368, 36988488790688256, -40449372439011794, 65305424214555645, 5546267614185690, 69095924100645744, 54435385630045597, 46447982598832209, -32779249048765502, -14122488545351888, -63193911723353797, 35277960452791769, -51892316826701576, 52617451119900845, 41086484718726764, -13679768303432082, -63409662282226190, -17180827083064351, -52600590690910930, 5231663700045124, -11266111000853060, -45114082382230951, -19572772491092468, 31465366904421840, -70452669721263997, -65532718038470661, 20368737472113735, -5383494450690237, -55248769525642672, 26761691474548362, -20158150017065062, 2754763573264337, -1876282004792726, -51965935776501674, -15010633550612288, -34023662840636300, -17418058197031172, -53599899133103398, 12052117124742731, 68007316989690173, -6101961925329046, -24804623930530029, 57980116953383782, -15287774566009139, 1175886783825093, -32659043750652774, 30094313813055898, 23497585177120022, -15695150163644480, -28370737295197574, 23996342167171597, -49145303139556482, -10265826808010715, -8138614623785412, -70909837991664769, -16166687819579173, -48325242752615265, 39467325228039271, -10895284300846855, 4657985238388919, 21894292933513970, -12387134483699257, 5913237638508459, 63303425682498856, 53573346914838884, 21250256356068162, 3717402294225408, -59694504424458132, 28761945646255045, 25395288259823026, -20959053912292017, -25098233119313305, 71411168403936433, -60539323766656599, -60966985493558641, 5412623625326494, -50515873291838060, 21866511201567823, 53211905158188896, 58739354552570195, 33557357375853443, -54684740524991588, 2887442432683272, 500182269449723, -62716422710518523, 6703096202244251, 55578467472241451, 5262028581943637, 50296206587746939, 63642911783081196, 24634659559507287, -61910239881206369, 5650893835012662, -52793015510034786, 60746465735860717, -70455676465744667, 430450927252211, 44515478220076842, -26371343462318024, 58897895916602903, 67404628655677058, 46643561288746262, -64398841244134329, 41205376615546456, -55470665157783118, 3828371997177713, 67107184489946793, -70348786290728933, 13671004823805805, -43257891486098946, -4606537635646474, 41891838038804969, -1370396551195662, 38588269525967427, -54977117816634453, 51969221205654269, -34563091117131351, -34377101171228776, 44432455830105296, -67078170584091078, 14389754665365865, 46822967943798872, 47678149682083608, 35868770050744251, -30325294276428738, -65385862374027308, 6182323894472186, -67934985088752069, 21069669187996024, 9114078018200530, 42707542736748594, 57444434950812535, -66663529647915037, 4023657963626082, 20027494456361617, 33820782851995445, 16765179482780029, -61861938762572612, -49023744655956167, -7992859169345663, 3773253897501354, 59527187111815872, 70660206115338785, 44464431469953836, 62397651296988438, -68560486508923179, 1762015172906314, -46770489867598235, -51546030787911513, -61245508036077646, 40187215326046690, -40546374144573179, 27799304405683128, -5893617463525949, 43944143470560958, 58658137843218017, -67371890313424315, 38840687311238671, -44494141141072015, -15716772958824795, 44517674882303050, -11898233028057305, 54132928217052748, 66177055403606387, 47606347609569137, 53589602553523253, -52048734512539475, -59500483698439481, 22308859952956244, 33152550174692462, 54382048672371064, 132727124818735, -50124501975770567, -60046943220059427, 51088154402687601, 37784347815861707, -45111799479065801, 45779790506592260, 61427349557960829, 20572794457797555, 2490140801140708, -26799518931348769, -40924929798964835, 65944323337585045, 23010192562213907, 229681612929150, -34106998314827434, -65762549379445778]
#r= [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1]
#te= [60, 20, 80, 100, 80, 40, 10, 60, 70, 90, 30, 80, 0, 70, 20, 0, 40, 40, 60, 50, 100, 30, 60, 70, 40, 60, 90, 80, 100, 0, 30, 50, 10, 10, 90, 10, 20, 40, 20, 90, 30, 20, 90, 60, 80, 20, 50, 10, 0, 20, 30, 20, 20, 60, 70, 20, 90, 50, 10, 50, 40, 100, 50, 70, 40, 90, 30, 20, 40, 100, 100, 80, 10, 20, 20, 80, 30, 20, 10, 80, 70, 70, 10, 40, 40, 20, 50, 20, 60, 100, 90, 40, 30, 10, 50, 80, 10, 0, 70, 50, 70, 0, 50, 50, 90, 40, 80, 0, 30, 50, 0, 90, 70, 80, 0, 40, 30, 0, 90, 100, 0, 10, 10, 50, 100, 20, 70, 30, 70, 70, 80, 90, 50, 40, 10, 0, 40, 50, 90, 0, 60, 40, 90, 60, 50, 60, 70, 90, 10, 0, 60, 0, 30, 60, 0, 40, 100, 100, 20, 60, 20, 20, 100, 10, 90, 50, 60, 30, 60, 100, 30, 60, 90, 100, 30, 10, 90, 10, 70, 100, 100, 0, 90, 90, 70, 80, 20, 50, 10, 50, 20, 90, 100, 0, 10, 90, 30, 0, 40, 40, 20, 30, 40, 40, 40, 20, 50, 50, 40, 30, 30, 20, 100, 50, 90, 10, 80, 60, 30, 70, 30, 100, 100, 40, 40, 10, 80, 0, 0, 70, 70, 70, 40, 60, 80, 60, 0, 10, 100, 100, 30, 70, 40, 100, 60, 10, 100, 60, 70, 30, 30, 0, 10, 80, 40, 40, 80, 70, 40, 30, 90, 70, 60, 30, 30, 50, 80, 90, 20, 60, 20, 10, 60, 100, 0, 80, 50, 0, 60, 90, 60, 100, 100, 10, 40, 60, 80, 70, 90, 80, 20, 100, 50, 10, 90, 0, 0, 10, 70, 80, 90, 90, 50, 60, 30, 70, 20, 80, 30, 10, 80, 50, 100, 90, 0, 40, 90, 100, 80, 90, 50, 10, 20, 40, 0, 0, 20, 80, 0, 40, 20, 60, 100, 90, 10, 30, 100, 100, 100, 40, 70, 70, 90, 30, 60, 40, 20, 30, 40, 70, 70, 100, 20, 50, 100, 50, 50, 100, 30, 30, 30, 20, 20, 70, 40, 80, 40, 90, 20, 50, 80, 0, 20, 90, 0, 60, 70, 80, 60, 10, 60, 90, 90, 100, 70, 90, 60, 40, 60, 100, 80, 40, 10, 70, 20, 50, 20, 40, 30, 40, 90, 10, 0, 40, 50, 20, 10, 100, 40, 90, 0, 60, 90, 50, 90, 70, 40, 0, 0, 20, 70, 50, 90, 40, 100, 80, 20, 10, 80, 20, 20, 30, 60, 70, 70, 10, 60, 70, 100, 10, 70, 60, 10, 80, 40, 90, 50, 100, 100, 60, 20, 10, 30, 10, 50, 10, 0, 0, 10, 10, 30, 50, 70, 90, 20, 30, 20, 0, 30, 100, 50, 100, 70, 50, 80, 20, 90, 30, 40, 50, 80, 30, 30, 50, 90, 0, 30, 10, 70, 30, 30, 90, 90, 10, 70, 10, 80, 100, 30, 60, 40, 90, 100, 60, 70, 90, 40, 80, 10, 70, 50, 20, 100, 80, 80, 90, 90, 80, 10, 60, 100, 50, 100, 30, 60, 70, 50, 80, 60, 80, 0, 60, 100, 50, 40, 30, 10, 80, 10, 60, 50, 60, 0, 90, 60, 0, 40, 90, 10, 80, 100, 80, 60, 20, 30, 100, 60, 60, 0, 20, 80, 90, 10, 80, 0, 70, 90, 10, 30, 20, 70, 20, 30, 0, 100, 80, 60, 40, 30, 80, 40, 50, 80, 100, 90, 20, 80, 50, 0, 10, 70, 60, 0, 70, 90, 0, 80, 70, 100, 30, 90, 20, 60, 30, 30, 90, 40, 20, 20, 20, 90, 90, 70, 50, 80, 50, 10, 80, 20, 50, 10, 10, 70, 90, 90, 0, 0, 40, 100, 80, 60, 100, 20, 40, 20, 60, 60, 90, 90, 60, 50, 20, 40, 0, 90, 50, 10, 10, 50, 80, 80, 80, 90, 30, 10, 50, 80, 0, 70, 60, 20, 90, 40, 90, 60, 10, 100, 40, 70, 50, 50, 80, 20, 100, 40, 90, 80, 70, 60, 40, 0, 0, 70, 20, 50, 80, 70, 40, 30, 90, 20, 30, 20, 40, 30, 20, 90, 0, 100, 40, 100, 50, 50, 40, 80, 90, 100, 50, 10, 20, 90, 30, 20, 60, 70, 0, 40, 40, 0, 60, 100, 0, 30, 100, 30, 30, 10, 100, 10, 70, 20, 80, 30, 20, 100, 60, 40, 100, 50, 50, 40, 90, 20, 70, 70, 70, 20, 50, 0, 20, 50, 80, 40, 30, 50, 60, 30, 80, 0, 90, 80, 10, 100, 60, 40, 30, 30, 50, 90, 100, 90, 20, 70, 40, 50, 30, 20, 80, 0, 0, 40, 90, 100, 40, 100, 100, 80, 0, 90, 10, 10, 30, 20, 20, 20, 70, 20, 80, 30, 0, 30, 60, 90, 100, 60, 0, 50, 80, 30, 80, 50, 70, 10, 50, 0, 100, 60, 60, 40, 70, 40, 90, 20, 100, 80, 30, 90, 20, 60, 90, 50, 30, 30, 30, 90, 0, 90, 0, 80, 30, 70, 50, 0, 30, 40, 10, 50, 40, 70, 90, 10, 60, 70, 80, 0, 40, 90, 0, 10, 50, 60, 40, 100, 20, 60, 20, 60, 50, 100, 40, 60, 40, 0, 80, 40, 50, 30, 90, 90, 20, 0, 50, 30, 0, 40, 40, 40, 60, 0, 0, 60, 80, 100, 90, 90, 10, 50, 70, 40, 80, 60, 40, 80, 0, 0, 80, 60, 20, 20, 40, 100, 0, 30, 100, 0, 0, 90, 10, 60, 10, 40, 0, 30, 0, 0, 70, 20, 80, 10, 90, 100, 90, 30, 0, 10, 0, 50, 50, 100, 90, 20, 0, 70, 50, 0, 20, 100, 0, 50, 100, 10, 60, 30, 0, 20, 20, 50, 20, 10, 20, 30, 40, 20, 10, 50, 70, 80, 70, 50, 40, 80, 40, 90, 50, 10, 90, 40, 90, 30, 70, 10, 10, 60, 70, 20, 90, 80, 80, 80, 70, 90, 10, 0, 90, 20, 0, 100, 20, 10, 90, 60, 70, 80, 100, 90, 40, 90, 60, 60, 50, 60, 0, 20, 60, 20, 50, 60, 70, 0, 0, 70, 10, 80, 40]
#c= [[-61093265028620426, -67518786063116145, 8106068320127782, 40778652210134328, 53198250960522113, 42966726326445206, 9740963712012226, 54650759244394445, 17836662040295123, -51387118766777814, -7703761073149302, 38342800912815214, 24490309752141966, -71101142483843907, -51525862242906016, 53156698925824985, -284232169037203, -33938547519641602, 8852022862315393, -26368770476378291, 31048431370877476, 65243249808920395, -17669488026360456, -40668977951255276, -5079353356907066, -68372550287990196, -29230005785616341, 17544647806603311, -37373603701624019, 31661581834908598, -14780682030294782, 65525297949484768, 56464563509381084, 11470856816299588, -36016089853231559, -43239648048751691, -65741580604349597, 35676825407849750, -20526536903503374, -16598843451182116, -43703836443944487, -4101857006633394, 29635172382186003, -60234574831128877, -4496215304093903, 59102142128849354, -55819077044996097, -3166275377982321, 41825525443696924, 65195837944498816, -17838531535421276, 26262075694829046, -51376020139281326, 65714365107335435, 59994740798459979, 69284676682689392, 9903029505580235, -21917177820231475, -54795161928783415, -23029919976446620, 15976907122846110, 18594054756893217, -30264405314051570, -41747590025593398, 45294056242243172, -16027855479352643, -63085068764818076, 4276343154909852, -51042252790560610, -10598560927979735, 26863239825078851, 58394318709666706, -13378009297350162, 23842517698621689, -51006788414282765, 23472314127518106, 67495067849253100, 42369781895120426, -1760125003345764, 55685188412115799, -56846540547854031, 7058466823303851, 23932236498157654, 3993421875965725, -64694107590223002, -62886916584667649, -68868333435618920, -25517483846403196, 10440432509396342, -31318764851052055, 33362669261803915, 66190319368130773, -72028287455534854, 65512911218557324, 6094571501214790, -15840948130925328, -47437819846219597, 27854885935623749, 22047185190547145, -7633060206422611, -52431866644536179, 51861296500110482, 53581352758528265, -6321101881415436, 65844184527890011, 38894372786259908, -11231286118013356, -45878999662732187, -25218080872755726, -65728364918518004, -11465723370197190, -44702807139501280, -43977010073426220, 8195781205417378, 64284364219423235, 11086538318518543, -26941153206059803, -33805085467546310, -13051432093932198, 21538339043912237, -17029484872456194, 49016392058851167, 40496224877085065, -12335004966525276, -18083614641164852, -16179126146469707, -44311669119043681, -8278204803414614, -70128940074644720, -4591946773864922, -61553635901130173, 9260513117069997, -18647170759044365, -5161166085289504, 31134838843537061, 51390600607753343, 6185536357414045, -17384941636586929, 36620514729762487, 1714915044488428, 1693072022390184, 1498947893440987, -44985302985938094, 10941462292711983, 32427952943604315, 43096254385899928, 62688980838924156, -58009228169405162, 10140624313412747, 66119750218177921, 38863520136969512, 10872051098036634, -30939752462387463, 5753915682153744, -48744147423737014, 43879832823153431, 2560845229007849, -41565641878167117, 57797110953480293, -18115783312250958, 54011500253838322, -43762538656297140, -24349876441186962, -18145839839290774, -71689521143976831, -34308868253489768, -2478442907953845, -22366373074867791, 52888280121813983, -38467590834410924, -8873364212772412, -11135942557865463, 39650944494767696, -2071932255712613, -32688614000412032, -50601558130807147, 36994565781710493, -23787704379625104, -53387566821785766, -40614919745404506, -43698656391070749, 51922659711753349, -56677444762176445, 53254472262308619, -67291405360218116, -23180015313893015, 682622824660575, -31904431815917041, -51050064409067957, -12921544258739488, -13253818469188617, 64999726993138248, -40355985636363672, -48114943743254103, -57505973645151217, -7557450881066694, -13365268197845864, 8517515778891907, -6061257104039512, -36658619583801379, -3524381692318385, 54186782795207390, -52383815765616652, -27912142724213686, 53118360864918922, 30098403028524552, -50244452230370591, -6796872716073917, 47091585104952578, -38599226110139976, -37534122558296625, 31466713083660195, -36181690326139810, 50327705808396476, 4123338284994643, -3408455039723359, 15662383999561145, -4821516241206268, -25457671309186435, -41792283803770726, -35565146952732298, -6005048674640222, 7747860453245192, -36011733491761070, -45302679841725905, 71229264033470227, 36335459689137809, -61410734380335560, 11122527559281076, 55778734283563852, 10178165619752717, 46534284468418282, 60321722453211389, 23821656143505864, 37215753578636634, -30322745873668542, 36053778774500456, -15457788894626414, -20182803679464117, 20207346152809088, 32666574043052462, 65767123192168665, -62434011263802490, -11709150815075507, 3312538862484095, 6925351094657767, -45073325704002522, 14861577003100035, 57828250142561780, 29863787769510898, -16435567228644538, -65977405222168341, -42730169461557657, 60867506530938743, 34923175897335559, 2729849561734566, -37473068000974004, -38840602490737900, -38572544949856397, -59365318750252245, -65980574320245564, -12693165320080088, 30062720088788772, -54234131071817549, 66332260177756876, -1542782520927410, -10530115682408547, -51021864578689504, 4589663819374515, -35393633458335250, 5445667659337982, 29702145269816793, 25619907934206272, -5800872854038657, -38452178319352069, 32125091588052469, -19955373033027430, 3071529532309425, -2051876816977888, -9700174672693837, -17113403116473070, -17708022736170086, -24033366445261550, 13782711747323720, 70138108016150842, 54507918660554147, 57248666252476778, -22442188244060349, 41326401876624138, -21578835458579196, -69897720041138088, -6545268871667659, -46124270291503743, -12612650589534368, 59207457384299234, -35802030045358291, -65183174799839564, 32801162323953014, 26210558259843161, 26149487562846482, 32200040962996809, 65859524735981607, -5347026722765004, 11391199665576946, -16918449245908583, -17684341252230603, -64853082486847481, -4308638337055370, 57890238994417694, -51881665585477356, -21920423805937876, -55091452717737769, -58566138880823756, 55588880506071823, 39540219295138334, -64545610534856512, 35182991325973453, -6116134361325156, -70451411234311569, 14792857202202483, -30242621239813529, 53639340679310517, 23292897083994176, 13323260191711824, 33921333581512652, -37320856297005297, -70422130371832648, 42984298180545935, 17728191867666560, -37629138241669314, 14594565975882720, 21802674835424808, -56015629383701052, 41325780968740995, -7280623294444259, -29690869636835371, -16261160495861079, -25144548792346045, -22874676177527747, 21510178310794094, 29616197536659800, -11270230334902989, 13858555133956748, -39944004269166155, 21640894535579637, -16409819559573067, -67696584725071490, 1395835171225011, 44689218221520098, 70477951258171954, 5830659356045242, 58909195388926665, -59293737612146214, -6325271613161481, 28541274164865947, 63684652688592198, -53452581023892627, -62654416596820942, -15487334427092127, 38217703756860079, -49599067036232029, -54819828087762360, -57222768760489653, -69422948242932303, -41132543326715470, 45835657062237784, -50979583681723522, 14300674486804487, 48819807952527513, 63550482082133430, 27518219909758048, 39311365165632494, -45419724417383845, 28166264376388613, -37102498055712081, -56990121288180880, -38490613970654824, 55115192003332565, 38119562238097593, 62154191385597251, -14832170891332275, 4227491060351668, -29343178023526401, -3181965192762237, -11375158124842317, 47295642650773560, 42947679256857260, -20954739638972680, 46547903719765600, 70801004720836298, -33632703613818434, 48926827861762346, 33226265106831002, 54764418056590795, -38662542126126995, 23597098245063075, 26772887411041111, -35194666215530941, 51730750273010510, -61928059851893622, 13714267154162345, -64755909248010406, 36374820656793704, -31695230597015493, -29469528415681867, 19614341928459710, 31401453097315674, -46302430400145407, -65894100326881859, -13973249783088167, 67133519991615178, 42435523195845219, 56552028310536003, 3056015253416000, 44982465629577570, -11939941976686151, -27706460627048964, -39993300708855676, 42660587449378159, -22308137348719101, 22427405529355319, -1477623955020542, -55242018795248012, -13164880980286162, 55849111671944487, -7929635454449736, -35675861169790767, -15302830126901282, -4814114996173114, 59993223751122279, -23863088198474543, 1821055619128148, -22201402660963787, 29150511126039050, 35769800428721190, 1812548155678721, -30635183925332139, -10361072427361028, -44377645428119136, -13233446362992288, 42183795169172921, 46667567488431076, -10123551970455164, 46855787917399481, -55787039593282670, -62766387990661523, -27427905552651872, -11738007242150811, -3615415298886392, 13843716769257544, 57403774149777808, -65469608996748690, 64142478854231804, -46715837910751636, -10542035910628099, -68848800758297989, 26496922475120198, -48419537071702903, -67363535667756082, -18045981523714364, 44724612307654740, -27010865741862258, 42927288371329589, 39544414763461187, 31298125330091360, 20366284772237183, 29145782751273768, -70007612044198550, -61000476558839316, 14469218991637576, 66750580181507555, 8832315445843016, 12800074865059375, -55611398124444139, 39148790142515403, -21235354507944885, -6883919868508079, 42890060427794788, -63332016052294042, 35505642871091508, 6195026836292468, 7903000544194615, 67586231261684916, -37166810061657935, 24036091503394070, -51502243954003009, -35045371471552493, -68517203494237576, 14931356151135307, -22506258894942587, -36327404093905636, 21946125816592067, -45503574086176498, -11256908201686937, 13696659723779504, -20951532983727339, -15772305561821117, -34650204836087842, 22788500582027406, 61998610851559224, -47569867877120040, 33106583936197299, -39693771744656271, -31093501598700599, 36108171666287776, 45315702334526679, -60350746345269384, 31951063194684964, 37483187213195019, 39521637024908912, 47893004182626800, -37755810242701587, -6878284774856686, 7252271186191309, -38698157395303866, -25431196770456105, 64532540213805085, -10783603457834225, 957980651553936, -24878169417382053, -8621949798917679, 39163617290496943, -44426795009880485, 68940654735173225, -13735763883650645, 71585193769435685, -17540690658058722, 8178672327530887, -17051190344100691, 32889208806778221, -38192374747962445, -26084761491051714, 42938816323710248, -47401299131578864, 45434714244592415, 32089388237673938, 44249162193772007, 36189381639097752, -66130240648623328, -27396166737887765, -40945848048939585, 29078302308351193, -69493040647660764, 62635676953660088, -38508318551605037, 70309797116058448, -53546837571587351, -16687001288550703, 8017403392764711, 40597147638269684, 37309418878627380, -38124178630531176, -14182524470144592, -38916390212014147, -60852431667297404, 28022145329826727, 35095611924908096, 7394650996299696, -23073726653757636, -18488272790036840, 10073326179460221, 50082233216180199, 45358739819207825, 55817800242334680, -68491780128134994, -53595601427111431, 55968752096616508, 15360065085113207, 54055205073324936, -37956357788228441, 61303707930352286, 70031680952890283, -56383143611923809, -71943517401318669, 14891770607094779, 12662365681935163, 13115282827422792, 55768164332838225, -56460017260918353, -66918427199718300, -24580140840290257, 5194182692193388, -36452513047206800, -4540555951499909, 34359589316954975, -65742757290690233, -8360793633561506, -67949876352306347, -32930113950633939, -1439228016146587, 57702301128983397, 69161440817947713, 39210763598962516, 70883665335251835, 46312907085945412, 50249400062729773, 43980348391965237, -14130780474812036, 67525419598626225, 18454111501959034, -71314415374094792, -43650814355781494, -53936595711723258, 28426785864871624, 13124217189736468, 65537638624503423, 35652559651049519, -18918264269151329, -29060675360837534, 20071647415086499, 22976883591079722, 7649386222006216, 33934567612144265, -40734835669509685, 56511600018738660, -20093526309077704, -62400510443806371, 43407047031068267, 52917109268603349, -43629286519544406, 21973349056598099, -68336419327622083, 6831169980087968, -59049969327190297, 42588963245520631, 14933011165962265, 54826117773456277, 4081741399519869, 31968609534391953, -8591730564300939, -38882453698629641, -21322969526059140, -24371714423998915, 45740446898417135, 3702754220150594, 48576758559459573, -63429474383899179, 26776203878558143, -17471105404964753, 17091744089843196, 377640214671053, 39642611654257224, -61781539851974889, 39231004857740008, 35983493300492768, -44337646165424368, 6258441718487374, 61996616817808185, 67637100113869155, 49382794875052340, -23879048454544069, 56126437442031345, 57794734312898323, 14939822137758478, 43763606759063521, 48606514889363248, -70276517086450630, 983213673679796, 12842128086597826, -68107995334739841, 50141727692698792, -8964099285529153, -36439448681437772, 69966686789877629, 40754763101719920, -37979777607537335, -11344049603112818, 52814148554759958, 69721439080059277, 48936545363637275, -17701730053910539, -6385513603569716, 60381467692491676, -6024997345170468, 9221290096824428, -25273369918245847, -2181354421048023, -29950293340825795, 32537254720981791, -39108642074397966, -26801606750063686, -37947426276391004, -38853852325515135, 69931310104229157, -54576497033277533, -6450263298321704, 42994817148915287, 38663182147219252, 20948832754400132, 30911589461778612, -5864927918330626, -50295697126784078, 58202222593285911, 2462097454488014, -69861312646077714, 3448117370509316, 62846416496101631, 24910024329784894, 1117896566154024, -62059757874996434, -58234252000277520, 51979498151083557, 67528622128499579, 27586918091857878, 23578682564336153, 28306920685274336, -35291122572881358, -36760260220943031, 9071882261096406, 59529749876085204, 55848720335462684, 70928617278341667, 31412624258310519, -65567551958556655, 421680076853968, 30482879853404755, 43706994343038804, 57768619056800895, 49751403799910706, -25474441334016277, 41360453963601343, 33497626338378222, -68354320870678908, -67065366810698843, 19434150549487119, 43744214376398574, -39708556670145335, 48819415339824761, -32074388696637999, 62728532821388467, 8465471778621637, -41754233064376428, -31332466087962252, -6060869953393181, 54117214997080165, -60659019513942795, -6662784561011676, 61885764101456228, -39727029500118569, 49944455262166908, -57733776113985175, -71974011885300019, -59087034624773103, -61499144980175496, 63908865809021228, -32743078364257841, 33950260018896249, -8079890512951109, -37232317216304814, 30187029559851569, 26789827805639159, 25079391281449012, -69675381864413179, -44327725632563443, -48356695918819121, 42686879539534520, 65911982820510882, 64606084642138532, 61325503724547838, -43933093848728321, -5651833917243600, 23617624778000361, 43817602004577703, -22325399828079042, 62660853049709677, -63045320981274059, 6209627731731191, 25161356576851656, 59162745957653283, 29110798824311345, 68635788932631274, 47313806409308875, -39104988967703322, 2177096890834426, 42500663904370176, -21694706097585269, 28978616536593193, 53692996239131237, 8233470029262054, -24764652021960800, 42197854023159029, 65238069523233316, -67654691319794971, 43375859377838119, -57249879681128309, -14377733016173909, -49275210967792053, -6487921288790885, 18739377126323912, -19766417065926687, -62488687466467831, 28688963405682898, -16020960004291474, -7148864633686033, 59643990833484667, -31365935369374536, -44472862837967142, -44430201961799936, 54032603902515825, -64051750351145134, 12951242774785243, -63318863238026872, 45069273577388883, 48994110130975476, 16056658166114472, 25721053513092014, -50858184516468409, -70325660141552474, -10079413909173009, -32049345232993501, -22789869771289169, 17044662001364453, -36138358969041871, -34850818929844954, 38358668288209700, -34781546506485344, -34429148075653979, -70348520303391615, 144584038847746, 36713842241467236, 44920068568876716, 31152315822870706, 61942040655333968, 58759553379557766, -47779773787275985, -10249167901597002, -21558264804450551, -67281566917848741, 19946723038085971, 40703711335165932, -69102127769550673, -52519302095590103, 26360420701840558, -41929306555393705, 66602718912269809, 39345110404424178, -70852956287512564, 19828619947581509, -65078017750667601, -13471959448327315, 8231740248505885, -20841588449824025, 46883402550282127, 67652278534712093, -13753103654254518, -15312203247676063, 38312150239148445, 11583788730865488, 35800481604483342, 72007325413932415, 4188614569138274, 21094691817987634, -26143974708190055, 56635547428160262, -26840549660780607, 59689643627134623, -56441703638037429, 33645302951891531, -16509125094812347, -69787060161722335, -5833205815296265, -1085811395563137, 18598806308442526, 7354914687472179, 18005531575834963, 41942037090703964, 7844405561332067, -57879475774755707, -14842214393828395, 25422718254294717, 34030906313896000, -23862396596651422, -6073588612050994, -31298777738754289, -17770744612761788, 20586944063226987, 35430960906826353, 5005388182565102, 25390292801137827, 63916444206390387, -70301212558349456, -24624021190092351, 8019933059888722, 55799669191709381, 22511285680100713, -65185497676820166, 63715880610399192, -55640586874900431, 44939311922126251, -47028995178682910, 69057261785749089, 38573671734313886, 34119322948925325, 13168500499951747, -56748451539564364, 29424446732094560, -32435309485614429, -33320017196554843, -56690220049384167, -26755838953482480, -29978495781054, -39305982995453649, -51785846421514987, 776182667789775, 4084617129589161, 9969268296461196, -18359428582995151, -70561191426336012, -31907990221461877, 27163189737635106, 45665452688906420, 26838315816787445, 10578192548277397, 29730610740738533, -43713741318318361, 59712712427736658, 71983247005301788, 63513881494841664, -27769728835745516, 68614762813227929, -18129547863919427, -55509816127815356, 31166292823457194, -47877350851719565, 51596836868714901, -38230877757741446, -26088254960485386, 41512946076172678, 52926250990252574, -7651672760297139, -65735934007609036, -47327345370526917, 25372934859246788, 19968201173034585, 48148893446526990, 44174076122017628, -70952251722911068, 24335601416228018, -53259296532615313, -66564084404185482, 18594273226232001, 26326723781312467, -22564783165483667, -2189811900636955, 43894998899996888, -68358662676363410, -70927109081147748, -3029939078143513, -15710726482248589, -19174185899592212, -60309902071258291, 59499522026550660, -20622371691581225, 68220050143417198, -52753660546562087, 42848039085326851, 48908287119914235, -45973358360205475, -64112392310344410, -35918713498864287, -69375742158014731, -21273909725140335, 52629927412175495, -26327972960445097, 19478184584479968, 46029411478819219, 56625376356278331, 4189850407221786, 8850466206086048, -7057126420748128, -10493802770364242, -12104360903432633, -49664881349213715, -70150997128974354, -43599480559560628, 48292310219363168, 19681056108702917, -11061304328778217, 50591581869134517, 18821051100717102, 27046901079148661, -40840783679254938, -22373862746110514, 40663992030086561, -56954721286698449, -12041080024840291, -47409417013689671, 13598717802217364, 68352407548298883, -65039262102976992, 28866876511219411, 23443108339529424, 37657971535609623, 26257270779999002, 70589734992749500, 21945533251616482, -14011028033399261, -56509336310454735, -71684625645004897, -42833127995499295, 31477908655323204, -3270563131949059, 25811523538191642, -30585275677645629, -24905560591250903, -60146594404306485, 60610905472277524, 34717382295775171, -50591915228141380, -13208652025613730, 20515263907419924, -65418733501289743, 17853399049334475, 4083169335806525, -20392918304683589, 696252723412373, 53824014571480410, 40214747008967744, 36894819168851826, -67604341109720429, -21817007371221151, -2469666285417532, -42923956535951162, 42870533137310150], [-2784530595486327, -38439163993170411, -16877874021237482, 37009622260219924, 37527514076842943, 47707153101440021, 43996092310845111, -70597477155478910, -21209851466606847, 9327296310507275, 29196796858330180, 61713952048423622, -41755623668492003, 5032782903855251, 31965355794425985, -386468258935976, -38516043132767884, 53888579846353702, -68750357875091105, 61649498948749436, 58085567189225372, -65624011586341509, 70796088657084668, 25614034055644854, -63248478148254712, 67658015725271609, 69110670097864258, 445900260041075, 19057202817651989, -47681621246988081, -29375901902534382, -5599562153043006, 22399991041617903, -48547477206694653, -64160662033478705, -41118028592594233, 32238908393541555, -30556145895106706, 33114527701950894, -58054421498506576, -53182525643356256, 20561349059438986, -18147258491452599, -62901785219875897, 14439258522437992, 67567908455647019, -58659006183285050, 8217868317075546, -26359910074215086, -70490820161231965, -49716826509745366, -5709981312186708, -2971684554945370, 53431333031562347, 30215951224684004, 42398070076587554, 3912193073887604, 67300134622137368, 3415246271039611, 58962850817859693, 34528163712119027, -67054977751895258, -698583570288077, 41223834612080458, 995584090024202, -62467602273030709, 13408681828503768, 14474070125211582, 9921497534361037, 39149750854818772, -3313213317414002, -37076391168924494, 10958491413420451, 4687491608935785, -20982443057228087, 45753566707293259, 8488673384713073, -69455354029889890, -25350966062607581, 39617114476968779, 2164134732231358, 50263952740750233, 33531663803718471, -11083283775175342, 8010795706312497, 57103522497722782, 14480558428246159, -29068515289902609, -380631482672759, -20734897801162287, 51300769082163104, 55875168432775765, 32584027703210288, -67012944447574092, -40491750553312043, -56529253785587635, -1814385537906886, -57953226952976984, -9130696424097257, 63922394526531240, 68895901785151170, 59987771399557585, -38849906992805522, 864371960469372, -19630907478996890, -16029771706975723, -9351939215017493, 71212172957620161, -11736859561105455, 64622549785111502, -48404597600411215, -39980253199898737, -58325669458806189, 22718476300457313, -37988806392669373, 43470303929148780, -34435549778845841, 41475405020204878, -41897288476551723, -36587720726221772, 49203218864023292, 15997543086696605, 50267857008268957, 44056015426616743, -59364983569579095, -39625538011038708, 23552511921814844, -10428700931031618, 42481990102791316, 58217550403008960, 41024672983464110, 20587958923510763, 46844707334120045, 68554945959537389, -28699588402757085, -1665012469961631, 20176410598735317, 69722329830332229, -50977240407024776, -20383498936279436, -14390658152553763, -17834705801152307, 61526086518270495, -6792170198816640, -8252811787453710, 16118221356416936, -56839253856994653, 37888104379633574, -9675759655872909, 63510865286897216, -14296473573562530, -4192833910304561, -51771456905143742, -27244560431940598, 36101585294331718, 3572459447369651, -61151098693904942, -14928795948355746, -13887580215699965, 5706580511568266, 13652768958143258, 36542691227233768, -17404218811438492, -37004801669253738, 68418781994507325, -7913073159005710, -31083460689571991, 19561214339545664, 55841476912667403, 13420968379438767, 8497352209562385, 45299992941262823, -61925178967153539, 57863001337943962, 21289952383092395, -66299098788772544, 12046806188692120, 50257373557132144, 17470466691301343, -64981482023977786, 67807549155416869, -5626770238419954, -47330523044885247, -8684884079224243, -23112862355529523, -4855720612436167, -13488875425891600, 23683000884635403, -17768349294151306, -28082295977663545, 20983523871954193, 19201025777321199, 34410089943699233, 69171395822953627, -19971747071778033, 29695033385124821, -59333430575733776, -3325250002573937, -2985388833117597, -41426810930876228, 34149473920744034, 67648373056292694, -51044309337628522, 1597334928386735, 69649730605335947, -65124555796630451, -56227360333796557, 17176717936864795, 21431986262124264, 48485528499780582, -29459549210376466, 11272186706375074, -69504828075938652, -57380338651438283, 11827594071500793, -12911031063759507, 52538408346119200, 14971662446449235, -67623557663793656, 36166094067286985, -32035188749152238, 21609069968347552, -55652195477446184, -5894517095738068, 31846843860540, -41510967946755492, -17617684876648486, -56001748122408380, 67197308188469367, -66904699250658554, -10721825746082158, -57283904465854937, 45343758493346964, -10124627794862617, -7525206059005033, -62151738491578327, 48207381891884154, -24457302257387222, 21154084006490203, 29370751684541170, -30174127248228195, -45764207790406542, -25111488580219139, -32401609707874717, 24166910480403533, -30823438479449261, 8713548907385010, -24472136999113385, 18499216980238968, 53667958519577545, 30676589438846536, -30027899156412778, -63271505319494121, 29117330535857964, 26710552735518214, 27281736342451121, -41763190564518363, -60927882080389263, 62847332438305862, 43176013064577894, -4435871226740678, 63464616458940814, -27953471612819665, -20497951527590142, -51439869266375995, 58030800795292294, 10297443506014330, -55033513878791429, -15251764766220096, 61151951464510986, -52904183768926708, -7110407467363323, -26834542548322507, 50084815742419386, -58285231234536033, -65810382879932797, -24646226555955425, 61363283844464512, -28033011368485655, -46204726040864730, 27864330891730337, -34550621022480261, 22479007260093636, 66109909298418882, -3808753197658613, 28937411581682869, -43044466252282987, -44847704628208914, -33974119224727736, 72040982845187833, -21020615059316828, -5622632707044452, -71323289683789699, -66200749814013478, 14440180169510672, 66207926666216964, 71598636554953300, -254395421667942, 48924811073724425, -46801478333465179, 18369179158184855, -17989939424984640, -22343684411143227, 7179985833656696, -26883832539912457, -50274140500801739, 43753324389890185, -48616466340389425, -68269502586039202, 46920162701011700, -968325342231484, 1999216175197580, 54920067737274149, 60521448057224682, -59636535522730589, -61653366906551434, -47932545887854112, 68693202125354548, -31567601130405879, 53547043876047914, 2918583362535704, -40191744635484349, -70692103073639812, -55622255189436868, -1047341675580703, 58228503305729040, 6853121161235847, -64281071423150729, -33811072227117451, -20951494497620523, 64118092360947425, 33630965652078744, 51635770544670360, 6872383245105048, -29587850856006471, -18806500374085506, -62990227239809623, 62357388836921492, 4021807460148776, 25001259804343594, -40901558195032245, 65928888014056614, -9521543117992987, 32496579420296902, 16428401008163027, 63111000349392359, -24195396386898596, 37180088449891607, 68889343237803087, -7847209756236142, -62782001774834944, 42737337252730855, -59308002720498781, -62702303853711192, 12804829397506978, -57129868069301920, 48801269371361704, 18980157765823690, 17004204837852568, -9818310527609768, -27937565192332720, 67193302270800175, 53970545058890755, -5442977784204344, -50770339318882218, 42781796017956507, -15796613585653994, -62647128522807776, 28788331707998037, -50380113133230305, 16812060954173425, -6339549548808280, -71675463850119897, 15554715038692769, 13734083843039648, -68750570522785556, -26781652991981074, -51980051950361347, -45562604040479667, 46707857761935844, -49331443769831202, -32513430693010093, -16203761585608465, 7519478979173245, 47373349020586113, -656553537918779, -22100063098544163, 8962978123054189, 59254283372489938, -15669243305305277, 12334501085014501, 67718771975727492, 30074177503076395, 68247480974481773, 45890438982939324, -15452671931815570, 25668796912063836, 54629029166720657, -23373951501412249, 41153949378541650, -7517732499357441, 13261748151680470, 7087488871999865, -56433113559034925, -25991021157640494, 28208351822176563, -35549797285677855, 7789072914361160, -22865191773968379, 35672631263543662, 41414702358010979, 20973512810321914, -29502919229921779, 53769600243349699, 71718744669089134, -33503037402938080, 53351149866319072, -53117423193157638, -37389131811348064, -44690435403146075, 40157353439569067, 6350049543114397, -67925510493088079, 65273199987957004, -51240446975557221, -20692017269905002, -50706279389572744, -71504536316797854, -4263266440119593, 14123865445468334, -31227352253019092, 3209992248612612, 64233516719852037, 16724543723703345, -7684651957022719, -24592198261795367, -22250625243616370, 18362334583576785, -26342407898699599, 40243855143579201, -56424638483974217, 69629891715175623, -8613477400926888, 60504377969776662, 43265199180046134, -67307595360123401, 60792003332161607, -27477922288715669, 25444620522135840, -70959894959186565, -54814502013157194, 17353478593706071, 51887835804943341, -55271196066776101, 36390528112694252, -43630780954865170, -23864257751572171, -64630144106057547, 24482352049716015, 10187166203290095, 40602977237531899, -60280557991288901, 61864498701214558, 18688009838618465, 33353898197839885, 20384754053720819, -48472179937755522, -40995525447474374, 21345851463752412, 24626955541759844, -71781582391191483, 66888385871536415, -6373027432395967, -56444306927047205, 25167079103621484, 16988906686782660, 34814852830255018, -6920901279240092, -1193596024128175, 23641400153465561, 2037022765787144, -65457685633763453, -52258685777819229, -10941055694405400, -54913824327164840, -24881893830253969, -63081170452685612, -58586070281665212, 33087514277785337, -21293400292019200, 23021029164399343, -34019400416770917, 45657923446343980, 5206400308106977, 38262926575464711, -46955375487849023, -50035293042156954, 45922318677578915, 38587234603461716, -49707499624811665, -49354349221869855, -24329866032984035, -70409535646454091, 457152794986627, 63431727734552280, -40712968194537531, 19784416333275707, 16765353318815588, 20860113983759055, 65940769415408952, 19728344248793121, -44089551805047843, -20279756855592577, 29642835276390751, 9075143307111432, 25844407753212698, -20626182873205269, 13474973129798584, 348079560163405, 66773900195379055, -54226485797059229, 44487908291601306, 70656694660836838, 71457404308516678, 19623974245731626, -23190649509938578, -45976437169050963, 6650601328778197, 45745876582972614, 59275826400082865, 13513042715074294, -55588537220875351, -23606804982556282, 58307150470384692, -1648954324895649, 61377678285573876, 12451549093293786, -58376844414839190, -818401448932936, -30839896667772887, 42701747999881953, -17716603473788451, 6715020980403159, 13781467163715193, 10451813679638704, -1186761420685260, -67825648788673998, -53907577278968080, -38996279000407104, -32018915387415341, -30949250651671832, -35012644017583557, 59542215493649190, -49311922863952786, -24852709237961777, -25804439810887268, -49976131482050500, -52515153879950301, 21927866924514952, 2295142733136763, -14382161925191441, 53391140201506201, -8574471960245969, -17621127178863599, -65421648168054222, -1686199371084694, 64661780095827287, 34959298435237561, 21007280821125037, -961373058923823, 15249927235584529, -16551782824132962, -50697772859071856, 61282282095004944, 15138750549388869, 29896849606278850, 6732991426441337, 62846996209965824, -54960079890429266, 64088343863374713, -18794872686124823, 11489223879445383, 54876288218931200, 16283880854913624, 6373831312794289, 12930985007169204, -39030199592001890, 11313657217240864, 42189036345684357, 27447245823344743, 9039424993120205, 22700371859773812, 8119428554065271, 24664167814389063, 34565383653488438, -1904988433480717, -48176376723141395, -29984546877799111, -21108381280495123, 48010398904810175, 22930816619791752, -43011300367055747, -16582884591747358, 55245978762251760, -46113174012451284, -15400057831511861, -42064689733308234, -22119515856766690, 28129936690351603, -60720615271789004, 21358048090629806, -397694955994850, -36825515165913250, -62358467248618655, -60366050961990800, -24503881049789180, 26461182057605198, 59203833592769714, -38338689249898243, -54723323486179, 25677478206013748, 54094170457395593, -52689201694521238, 16518601218380234, 18355333103939859, 68102058333017645, 6244321197492001, -16386942593643935, -56592425854851995, 20336231264379187, -39916293136996442, -41065907571203097, -32560345253230172, 17481254815806995, -41743584518688942, -17260842552089984, 16998231055709896, 55238133129098488, -12221531691621324, -65547780494946226, -39498948745399166, -15980091103043258, -5842252574163093, 46958379327683078, -38314170538991248, 59753499301457076, -38926719037209704, -51142638781212039, -2258892539621058, 56262587683116884, -44180522183784748, 16151860256575104, 9903756089125398, -52836344983820553, -30928810297565200, -42541784549884744, -22715744410011334, -48358204828968507, -28145318373127452, -18913366654041733, 57008221623711391, 2542600314631382, 55356087832850630, 61103067047219541, 64495582505487895, 69747065745271520, 11794014755146379, -316053043140857, -31189635040433264, 55265945109262921, -58626234154913969, 57255666420064183, -17600771851749283, 3647118930393442, 70170827917779633, -59916538985979331, -20836294832653877, 61975190487118375, -6821081495607669, 68149210294452689, 51729935905633353, -34270165059349397, -70481468497710896, 46704580048237730, -12171345435622964, -65601967444439795, 33301118134302037, 61922947594471163, -23352319944859481, -60338689683245511, 36324549569907376, -22694631565202693, 63443389669959468, 14020536632554120, 8790527086043626, -71830995935711327, 29250611681935963, 4559866299007374, 65359329520397535, -18112355372099767, 52552819781941840, -64947460625042323, 28112246285931491, -50291029278555883, -56882854523664715, -41822604732031801, -48782062525600031, -30244941997212575, 32781611342275477, -18612656659954404, -17894490020716698, -59647533784425373, -43645352618929066, -65050941775838342, -35040344820626821, 33660985957260702, 1806678536736201, 21191140293838947, 39628362478121559, 56902939647249770, -33395258671813201, 21442023450012155, -23735864983339528, 56559448961656995, 10561876286157398, 44814685943699989, 5154514803622631, -14711260548474256, -36576685256469521, 64755145293885348, -19880159996885605, -32487786439708557, -51774260376263776, 24825593289414429, -52615287724260412, 46525522126439191, -27632434162787499, -63419680636545749, 50369427724017525, 65368935700347184, 57506691887643071, -19480727823101462, 5416326588025733, 45099401251497420, 29538867401262509, -11543067378680389, -35755956248261223, -9795770426519679, 29614688669541536, -29963644494898348, -16133091308668157, 33507055913878658, -31786808619445909, 15628223019972134, 20371468828038794, -54180963383867436, 56150568425697318, 36098165222212838, -9155312379160006, 57051215026193785, 57249381874608717, -58298207413371681, 48211419804543676, -52194316791445006, 57155326110573018, -40384560671229539, -42058243391080077, 29467642604093502, 38351337419213941, -23815919899727167, 40939698213906194, -55125925501017764, -53930873167487304, 66526759915017553, -32216707957415371, 26192067348325187, -2674888776125957, 46492652710246169, 13401580564580658, -61197473302978863, 3466701421588784, -4358613387843511, 19279774718720887, -71236191682816543, -55667888923763369, -31578639030978578, -22839919263462294, -9247115980160813, -42568864253169415, 39560365478551833, 61329675553208407, 18288170986136366, -14492532330412990, -3938805957201210, 32532498910558225, 67688917351474585, -42145976074954234, -23257637960517690, 65696023447976353, -17484493692377785, 38537358342749604, -64462885685106537, -54559646206716619, 7469599010258139, 18764397595811195, -12285928690213955, 16023173804394029, 23599707494321170, -20691409758649759, 22220229153248227, 956943811204023, -70196784472558844, -51819326438160439, 42522695379263971, 43004532814065495, -22771564815538099, -53108422493289317, -7146981130201849, -54409713942887057, -16507645027383020, 21547652087659726, 61379983752330883, 64881776062113110, 47755247680505792, 46067529565044462, -45954674045455692, -62266205122325814, -34164502051676824, -22118044001434136, -30119194834130930, -46348573713397634, -29583384683174376, 71997927546001341, -26781070032670228, -31767962077779142, 11566611233861548, 7977275445700980, -19218496653776020, 47101077391948220, -66514949403002363, -21328507719999246, -31913409252446315, -20677353792548137, 71321888343276504, -18213418861088631, 34651425772923063, -2296651862731033, 6198308498378539, 13605550198472623, 47082122494860683, -61090576667490638, 280136464682227, 25831830066294658, 41123906589661739, 8512327098727595, 14219287112687816, -4861877165891372, -17094968503976874, 55981716220132675, 21176700886709442, 40799501722819663, -47470483170366843, 33716707753583351, 62939326101608723, -17626824611857793, 58602571580532358, 13720677898388313, -30362047087471981, 31573376134309394, 25962009971845465, 69934934323070101, 53355084243725699, 35673667192512596, 71213009675444077, -13106550972227067, -12481449605912217, -60041778301178802, 44320647101832903, 2204737583322898, 8438735301352108, 58102284049865009, 17504449540722395, 54695735731748118, -9505578118244589, -62124360523828063, 8781341807325771, -3309471651004060, -58838207833375626, -29246719116874383, 15570783051310840, -60356009421030425, -26287273467393014, 15138187588738640, 47101306309947829, 62816528358058373, 1278527802589035, 58006433570148101, -10474381457073683, -59556338422273804, 28438482866208586, 54802488920884273, -65553212825574448, -63542285751241886, 17166468298438952, -45151538298333381, -2548318995938722, 3257785216126316, -35358972859620243, -18076023090299341, 58849204856761647, -35211863259259399, -67877379411134782, 10142996296734278, 35531806064741584, 11651288328628807, 51984603376364091, -15352463975997469, -10070874971828795, 60988876555490698, 18978270547197693, 39529796112361894, -63632754117315982, -20543654358141815, -65185903732957146, -47200263471296225, -35966795534519260, -39156895553339859, -44508674607466673, -67483729476455058, 5222503084632045, 23251331777055455, -27811335105300863, 38097416459614886, 60526713470321567, 51541810627606944, -49728232729220078, 40441371616377059, -1485976303391612, -57334189617814048, 35278779640567107, -1030959677144024, -46638128274956518, 40872397905415722, 65699610884523513, -21393036076647864, -60164506920504107, 14652478485506792, 55231254449037397, -23544477693984729, 24478132375286155, -7180463704568582, 59910237043130973, -52160702787068526, -7679693258576257, -5210619553229248, -29620527530597813, 2001411991040984, -8805125367776878, -27259923257688164, -41487004775467446, -6805793001153507, 62787012755272931, 9396336933618605, 24235239215647514, 9113915271938509, 36690706515653342, 14365751183253678, 57310780225120538, 58903054595307917, 7249695677969769, 25064793649808332, 63254775227636048, -15255481053247376, -8520910687612570, 18279778410427731, 43458590694701737, -68667338789772462, 53284922201886139, -42201710845123763, 11496188845282350, -54665779177390660, 20477163843848913, -51071814234202225, -60544270942435704, 45473071651937501, -6453363314919002, 49858485217974213, -10411871202427862, 65298354356280964, -40984294863611398, -53151581188078886, -19993032279257674, 57310840350689840, 64379457833325621, 55483675744963152, -38512610244371292, -22054895898151369, 5817299533595438, 55449157883835047, -22167482647164275, 8769451349470645, 19163547159456385, -42108121389442222, -65415265409683806, 62243526325518193, -27633889310966236, -52708410996103305, -62614975925741987, 42855835938687553, 57855512097846064, 59197878768650846, -37340094029605350, -457038065260219, 21566511546340506, -7933515270466135, -49584093802963726, -17261210565473363, -5241998236515822, -9261367613861083, 14132607733047780, -29387878800466842, -58494500584216297, -54356567886577156, 54442411217373530]]


In [ ]:
# BGV方式の乗法準同型処理(1回のみ)
# Input: 暗号文 c1, c2, 自然数 k, 法q
# NTT用：N, Ninv, w_list, w_inv_list
# Output: [c1[0]*c2[0], c1[0]*c2[1]+c1[1]*c2[0], c1[1]*c2[1]]

def BGV_mult(c1, c2, k, q, N, Ninv, w_list, w_inv_list):
    c = [0] * 3
    c[0] = ntt_poly_mult(c1[0], c2[0], k, N, Ninv, w_list, w_inv_list, q)
    c[1] = poly_add(ntt_poly_mult(c1[0], c2[1], k, N, Ninv, w_list, w_inv_list, q), ntt_poly_mult(c1[1], c2[0], k, N, Ninv, w_list, w_inv_list, q), k, q)
    c[2] = ntt_poly_mult(c1[1], c2[1], k, N, Ninv, w_list, w_inv_list, q)
    return c

In [ ]:
# BGV方式のチェック
import sympy as sp
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k
t = 10
e_bound = 10 # エラー多項式の係数の最大値

s1 = random_poly(k, p)
a = random_poly(k, p)
r = [random.randint(0,1) for i in range(n)]
te = [t*random.randint(0, e_bound) for i in range(n)] # t*e
s, A = BGV_keygen(n, t, N, Ninv, p, k, s1, a, te, w_list, w_inv_list)
m = [random.randint(0,t-1) for i in range(n)] # 平文多項式
print("s=", s)
print("A", A)
print("m=",m)
print("s1=", s1)
print("a=", a)
print("r=", r)
print("te=", te)
c = BGV_enc(A, m, n, t, k, N, Ninv, p, r, w_list, w_inv_list)
print("c=", c)
print("type(c)=", type(c))
mcand = BGV_dec(s, c, n, k, t, p, N, Ninv, w_list, w_inv_list)
print(m==mcand)

len(f) =  1024
len(g) =  1024
s= [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# BGV方式のチェック (加算)
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k
t = 10
e_bound = 10 # エラー多項式の係数の最大値
s1 = random_poly(k, p)
a = random_poly(k, p)
r = [random.randint(0,1) for i in range(n)]
te = [t*random.randint(0, 1) for i in range(n)] # t*e
s, A = BGV_keygen(n, t, N, Ninv, p, k, s1, a, te, w_list, w_inv_list)
m1 = [random.randint(0,t-1) for i in range(n)] # 平文多項式1
m2 = [random.randint(0,t-1) for i in range(n)] # 平文多項式2
c1 = BGV_enc(A, m1, n, t, k, N, Ninv, p, r, w_list, w_inv_list) # m1の暗号文
c2 = BGV_enc(A, m2, n, t, k, N, Ninv, p, r, w_list, w_inv_list) # m2の暗号文

c_add = BGV_add(c1, c2, k, p)
m_add = BGV_dec(s, c_add, n, k, t, p, N, Ninv, w_list, w_inv_list)
m12 = poly_add(m1, m2, k, p)

print(m_add == [m12[i] % t for i in range(n)])

len(f) =  1024
len(g) =  1024
len(f) =  1024
len(g) =  1024
len(f) =  1024
len(g) =  1024
m_prime =  [[4, 2, 3, 2, 5, 7, 7, 9, 4, 7, 7, 8, 2, 4, 9, 2, 2, 5, 8, 4, 4, 7, 1, 2, 5, 7, 6, 5, 8, 8, 4, 1, 9, 9, 6, 2, 4, 9, 5, 8, 8, 1, 6, 9, 7, 6, 0, 2, 6, 6, 2, 6, 6, 1, 2, 1, 0, 2, 9, 4, 2, 3, 6, 5, 6, 6, 1, 1, 5, 3, 5, 5, 8, 5, 6, 0, 7, 8, 1, 5, 5, 9, 6, 2, 1, 3, 0, 6, 3, 7, 0, 1, 5, 0, 2, 2, 2, 8, 1, 2, 8, 3, 9, 7, 1, 1, 2, 1, 0, 3, 6, 9, 5, 3, 8, 2, 9, 9, 6, 2, 6, 7, 9, 1, 3, 7, 8, 0, 1, 8, 9, 4, 3, 9, 9, 1, 1, 5, 2, 7, 1, 1, 0, 5, 8, 5, 1, 5, 4, 8, 5, 2, 2, 0, 1, 0, 3, 3, 5, 3, 6, 3, 6, 8, 8, 8, 0, 3, 7, 4, 9, 0, 3, 0, 7, 0, 2, 1, 4, 1, 5, 3, 6, 6, 7, 7, 8, 2, 3, 6, 4, 6, 0, 4, 2, 5, 4, 3, 1, 4, 5, 6, 7, 9, 0, 4, 9, 7, 4, 6, 5, 1, 2, 7, 3, 1, 2, 0, 5, 0, 6, 0, 9, 7, 2, 8, 4, 7, 9, 8, 9, 5, 8, 4, 1, 6, 6, 8, 4, 1, 5, 5, 7, 5, 5, 7, 4, 6, 3, 0, 5, 9, 7, 8, 1, 6, 3, 2, 1, 3, 7, 4, 6, 7, 1, 4, 7, 0, 6, 2, 8, 1, 9, 1, 4, 2, 4, 8, 3, 2, 1, 4, 3, 3, 0, 1, 0, 0, 9, 0, 5, 8, 6, 2, 0, 6, 7, 8, 0, 

In [ ]:
# BGV方式のチェック (乗算)
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k
t = 10
e_bound = 10 # エラー多項式の係数の最大値
s1 = random_poly(k, p)
a = random_poly(k, p)
r = [random.randint(0,1) for i in range(n)]
te = [t*random.randint(0, e_bound) for i in range(n)] # t*e
s, A = BGV_keygen(n, t, N, Ninv, p, k, s1, a, te, w_list, w_inv_list)
m1 = [random.randint(0,t-1) for i in range(n)] # 平文多項式1
m2 = [random.randint(0,t-1) for i in range(n)] # 平文多項式2
c1 = BGV_enc(A, m1, n, t, k, N, Ninv, p, r, w_list, w_inv_list) # m1の暗号文
c2 = BGV_enc(A, m2, n, t, k, N, Ninv, p, r, w_list, w_inv_list) # m2の暗号文
c_mult = BGV_mult(c1, c2, k, p, N, Ninv, w_list, w_inv_list)
s_ext = s+[ntt_poly_mult(s[1], s[1], k, N, Ninv, w_list, w_inv_list, p)]

m_mult = BGV_dec(s_ext, c_mult, n, k, t, p, N, Ninv, w_list, w_inv_list)
m12 = [ntt_poly_mult(m1, m2, k, N, Ninv, w_list, w_inv_list, p)[i] % t for i in range(n)]
print(m_mult == m12)

len(f) =  1024
len(g) =  1024
len(f) =  1024
len(g) =  1024
len(f) =  1024
len(g) =  1024
m_prime =  [[7, 2, 2, 5, 9, 5, 3, 4, 9, 3, 3, 6, 5, 8, 0, 7, 7, 7, 3, 7, 2, 5, 4, 6, 5, 7, 5, 0, 4, 1, 9, 1, 6, 2, 3, 6, 3, 9, 0, 2, 3, 2, 3, 6, 6, 6, 4, 5, 0, 1, 0, 3, 2, 7, 9, 3, 5, 9, 3, 4, 8, 7, 3, 5, 5, 5, 7, 8, 1, 2, 6, 6, 2, 4, 0, 3, 9, 9, 5, 7, 2, 6, 4, 4, 2, 6, 6, 6, 4, 8, 9, 8, 7, 7, 5, 9, 1, 7, 4, 8, 4, 8, 0, 4, 7, 6, 4, 2, 0, 0, 9, 4, 6, 6, 2, 0, 3, 4, 1, 6, 4, 9, 8, 1, 9, 0, 3, 5, 9, 5, 5, 8, 3, 5, 5, 3, 1, 0, 1, 2, 1, 1, 8, 0, 3, 6, 0, 5, 8, 9, 1, 3, 9, 9, 3, 2, 4, 1, 0, 2, 7, 5, 8, 3, 9, 5, 4, 3, 2, 5, 9, 3, 0, 7, 1, 6, 7, 2, 0, 4, 8, 0, 4, 8, 6, 1, 6, 3, 7, 7, 0, 4, 5, 0, 4, 3, 8, 4, 4, 7, 7, 0, 6, 2, 1, 9, 5, 9, 0, 0, 1, 3, 0, 5, 9, 4, 9, 7, 6, 9, 3, 9, 9, 0, 8, 7, 1, 7, 6, 4, 8, 2, 0, 3, 2, 0, 2, 6, 3, 5, 7, 6, 5, 0, 1, 2, 1, 1, 1, 7, 4, 8, 0, 5, 0, 8, 0, 6, 0, 9, 4, 0, 1, 5, 7, 6, 6, 6, 3, 8, 2, 0, 5, 0, 6, 6, 8, 7, 8, 4, 2, 4, 2, 8, 8, 6, 5, 5, 6, 9, 1, 0, 8, 4, 4, 5, 2, 9, 5, 

In [ ]:
# BFV式の鍵生成(公開鍵だけ出力（秘密鍵は入力))
# Input: 自然数 k, n = 2^k, 法 q, エラー e, 秘密鍵 s, 多項式 a_poly
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 公開鍵　-(as+e), a

def BFV_keygen(q, n, k, s, e, N, Ninv, w_list, w_inv_list, a_poly):
    as_e = poly_add(ntt_poly_mult(a_poly, s, k, N, Ninv, w_list, w_inv_list, q), e, k, q)
    return [-as_e[i] for i in range(n)], a_poly

In [ ]:
# BFV方式の暗号化
# Input: 法 q, 自然数 k, n=2^k, delta=q/t以下の最大の整数, 平文m, エラーの分布に従う e1, e2, u, 公開鍵 pk
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 暗号文 (pk[0]*𝑢+𝑒1+delta*𝑚, pk[1]*𝑢+𝑒2)

def BFV_enc(delta, m, pk, e1, e2, u, q, n, N, Ninv, w_list, w_inv_list):
    md = [delta*m[i] for i in range(n)]
    c0 = poly_add(md, poly_add(ntt_poly_mult(pk[0], u, k, N, Ninv, w_list, w_inv_list, q), e1, k, q), k, q)
    c1 = poly_add(ntt_poly_mult(pk[1], u, k, N, Ninv, w_list, w_inv_list, q), e2, k, q)
    return [c0, c1]

In [ ]:
# BFV方式の復号
# Input: 秘密鍵 s, 暗号文c, 自然数k, t, n=2^k, 法 q
# NTT用 : N, Ninv, w_list, w_inv_list (NTTを使わない場合は削除)
# Output: 平文多項式 m

def BFV_dec(s, c, t, q, k, n, N, Ninv, w_list, w_inv_list):
    cc = poly_add(c[0], ntt_poly_mult(c[1], s, k, N, Ninv, w_list, w_inv_list, q), k, q)
    cc = [(t/q)*cc[i] for i in range(n)]
    return [round(cc[i]) % t for i in range(n)]

In [ ]:
# 加法準同型
# Input: 暗号文 c1, c2, 自然数 k, n=2^k, 法 q
# Output: c1+c2

def BFV_add(c1, c2, n, k, q):
    c0 = poly_add(c1[0], c2[0], k, q)
    c1 = poly_add(c1[1], c2[1], k, q)
    return [c0, c1]

In [ ]:
# BFV1方式テスト（テストデータ）
import math
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k

t = 3
delta= 48038396025353560
a_poly= [20243883067386795, 64739373492806610, 13268673534454237, -25510059146507325, 24061966407636498, 2766344571641204, 7192148598333927, 61376012261761341, -15725836895484245, 66460287081631727, -3053125744008357, 33770469529651377, 31566999683759598, -13104248405656654, -39083329396258818, -34967823184411637, -10698192184994744, -52326826916094516, 6622086713979984, -54672172861425017, -46033161734705039, -34367381031181419, -44715588231881299, 23140441219819673, -1639500124541613, 42326422348480369, -8271453500414511, 69107401899448605, 48990981661815206, -69051350030360820, -14735059273209942, 15926568103247567, 51510250418774716, 9648882474097071, -36317734114048315, -658285588780278, -17472738881745601, 7488433151955801, -68462570357852618, -69983789430416582, -45386429895160892, 25599395104949155, -29538560859148743, -69004223092484392, -708182805852024, -56894371218194586, -52476148758718565, 35373938065892312, -61175130610445737, 62075696633953725, -24135855132176462, 69563394967221154, -16437800639996126, -53150485067741800, -21380523445942065, -23032730964982395, -68993590776312642, -10605343228521975, -48562662721222388, 70647944309119088, -39853415822123072, -1874188796409851, 58336615044624695, 60638554888632515, 16074061595916696, 8299866771576830, -67975270212864084, 48447354469387250, 58279671414776096, -35754967610832320, -57130601534836989, 42749099471888097, -64425100846565570, -55452581767367118, -36067665624406599, 47299258256916110, -20384581881301559, -60963591170435698, 6237825034491227, 21646911521515211, -45208007119656648, 51267742955979392, 57549823567473232, -29868683949854421, -7706976657706465, 70826537258436054, 27567976586610742, -25260413135270439, 57611954754073714, 66192728069476452, 27208750823086066, 8172062685934029, -19271871103792015, 15405456493012376, -53661465154534793, 20564925820474230, -44398340869706797, 59298991667499479, 52491912499110605, 53598269277617739, 62920441437490297, 51343858358831394, -34497911481480600, -64964318784480573, -27163520791977360, 31254795650659200, -26131312951640473, -7980268969310620, -15917543281252911, 27024175267854325, 41084967017030821, 37326175625425283, -18489754076398091, 9505877293350560, 64746993735161958, -46376496543399691, -7487587994596268, 48573603172534114, 33141963515404692, 12826608887870767, 45531900408045591, 60460096290901074, 3897822520370280, 36047207785927619, -59608574287638918, -8364947639424154, 35892639138238143, -10027773522570747, -67002158357290700, -63746218438608152, -56782070844111211, -12161893350014298, 4980307888696171, -29739496962009840, -62692199692691918, 58911719609621688, 9551538767779996, -33839993000834427, 68338833817033781, -31259933382556847, 31835434094606691, 50621426057416610, 2876195994467614, 46272462593882122, -50029734912640787, 67139263575634001, -2310442465844564, -24297913734290848, 50804647107504545, -34889169556842811, -2063380349567331, 53859417399974437, -39286801524042891, -61124883591100278, -22742856324544551, -42841811563370030, -25211222815998178, -16361210832858941, -70669586216219469, -18476913803400005, 70942282586497782, 24419937198924505, -24313321549544203, 27774205276768061, -2808567081242141, 63132709283110859, 15014400424842677, 69301020918272971, -34187765904206278, -70784927106121956, 65568629482884405, 6762562239837458, 71640149554270782, 6331735248491072, 33959950138068733, -41672518654323518, 18914168213381387, -35778934385993798, -24720840804742999, -53833565907111311, -4057864036887067, -71172586505872535, -50511017527169123, -27384267358353903, 56746635911878638, -44073626764020870, -70046652382195608, -41923331071599950, -8916829970792182, -31144280887404216, -71582609434937158, -13007051868046337, 36149669452894456, 62816259197678343, 27283195298905591, -40589624156513861, -50274040861069901, 10185870325822500, 9456627255690864, -23728528488839336, -15862347090727185, -40268727787487185, 24787683716189982, 42593732581336361, -60320027781976338, -18500894059157323, -67117266141406608, -66706118999258615, 33462765765334379, 25192852387461621, -63385451470691119, -60259561826339440, -30409366854890557, 7200642003314962, -5946821420439803, -7960374458618789, 38172886638436336, 19754398563446258, 50444423924958526, -68714487286686301, -15900830727150478, 9384164165694215, 44081599875137254, 40089049098711612, 15807749309173405, 62121533688140556, -37246869414481357, -52808409152876906, 52541608354528268, -20905531188396871, 37412370108652931, 49761452672900646, -1540369945630947, -25738641543462879, -55444018359720823, -51559029769506599, 33708430963064148, -14178200322841110, -43667130071556206, 13450221183504727, -60218454129173009, 164590417844017, -51596974593781308, 54161752218339271, 33329824050283767, 53645331115504460, -12026794251632454, -47975426260074767, 10704196215614999, 15738796405695674, -66898929678908867, -67018343646066480, 57515874375985134, -70005148857683178, 67010416302552448, -30780703857172773, -60275749668766471, -14256180222478470, 7142353793191101, -58755108129951457, 25060875770871264, 29996154278836741, 60119588738208027, -12869379785010868, 6769117758112403, 65247635814932374, 66537168125842177, 30602624379917415, 67604752731727388, 34042645914285754, 13111679271894585, -60234569445454023, -39032225547514925, 6084567302708282, -58717714199974950, -57738775313189089, 17247251765292742, -9432012378703718, 24203081879225972, 3397642023020758, -53795491055490463, -52124562254925888, 772557271908729, -26854019621962285, 55820664143810697, 5967105770596220, 33668231695159740, -41520893664527604, -12376007321110207, -68828450728371565, 5347857170512588, 39148284802757905, 67267858881831822, 27473385780541953, 19236627679865098, 28215232832235496, -29949915028128205, 53380511604138862, 2590403962115160, -53248298279064544, -5354104496339327, 17072363530090856, 60430176874227103, -27007954776376386, 14914898492165180, 2701132870189464, -42300708192484825, 51390467150735200, 4476819398894796, 43205712888250645, 17905896075223861, -4573487639112874, 57017893685089050, 27120621942143083, 68743236334494416, 69159300250222497, -12711841463772625, 68841081657774686, 6894772832878958, 33055674378782381, -14202517844462060, 65329563799899892, 24562858858072329, -14376742863019381, 18037832962513068, -16619104644995776, -47482961027740811, -34005653209193484, -6327181297037623, -47558501416213123, -22233925669196720, -25787022950235280, 41136766148736351, 47184521906340763, 56810108642711861, 22443012336130220, -29530385336408200, 26362387275021117, 23148769285540217, 38190244150155249, -64986368952664057, -38298908401530135, -3717167510916428, 22793006711684988, -6194731157199276, -24688831426037820, -38297103935867652, -44717149829865419, -44322984561905781, 12815551426269261, 9685037097619404, -13202649547459777, 2711599739880402, -69447462666905466, -68485464357777408, 50170280111818606, -16804874843337964, -46779889396205926, -24573752673218168, -23916036604654266, -46137753320725105, -8123441015058530, 11182169173883291, -56157354648793698, -17335089995609049, 56805926120970283, 40895164745051047, 70957551656661411, -16212636000300023, 35451938329032042, -63867661579967187, 58498686407464101, 12735159033099123, -38068098175383372, -32042873450243724, -61125749950180866, 40922918105641113, -11325277017117619, -48927749669617271, 7142607360893385, 31627428607045014, -60188349397285434, 11900383469574599, -6706257596158205, -31883142892583737, 8937308031438256, 54155150011267100, -47009384832729364, -21417232602257907, -34331174526540433, -49630401477145545, -59077355214372926, -25594336026968689, 56089358096839790, -33308308674746635, 15703908523001263, -5553536415981451, -16612866881847918, -3921977418657386, -8417119147181434, -21949726390330758, -13030398766739963, 24787692435285101, 13298631881194705, -31911827680641952, 27651267105511599, 52176515302343070, 44128495524995218, 53709735252520603, 5412011738929720, 70389009350947063, -45131924743194402, -35416836717423323, -66650831168453848, 20383307116317470, -19622085090164660, -36524041128613956, 32003692578962102, 34755098872913998, -45181102944543431, 61263957272681677, -32662191540798463, 37986181554126843, -35332155052433523, 55686064948776994, -17177333079092515, -27312653460376229, -41997123153122646, 52519485580219343, 1778371227355485, 28803154194565117, 68379569460764599, 67960426107096145, 25689712147441108, 9010367005178538, 62655377891688796, -36153653203954746, -52365883298608509, -14222341103933840, -60048277867094637, -14961848768737891, -49702666894144053, 4334725541468282, 44424681612680270, 21752943199807505, -61097362237679227, -13934456932039153, -66789203222855432, 36905773013305902, -51559902017260783, -25439383875819593, -44609921368439770, 40246044127011507, -4198470400170365, -53412346669126125, 60596181337297286, 32388448319483994, 12145717536179682, 33822207952104473, 4430635183552624, -53701689202551241, 58914841150999081, 48895011748729327, -19010025315669784, 14117997616347286, 34807013288792311, -28683599772680177, 62366262125670427, 44616909004427249, -29663204214557653, 48997718717367440, -16248207073616571, 31749967762055506, 1845165406034398, -34014603153912719, -32991885938231297, -59752584514153993, -49194234334227099, -22448994875948245, 30536135259735623, 60340595009386687, -27168642011183802, -27119052367401580, -30715220863599200, 23164236800221037, -29860708764071407, -14372018648541224, 38424559727368036, 4234400992332975, 57155798837796685, 59985807609969039, -34553052613931951, 67625673005492228, -23275636006330496, 38524924686604477, 50733670290584758, 10017216091236485, 48128111649043024, -11603961730671210, -61961995409590761, -59727859522665323, 15754356876843365, 33296505449749099, -53200957305608569, 39154271589833972, 31283389397533223, -29403619410453049, -52477074140161777, 60861955818867822, 39869629949528198, -50950620951557852, -31932115629356045, -10798381122300474, 18563292614144195, -45692736148559154, -54895474736628515, 5606213767511263, 29252466013454853, -44332892328689419, 3707077663042824, 63235886453472865, -34312457696279899, -36029396331237623, -61847816252528863, 54935530200143565, -71092674637001798, 63340237536005971, -34718936289891717, -64313505361658228, -56190502784390307, 50258261110579084, 33568080121523431, 3231373563934209, 13766544975549709, -6475984344846075, -31864039010461340, -35192751518377548, 30037986546739809, -27332612321273272, 45362851971589067, -59047377851748440, -13467906916962261, 27763262290880210, -52653306228290904, -10772987628514778, 29028488186569707, -33627420701741792, 46162668911900893, -36518669124612740, -70032586367095911, 48677926493027894, -64335247305201874, -52167461316243584, 10018325827491774, 1209787549407830, 19428780452063092, 58895287662842279, 52714181069717700, 51376789523414146, 39838746530145598, 40151650815036049, -35672264621565096, -2753396807092582, 22364489662437466, 25726836615335708, -16057318367063532, -21485079907427950, -29446821625877657, 46248216880742043, -37145888396109403, -58542996415727029, -17303103807689176, -39666175804512287, -30315547410174660, -59373152526870005, -40318724023369585, 40929137746619007, 44769230140753527, -63343825784615842, -20778300239322073, -46104888039257400, 71931387729728979, -45023595177944386, 52620927522876137, 29240284243378591, 35771990905248326, -58614511680820756, -60285596576516891, 25636640106376514, 17320099610194218, 61400943764126048, 63390054293295265, -18839263055960924, -70159448278623311, 40356895669108822, -20936376438363661, 55667839452358749, -29041826985738046, 53894386806633589, 48426709490184180, -46886840986426840, -2225502386203256, 54384761263538447, 7008217125431372, 44892226022933884, 375348372867458, -36769616937606363, 10930250945468942, 21448376946897791, -30906299672720727, -49010186910554977, 12957218933198715, 38746232650679928, -6783304831666683, 24140729333367297, 52523688684545772, -50785096982569085, 2880406890497550, -23378227775240440, 44237340907143405, -2737534836450813, -60551217385672459, -35201548866200720, -11283815521785959, 27096330360463234, -31588803600296296, 67833589298962929, -58551098099244876, -24243206072316629, 49660161647956304, -49714462149664533, -45850343232992323, 52593825554808822, -50940438450714789, -31357238341058795, -44818033200912110, 38102285285311841, 10186186725578273, 41467437429473260, -19262116534435889, 1729960973560529, 59552845327120944, -68953395560271875, -21604387171398705, 20138000996498037, -70516767012719203, 58001562749101612, -60618173569669439, -5233540054020120, -27941386074595992, -21283565267791376, 69849156383559400, -52905126282636468, 62411652722437899, -70054037065079575, -19834424189483501, 51870606182442008, -32933834634146825, -49317433628036501, -26965310273808173, -9987724782125480, -64576595097679402, 975891683155550, 35169160635718674, 30831567655075440, -70768968981388074, 18974164370666177, 13844923314569061, 52108205495498104, -57772704355067450, 59879992914933148, -10742685899854659, 58478136056288790, -58786951634052779, 16177574306992063, -52137657338486666, 3112598299923142, -490590355193208, -71657239411524989, -1755072264003374, 24213582325854617, 51649934978669575, 46081696152071090, -59006726032323177, 18463905606742600, -64413492164578131, 16503761568583304, 54615065977442066, 59588212458617571, -27268492557730459, -6166734977572168, 36521778986567712, -38558967771625922, -44093943150022158, -37348214204082820, 44160973365108232, -1413870555014654, -7730150817563312, 40879292723915035, -4826913294389688, -8614589430120522, -13921276494627353, -66778229711110663, -64373278065099514, -53556647487179823, 54859404305989234, 68115296639861288, 12622300819987921, 27956409055373958, -49644999883716899, 67505408135166003, -67186098998361290, -9184234785431512, 13622294819490923, 29629022504641090, -34723964826905460, -2411113917411933, 12493173042105373, 8768014345857097, -29391497820528670, -43144803056303392, -67458624236758665, -5187894827955483, -44385914298306906, -43106125161917199, -25651148044339736, 69030955087558162, -9047241622999923, -3893414786372278, -66615605740793557, 4945278666090529, 20502201929671888, 58920166355924460, -63347425887554442, 56881600393822614, -4939701060226342, -58378796059070040, 63895385775585272, -29409281174144180, -47429627388173074, -56131146612351448, 5721185643901589, -40129872020968130, 5053135791514053, -51508740197091020, 71998036101977893, 36342682634835461, -17908280727937929, -6770614868289082, -28772053267171162, -68773245857061550, 49440097493949133, -5763028196903423, -22025574541417222, 33163605795833710, 68361286591608552, 54118128037859587, -524865348679355, 13924709850292026, -17564183029578227, -43465122042647715, -5864373896235484, -8655242244008194, -16680445782396056, 57616656326998674, 2590530338041191, 5983680123192215, 63007884159641735, -13279971560495575, 22490021478105316, 1177380158895419, -69354524864414705, 20271538029970874, 28900211440987489, 26296489706529515, 34773337936176504, -36955273275655654, 5177762223011336, -62799724430941574, 52333507172541700, -24421070792680070, 34570682778276048, 44665357647885352, 961756179527229, -56687362688881445, -36479428178198199, 32742144544220534, -11001034430930785, 10397566660956696, -54615897672685193, 49900200248711902, 9453495569370401, 69625190265344083, 45323663845470830, 9373150506835301, -32321262896678438, 35078770063528436, -33550052939425237, 49133293665429253, 5243737286486520, -10335117104111110, 41112008277869814, -35981468372307698, 29480507011018901, -14889822934982235, -37163596430511383, -60823573807398239, -5639085816908920, -25999523023202430, -50338190316482254, 10839879631132602, 70211839444866115, -43289671716913708, 5239904709412906, 49362086951935204, 47361408142870508, 63948975646704898, 52260487389249757, 57540662688806220, -2000147844883684, -66235938135965427, 56570345419364912, -19922567866702784, 35283607551066148, -27641040430907754, -16638660178629742, -25958585754211037, -22937750265004302, 57912206148909283, 25394500790835426, 9147042883923634, 53054057111726102, 67832284850254832, -48877490479012753, -31907418597360080, -68805532206696578, -59424305084859246, 17985893950185551, -22831281162074809, 39458858589205365, -34746010875810797, -30632651610048782, -374966650985560, -23901885035605381, -7212392421976872, -36079394714002019, -16885805901515496, 6576403427683340, 14520375163522882, 30584690225374185, 16998455748467518, -56640588011562205, 23721976354842127, -40177569494498594, 58767817559541490, -54994537972510533, 46828313724509612, -4870478195903286, 13344673656735459, 65514540264984998, -18556779509689124, -54265867541857175, -6396605994624342, -65371306523309197, 53831377560691957, 914706338406913, 138542033111550, 63214121467825101, -52472646866538127, 53799988521048429, 1952714303457492, 60996766159916114, -8290324000697683, 9680701926173706, 31978790816412549, 37133555877642317, 34962462471072670, 28539324278381211, 56055669109488458, -23146243335677098, -68377981591117117, -1725349435797746, 63459683745177747, -36114650161074635, 12473035399664836, 60931338088294417, -16169886250504432, 34965643413558235, -35201953107081034, -58724829227377508, -20980154178876031, 43581392035327148, -7133103889369499, -13506690410074190, 16404702086005270, -26325011489240895, 44586375918141406, -7405450925016613, 48598506404460376, 59526748315651443, 41147600991955117, -62047736235913877, 69967345234468399, 18283970483221288, 39467495473892978, -39990422256560638, -68852298006294293, 60934035398156314, 58704051770048246, -63589366793022383, 21605809159070642, -63887348837674916, 29529859576426069, 19178390987360622, -23413448360571037, 36590068599243650, 37897318210524795, 5993165401096494, 29799880874416713, -34571566982280223, 62446535463205153, 4663175418337080, 33827183264514593, -6768536403179965, 41539613514161138, 2446305645153129, 7119132942531456, 48276484762646486, 15288215211183641, 23462762894948229, -36567723986525241, -24300488721027574, 56287458556094978, 70071794338548950, 64299741790620054, 46250522532829109, -59287977298820351, -28037535593149244, -44851702366696011, 47724804627706991, 43460693010161006, 41745050439044756, 69189299161181357, 20646969436688799, 9217650784123564, -21787969112869083, -63909326161712182, 62260175385377506, -15597328444227334, -1234045245600605, 30113567266780096, -13071116264903108, 28319593125340765, -5773234303385446, -61821700079306941, 11639866496754090, 64247489015149248, -52396491508136240, 68270707536750427, 68142425043810446, -2538374011172828, 457671204576721, 30922253652006736, -69557711567292955, -2210064249137995, -22348742591487912, 11519619622894160, 48871624829340616, 69592908960818636, 70528240483519026, -8908896051242780, -59612417805259089, 32001139453267559, 51674570911105027, 15488563569831832, -42583945213154665, 42540225741013732, -9351919331735832, -16540753795085139, -52663449244979386, -55909059300542192, -44178697878359093, 21131138485785761, 45716967958364717, -7519525193142125, -58839095928627617, 42962177948420695, -23197870309634000, 12571261055808041, 47621732860722882, -46455540266602661, 11719638276908527, -32887031304589398, -35760319012307154, -26266563510371259, 7204283118718134, 5317745096231762, -31168607896230702, 42503913626337930, -38328095955758585, -20180519035131225, -24717288460444867, 60478952394395925, -52533097607726002, -52563857501036775, -51888810655238726, 34534353372182578, 28436948143878029, -21335409389306081, -71079253863746427, -10114705165055523, 60084696766148655, 35472436407051307, -15766677148710969, -43100949802242572, -30622970440018873, -60233322671198846, 10408654388159349, 20273812028048845, -30431222195746907, -48698114895884292, -36758804221135686, 60600761113626316, -59789954863202248]
m= [0, 2, 2, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1, 2, 1, 2, 0, 0, 0, 1, 1, 0, 0, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 1, 2, 0, 2, 1, 2, 2, 1, 2, 0, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 2, 1, 2, 0, 1, 2, 2, 0, 2, 2, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 0, 2, 2, 1, 1, 0, 2, 1, 2, 0, 2, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 2, 0, 1, 1, 2, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 0, 2, 1, 2, 0, 0, 2, 2, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2, 0, 2, 1, 2, 0, 0, 0, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 2, 1, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 0, 2, 2, 0, 0, 1, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 2, 1, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 0, 1, 1, 2, 2, 2, 0, 0, 2, 1, 2, 0, 1, 2, 2, 1, 2, 1, 2, 1, 1, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 1, 0, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 1, 2, 1, 2, 1, 0, 2, 2, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 2, 2, 0, 0, 1, 1, 1, 2, 0, 2, 1, 1, 1, 2, 2, 2, 0, 2, 1, 2, 1, 2, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 0, 1, 2, 1, 2, 0, 1, 1, 0, 2, 0, 2, 2, 1, 2, 0, 0, 2, 1, 2, 1, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 0, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 2, 2, 2, 2, 1, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 0, 1, 1, 0, 1, 0, 2, 0, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0, 2, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 0, 0, 1, 1, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 0, 0, 2, 2, 2, 1, 2, 2, 2, 0, 0, 1, 0, 1, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 2, 1, 1, 1, 1, 2, 0, 2, 1, 1, 0, 2, 1, 2, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 2, 2, 0, 2, 1, 1, 2, 1, 0, 1, 2, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 2, 1, 0, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 0, 2, 1, 1, 0, 1, 2, 0, 2, 0, 0, 1, 0, 2, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 1, 2, 0, 0, 0]
s= [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]
e= [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]
e1= [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
e2= [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0]
u= [0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]
pk0= [11120513890052099, -63153401979959709, 69430848826272997, -58230762441496064, 12650095172019455, -44000259578572843, 68771964983274757, 71467130071825558, -20520831879170948, 51951325395352072, -71925229700791829, 19036348517194435, 41430875971226601, 34790433733048189, -55104107502598397, 64831284198194154, 437330861476654, 46191469051598575, 48513453538002207, 66953354807437186, -66436043919567798, 54847980938800122, -38186700624443356, 30736485698354434, -46628688822697311, -25015542847421682, 58866884138417717, -70467388687519461, 12094160960292645, 6406742525019087, -19249105828624233, 21904293101323270, -72040760244148428, 60543712702456515, -16234898206168061, -68856684314189529, 67291982114089219, -41805620509626517, 38722674624049451, -28399801099911540, 62519270602084607, -51830844431928099, 14852829554732465, -69621542475821751, -10055566728204123, -43091561541493207, -54718083042157847, 29947210737251381, 71549496200574429, -21865796241049699, 2395399272085172, 2892508427948223, -62335507122281236, 54239101162354073, 56805023281118464, 63207743178899148, 28449432967334896, 60293582973033177, -7792717836300717, -39500580838974916, -26671892284069369, 38901210321777387, 36010845891353992, 58496887916083023, 44118833068237652, 2978300206041711, -36735282489355823, -5226884880661454, 62304206414832186, 3751615622957812, 70549811583608342, -31526614146320970, -13788516485961984, -31025210046687246, 27157806328877430, -23291872710385253, -15460522344631547, 6667685608442156, -20053480200121019, 40019191955957809, 70944497714452472, -34466026497208930, 39946821641719674, 19078629291668298, 219759163765551, -52099630230598965, 66496856511279528, 581077233204418, -47368149931212013, 48544978483250991, -32286052710717641, -7631128067120124, -49195633750674658, -11447235051508833, 12964473113030105, 60148785544148997, 13026488838211465, -6948215729229864, -37626999161681710, -29999366042574120, -18372753139436059, -22745960348772557, 16869963470083966, 16118947912403002, -70513073806307646, -50465191374049563, -22606317671787566, 21476217296583550, 41280849739139458, 39309013112086642, -1076508464968389, 22475034877519574, 31739441731221606, 57191098122010956, 38887758004700914, 16144833607503594, 37459764646390104, -64302170399851811, -57279097650460284, 34153327947834581, -30217130684489571, -45165911500278284, 43900298773206303, 34999253205479415, 262486374795380, -61704101969997699, -23120222498049002, 190692296027711, -26938459391006338, 66655590593932115, -4024184716272853, -6646829324319912, -51277586991353600, -52534290192333051, -6236521929772055, 41233485967311339, -35989111859994053, 153254568983543, -44386565570367450, 32149217762252898, 2891041872577803, 53167767170576065, -59760432105381880, -71149185053666654, 9851806830563018, 22266593549418845, -16830801300991776, -39870070518669668, 23532543854874194, -15053645276927672, -21647087186119648, -18570763949659066, -21711943937398345, -57878419854584741, 34012495724261860, 1264328760974065, -41209436219294695, 66237640436197146, -50398448544267653, 69954949892038279, -22182906345244410, 64163014675245585, -2406367785523957, 68334528456976099, 39119934189838752, 8700924136639531, 52797065239307465, 47086609269443456, -36324230319054904, -23245188716144382, 30230446790094773, -11420535285405487, -59080428679783734, -55603093441201640, -27754638378735144, 44313718337548319, 10595410106579954, 58121625343592487, -65947140049061114, 67921622720897706, 58093529407813913, -25513405141854119, -21645565039482651, 37759396802679778, -25290769335815787, 3103760656874627, -6914500170444363, -22814459909297870, 2521380031766370, -36904768244358051, 42345424858614013, 36049892330909681, 64257471057350732, 47892578272500054, -49479077293120579, 43576962213634667, 3920080244140613, -5994741526367684, 25308204236799646, 57742932985460066, -6068883371398703, -23745374568122882, -36939803504711942, 33335370016143106, 28677203742749088, 3948827100199916, -10465770573061492, -36391749001836906, -40600315737564454, 40484831222304282, 69471538949154034, 41421683377381069, 38529645570725063, 70487881547170875, 29854874020833744, 31641364784081407, 3413223323481490, -15140678507608646, -45491335432931758, -17305794419363461, -11221885859518869, -64349762280017641, -6150677083597892, -54063040539263039, 62585397041229006, 36122081331620039, -11067398721749684, 24895317371865019, -68446611582916741, 61090337447502888, 43571396145717331, -513957574986978, -16491002316961918, 19437608012377587, -33639007718932685, 775594821678281, -55679949384873105, 14833831106002122, -20181576489084582, -33823974668802099, -51695246558802480, 24063393278248015, 24108456785449837, 65919917696201373, -57335807105166572, 57586477304483831, 10139693547541716, 1899269375994036, -29444248580783385, 27836122241360093, 47349033159195861, -526514307081743, 55550695587661840, 53845377846284692, -45078288642913136, 62882195669320060, 59041222199805885, 50058750242191357, 16662806624105108, -65480814671556264, -59422714634453454, -59774403963390483, -16569386631787551, 908242992799511, -11540815143100754, -8649114556565751, -26690521264123924, -66761115348857181, -3319153109227207, -42760590781011509, -60753355799867856, 27014129227599547, 50208075933526277, 33275119128106773, 29545238407828125, 57997601703204594, -53052563935840063, 28757012489945237, 67942703654509678, -12159996038384445, 26347316518497586, -60461881970003230, -9176209400907164, 33810384020858266, -69752181295617152, -12814679494532163, 2835752034196929, -69658428704000929, 16383667684504855, 49222322017678024, 33796935793372640, 24744721457089943, -68872817611584628, -26123610444245263, -48489831802816852, -24097770381646300, 1151534015635579, 5636912719330887, 24627505471450031, -51129821654934090, -27853438635848866, -17604807982776503, 11100301890298449, 61872949212423232, 50788807737714369, -59736757619129453, 49530011443390886, -51081494167641200, -158329725671292, 57205958336451839, -61015612166056188, -31691966306191368, 52405162928338456, -5075816814093340, 36131814415594540, 22724214784497382, -34370360720990153, 29873273760434865, 52554262245299977, 12385175196591206, -38344160633484368, -46752288808273907, 2641009792389812, 50120833770761612, -4955610742543528, -52326469535418429, 39872844214949307, 8588156791186121, -55555288525746768, 15171434181361948, -62854565022434870, -70369426990659350, 32347016275994408, 6065712876233051, -70137752712014945, 11951139311486010, -9594357231014392, 50525361542955938, 53908912671528290, 72005249760240214, 26354411115715241, -64678431664143263, -22355444630475271, 68881255802968080, 11368632629763389, -57287305921201954, -38754171836767146, -24652895554893282, -31932012687257600, -71911983598658504, -10436018265094242, 20298308705915857, 37600623744754602, -58396791279272359, -17956417683489733, -30773636751776903, -65468863169234489, 56526400400437662, 36851118482608140, -59039749823116274, 25591373294217193, 59764644759173389, -47382957786985470, -38715957487319770, -6900774219807013, 8773633889109328, 48156001343349344, 51160676918518482, -6518122274848090, -61163696819287935, 32224391142220726, 71373488389577289, 46878600318072027, -11264097154939448, 56505021982968661, -3743198140220699, 39643530597393090, 35897271098439916, -10423876495922538, 17532884323041954, 9658988836899677, -35843827584246598, 1896079811450432, -27421936760502664, 54459662740171790, 37742332622750486, 35201656195898618, -31092291150165061, 60857894849198940, 53050383562330285, 26863784582270114, -42684199996470760, 44292538029891, 52930819074922095, -27460096202340717, -29668881350417999, 17595486059392137, -38620825296672920, -10024490757441152, 68210520714661106, -8564469253814882, -55084530991912384, 43679181773242034, -42315071004930110, -54534716356520777, 2858044011046856, 35204867887922491, 22567219781954254, 58811350570917352, -40197190024225973, -40431016345986889, 7851596121215216, -49929104607541877, -50661290916110854, 41070462200794840, 11453861349776023, -17617586498781680, 48115392162045320, -20148674480654940, -57078191840013054, 24759919474435832, 6753903522711913, -37550050889936829, -4939396973659164, -14335961968928472, -29556608605093201, 71848532831289009, 54439275063613461, -12436823049000695, 35029278812156326, -32894913773168388, 48214651077805535, -49370365589440532, -56166124169458434, -65599973516316389, 13614880082322479, 31836822367409263, 3787676633326591, -65662402257379024, -68193059481765093, 31150760223928563, 11621466868882854, 43074584358885980, 47724669902444230, -844126775995577, 50774871542360598, -6974158279661315, -2352722738571336, -6434968491208507, 58028982704552510, 13331010907281017, 6978556236255546, -1529868908327309, 49991846347565911, 34258838417871942, -29296738614628124, -60948869016059652, -18778434625295245, -68189709080160053, 67711939157750707, 38531923745419206, -44077714453233743, 35507031876102920, -25955009487851417, 48175458588701571, -12848975381234016, -21223130933709947, 5007497901163773, -9402357455379710, -70684282993046495, -2433237458124400, -70174378031101054, -33177054999419436, -7877861838993372, -40128048841725380, 38456085166612991, -47676181145103740, 17649545494156671, 38319343101417055, 54693842444840994, -27772948260244639, -43570497355606878, 40947350043691002, 51136442462487516, -55427614057224166, 38726215972981034, 3894753673176572, -7220182244430080, -69133280097074191, 30353418019544143, 52585605914624178, 57040678792730951, 60216390003709976, 59628600181975313, -47349889996386506, -2212230000616649, -23789480322566704, 21533228681527664, -68833767121940084, -68418460821603807, 23837487456262093, 38812510807378563, 43143891527174836, 50461264421085815, -2949800987857518, 47676325648332931, 51964635856291772, 66916744369944004, 2190957861068601, 14177286217825510, 11481231015994739, -50530862355633794, 6237681409867624, -21992788489505712, -43507617741308678, 31655723495259361, -8571394806088122, -62315909879344059, 20101233820043205, 15850359851058745, -62284167968118062, -60455812683467972, 61088973684271779, 8233130758706982, -67459375193472799, -3997620852105920, -59334864395001756, -36818561852059474, 945528896535680, 44745286291613062, 42796707710794710, 49086005690157104, -23215741689775553, 34713944591135208, -63476555118608564, 14349621075174355, -58831822905043238, -22062386379810303, -31438731394975190, -9006663818570229, 20334837445562406, 55135644523899637, -15012282217944275, 9680894414792051, 11189126032117583, 63141082002207209, 38739500937770946, 68917204282017435, 32731446015642672, -4556453499343677, 59809119560250836, 39895878971266145, 22291348960574232, -56171567475692725, -28743364630681123, 448807848547281, -1119362808995267, 25388303854093118, -45756054138820481, -67479240190183378, 26366377127778100, -39294560393568879, 16441090988887319, 31478699007838906, 35531432253164328, -11776437040055106, 13930098498269868, 3158926747783124, 42439066363394616, 10780888765061006, -27310406011228722, -8981796758181082, -62726892444486504, -26451072103773570, 48703681041844398, 61269680884308934, 49371692603229222, -12712063092353023, -40654116114695255, -52306671427408656, -14002882692621570, 59009804005502106, 37647038936898652, 43561124754402649, 30846171645269174, -69200563486422757, -48408495853296812, -66147235171900478, -68809390137127737, -20639403705602486, 22999344529089902, -20503877179189477, 70873156697112612, -39598044468468185, 70797790667556258, -19760409815066325, -2794239502409879, -53460659866335923, 33940169466063412, -11001569287622036, -65372158393975318, -35391013971831733, -62889449813815507, 726469503406021, -69219141299371435, -60561916948873601, -5167340622524010, 71094022238641966, -11069535481251592, -15041765055230294, 22748557378362609, -45323920511569695, 20735773817512958, 20662278441894230, -812283461392515, -21119050472296224, 10451124919731837, -5387921402857280, -35497908042699771, 48827588561543516, 46963416320456, 63415994387672587, -68008722824838916, -26086834147408767, -29248094029349495, 31188478987248248, -58329641779994517, -71843378924034852, -20502197036310969, -16122359107210612, -69098237180280412, -7806091738618038, -35144810379530057, 68559125144697253, 56209878948526120, -14122584666061970, 22865955653712204, -20489724951003419, 49423619758519639, 15805860305673493, -45536980621940422, 37319259161743852, -26595692134567214, -62824894642934188, -46408399339260281, -33964237100022268, -11743225247763312, -61832576795192965, 29948722979020413, 25659746431991805, -64916609698258731, 63750798255810857, 51876628240207942, 5888583134929036, -9409554277347871, 6637662797380829, -41290115812682704, 69121324038765338, 61372308726940601, 46066909466250660, -63733782547608532, 28378895234892040, 7250565573793955, 33870666411107600, 20561541593848578, -36825518918088752, 12356227084414576, 20430639088032115, -23204782895458592, -37142062793337585, 65750817115820284, -3252721107905510, 61354809641468794, 56069241253129712, -41900200506355858, -13231180308204358, -23195170397916403, -60423142469291263, -36386185143054420, 34033977800566494, -36209937618421010, 52385816379701508, 28089393888573652, 12571685562927914, 27843723927829401, -18411705417825009, -69152412765978504, -47941713234523392, 62448537245683428, -38317726992296266, -26343518862333163, 44687215105271113, -13333315813919007, -66537522378977008, 23394676441299595, 35930981598769697, -5857340654165320, 53544985822567025, 15013546405573291, 45963325796780873, -5502537270748689, 17847100856007162, -53455298654384724, -47331392960365340, -5872675713921146, -17863391965309390, 34696295685106793, 71511160418367463, 71835564272445879, -33237559262685856, -58082107874487694, -28446843386563205, 39252128638446884, -8124266629642543, -34812363544899553, -18384719576849668, -12246009639108583, 13487520291543571, 63527058399310108, 64771202498892941, 12409718750248415, -62048276745984292, 23148347735850168, 45955051895157392, -65533310485606078, -38296277916400669, -26993806139646477, 23603955728659687, -66091420231299421, 51611433567182999, -46091113299692146, 31706329791878348, -13868052947358087, -50624388458606885, 16120221046511098, 1924654162080998, -41599163137885789, -41566951661023678, 65160592670000403, -9045382558547731, 54468968920247542, -12221725151313188, 35354935613419952, -4081535981579318, 31619147819478736, -71946437527141646, 46660784874393363, -9488290744564790, 62536739061643150, 17294808168327613, -15401462335909007, 7616563609174469, 69921376667499694, -42808525363773322, -26359805617383148, 10168172694507243, 11232458628659152, 19983022273746279, 70717154219466458, 9391228129145526, -29462497672344951, 58409522096401112, 26229004440536767, 24807200404956126, -37516461071804443, 46417298107648080, 54696770976774320, -65449889653738705, 45451712423489895, 14994556564848959, 35824880709328102, -48074059988475331, 46138277505484538, -25767531503607819, 31372309431769753, 70142953526521527, 59910774686762101, 71005887137942875, 19246396983204883, -20278184011800431, -56946006206032470, -12024700648197130, 71093953004025585, -32024628641089077, 9852721910492038, -31524473151794524, 29539867245862799, -50912625328964454, 5084236324779296, -47808211691240247, 34474694379622815, -67493884222643809, 51110390851193301, 42562405667145888, 38889700089638512, 21599288939659935, -24398397244230851, 69581191047161665, -17872010878582318, -33079241744013702, 68605398590102235, -62338721752654776, 16246809850102454, -5451588532479352, 6077736674369509, 217980822568448, 1344196990981841, 10532373267982391, -47964777772879939, 12543407677323265, -37342290613448347, 2677812810310771, 44391279197704869, -23771762091891942, 55781099530868407, 63329370668097386, -61472481042978679, 19760448706450845, 66839365873551650, 41310473753181833, 14051112034471069, 68378506081490364, -39550227027972395, -26133975355372604, 5509673721023159, 18813939473910169, 18905410509622311, 40255601109548661, 46063749394914957, -5694018605283085, -55219006544259046, -24963389128265714, 47345882723504340, 7730970834047609, -35951766007020306, 6041774166918795, -62566144184115177, -31878672791941914, -52583905956829154, -5120559842812361, 48037456418262045, -68864840336654942, 9742268897588719, 64354716578398054, -47457248664844883, 4870908543947227, 68718099465024219, 5971072558795039, -37103389510259203, 42569401119667485, -71618610384241982, -45852239339473419, 36294393599332103, -67988593010249371, -34425927233130787, -53295867521331537, -26533089389705306, 61154996781945526, 45498447002786265, 16689024913583025, 10311591794027662, -9527374964273939, -63286315923261179, -13699235686196926, -3790125541616835, -22354177804532060, 844215231025723, 9467517406964775, 21982742241176194, -35364724678517753, -45562205648727075, 49576565765790901, 33137725560658132, 13544260643392931, -19290656255889587, 45066409237093503, 46293962979147879, 46581729673519919, -20443321737382325, -10433607613449156, 9083325313686426, -68414750675323514, 32360458141741523, 42666699684866346, 25576245144155818, 4007625245760272, -29534025901142835, 34085572954315471, 43633008306458980, 18637449617812326, -61536126352488295, 16273964503998849, 29128327581808355, 69677399258097414, 34303559934341983, -33256140108472500, -67959153907970755, 6546996803235379, -41095142831476223, 44768962374895878, -41059686814000520, 67686235476569340, 8207985564294065, 25393003212125958, 58724531619526562, 53783460578240943, -27834637628744819, 17261809752584596, -52406531085414190, 25646897474852272, -22411034531783788, -48613377144162332, -38867655708437646, 62476640252089671, 46386213840375001, 38527037405009701, 43480490576065716, -23537954718007194, 7291734879871632, -71785523749606543, 41587000548130465, 44876989254788146, 54703006200200641, -57839104462241442, 69490811400823184, 33120418548111643, 59499869688593382, -50844612194312920, 11614685884206287, 10958405406468516, -53263532270460972, 16962007947546613, 34497581897237411, 3568074045933838, 11561761833602405, 39880948824405181, 65808314724628120, -45411652960125645, 26154061900762625, 43051227586125421, 68793419558308115, -21071518760752401, 34224345840423428, 5876310960831522, 68785552450856840, -45821142149019867, -39622714574425561, -17858296074356083, 42086319764451836, 63642533849648662, 11872127741477889, -64732969123017334, 65916344361341154, -57003533284282307, 54159318337760955, 348301538563251, -42614623562209560, 12104518501475899, 44972249085356008, -42822391876991143, 32000212878574667, 5769146553648524, 51875361055411177, 29796563419289884, -18630459896127193, 640609444944872, 2661884623296816, 31325594881764482, 31518864206863014, 66471714576887527, -30256361067513110, 30780822926729627, -57721938493673908, -37840765243151416, -6280095969203191, 39939306200912908, -56523696734970387, 7520928641887502, 24473803315630632, 28656798934859315, -44014579626947806, -9134287527650814, 52987326242525245, -45922301769001179, 14604763852592008, -65125766568457561, -38290585984140124, -24397041734695406, 70282429277199229, 14844714185269933, 47750533803886441, 313209218815266, 2853731434324688, -20307227597596461, -39913688152386986, 22186155251189293, 60251652358352807, 16105952207987758, -69506341309918666, -8523822202533496, 15069581702834335, -50225518322566496, -14691749820519302, 27840398722093664, -187235836786640, -11403815517673396, -31371175861574052, -60373372817078735, -12401301590516951, 30018887545831486, -64354758329249983, 67331501033320650, 24506777526904723, 8600378550736083, 22595847738606121, 33025385088454246, 45393143803510682, 12710384572903612, 36430184330357363, 55785044349263599, -42747785579952404, -3634959814022080, -8027578484435467, 12038797198106059]
pk1= [20243883067386795, 64739373492806610, 13268673534454237, -25510059146507325, 24061966407636498, 2766344571641204, 7192148598333927, 61376012261761341, -15725836895484245, 66460287081631727, -3053125744008357, 33770469529651377, 31566999683759598, -13104248405656654, -39083329396258818, -34967823184411637, -10698192184994744, -52326826916094516, 6622086713979984, -54672172861425017, -46033161734705039, -34367381031181419, -44715588231881299, 23140441219819673, -1639500124541613, 42326422348480369, -8271453500414511, 69107401899448605, 48990981661815206, -69051350030360820, -14735059273209942, 15926568103247567, 51510250418774716, 9648882474097071, -36317734114048315, -658285588780278, -17472738881745601, 7488433151955801, -68462570357852618, -69983789430416582, -45386429895160892, 25599395104949155, -29538560859148743, -69004223092484392, -708182805852024, -56894371218194586, -52476148758718565, 35373938065892312, -61175130610445737, 62075696633953725, -24135855132176462, 69563394967221154, -16437800639996126, -53150485067741800, -21380523445942065, -23032730964982395, -68993590776312642, -10605343228521975, -48562662721222388, 70647944309119088, -39853415822123072, -1874188796409851, 58336615044624695, 60638554888632515, 16074061595916696, 8299866771576830, -67975270212864084, 48447354469387250, 58279671414776096, -35754967610832320, -57130601534836989, 42749099471888097, -64425100846565570, -55452581767367118, -36067665624406599, 47299258256916110, -20384581881301559, -60963591170435698, 6237825034491227, 21646911521515211, -45208007119656648, 51267742955979392, 57549823567473232, -29868683949854421, -7706976657706465, 70826537258436054, 27567976586610742, -25260413135270439, 57611954754073714, 66192728069476452, 27208750823086066, 8172062685934029, -19271871103792015, 15405456493012376, -53661465154534793, 20564925820474230, -44398340869706797, 59298991667499479, 52491912499110605, 53598269277617739, 62920441437490297, 51343858358831394, -34497911481480600, -64964318784480573, -27163520791977360, 31254795650659200, -26131312951640473, -7980268969310620, -15917543281252911, 27024175267854325, 41084967017030821, 37326175625425283, -18489754076398091, 9505877293350560, 64746993735161958, -46376496543399691, -7487587994596268, 48573603172534114, 33141963515404692, 12826608887870767, 45531900408045591, 60460096290901074, 3897822520370280, 36047207785927619, -59608574287638918, -8364947639424154, 35892639138238143, -10027773522570747, -67002158357290700, -63746218438608152, -56782070844111211, -12161893350014298, 4980307888696171, -29739496962009840, -62692199692691918, 58911719609621688, 9551538767779996, -33839993000834427, 68338833817033781, -31259933382556847, 31835434094606691, 50621426057416610, 2876195994467614, 46272462593882122, -50029734912640787, 67139263575634001, -2310442465844564, -24297913734290848, 50804647107504545, -34889169556842811, -2063380349567331, 53859417399974437, -39286801524042891, -61124883591100278, -22742856324544551, -42841811563370030, -25211222815998178, -16361210832858941, -70669586216219469, -18476913803400005, 70942282586497782, 24419937198924505, -24313321549544203, 27774205276768061, -2808567081242141, 63132709283110859, 15014400424842677, 69301020918272971, -34187765904206278, -70784927106121956, 65568629482884405, 6762562239837458, 71640149554270782, 6331735248491072, 33959950138068733, -41672518654323518, 18914168213381387, -35778934385993798, -24720840804742999, -53833565907111311, -4057864036887067, -71172586505872535, -50511017527169123, -27384267358353903, 56746635911878638, -44073626764020870, -70046652382195608, -41923331071599950, -8916829970792182, -31144280887404216, -71582609434937158, -13007051868046337, 36149669452894456, 62816259197678343, 27283195298905591, -40589624156513861, -50274040861069901, 10185870325822500, 9456627255690864, -23728528488839336, -15862347090727185, -40268727787487185, 24787683716189982, 42593732581336361, -60320027781976338, -18500894059157323, -67117266141406608, -66706118999258615, 33462765765334379, 25192852387461621, -63385451470691119, -60259561826339440, -30409366854890557, 7200642003314962, -5946821420439803, -7960374458618789, 38172886638436336, 19754398563446258, 50444423924958526, -68714487286686301, -15900830727150478, 9384164165694215, 44081599875137254, 40089049098711612, 15807749309173405, 62121533688140556, -37246869414481357, -52808409152876906, 52541608354528268, -20905531188396871, 37412370108652931, 49761452672900646, -1540369945630947, -25738641543462879, -55444018359720823, -51559029769506599, 33708430963064148, -14178200322841110, -43667130071556206, 13450221183504727, -60218454129173009, 164590417844017, -51596974593781308, 54161752218339271, 33329824050283767, 53645331115504460, -12026794251632454, -47975426260074767, 10704196215614999, 15738796405695674, -66898929678908867, -67018343646066480, 57515874375985134, -70005148857683178, 67010416302552448, -30780703857172773, -60275749668766471, -14256180222478470, 7142353793191101, -58755108129951457, 25060875770871264, 29996154278836741, 60119588738208027, -12869379785010868, 6769117758112403, 65247635814932374, 66537168125842177, 30602624379917415, 67604752731727388, 34042645914285754, 13111679271894585, -60234569445454023, -39032225547514925, 6084567302708282, -58717714199974950, -57738775313189089, 17247251765292742, -9432012378703718, 24203081879225972, 3397642023020758, -53795491055490463, -52124562254925888, 772557271908729, -26854019621962285, 55820664143810697, 5967105770596220, 33668231695159740, -41520893664527604, -12376007321110207, -68828450728371565, 5347857170512588, 39148284802757905, 67267858881831822, 27473385780541953, 19236627679865098, 28215232832235496, -29949915028128205, 53380511604138862, 2590403962115160, -53248298279064544, -5354104496339327, 17072363530090856, 60430176874227103, -27007954776376386, 14914898492165180, 2701132870189464, -42300708192484825, 51390467150735200, 4476819398894796, 43205712888250645, 17905896075223861, -4573487639112874, 57017893685089050, 27120621942143083, 68743236334494416, 69159300250222497, -12711841463772625, 68841081657774686, 6894772832878958, 33055674378782381, -14202517844462060, 65329563799899892, 24562858858072329, -14376742863019381, 18037832962513068, -16619104644995776, -47482961027740811, -34005653209193484, -6327181297037623, -47558501416213123, -22233925669196720, -25787022950235280, 41136766148736351, 47184521906340763, 56810108642711861, 22443012336130220, -29530385336408200, 26362387275021117, 23148769285540217, 38190244150155249, -64986368952664057, -38298908401530135, -3717167510916428, 22793006711684988, -6194731157199276, -24688831426037820, -38297103935867652, -44717149829865419, -44322984561905781, 12815551426269261, 9685037097619404, -13202649547459777, 2711599739880402, -69447462666905466, -68485464357777408, 50170280111818606, -16804874843337964, -46779889396205926, -24573752673218168, -23916036604654266, -46137753320725105, -8123441015058530, 11182169173883291, -56157354648793698, -17335089995609049, 56805926120970283, 40895164745051047, 70957551656661411, -16212636000300023, 35451938329032042, -63867661579967187, 58498686407464101, 12735159033099123, -38068098175383372, -32042873450243724, -61125749950180866, 40922918105641113, -11325277017117619, -48927749669617271, 7142607360893385, 31627428607045014, -60188349397285434, 11900383469574599, -6706257596158205, -31883142892583737, 8937308031438256, 54155150011267100, -47009384832729364, -21417232602257907, -34331174526540433, -49630401477145545, -59077355214372926, -25594336026968689, 56089358096839790, -33308308674746635, 15703908523001263, -5553536415981451, -16612866881847918, -3921977418657386, -8417119147181434, -21949726390330758, -13030398766739963, 24787692435285101, 13298631881194705, -31911827680641952, 27651267105511599, 52176515302343070, 44128495524995218, 53709735252520603, 5412011738929720, 70389009350947063, -45131924743194402, -35416836717423323, -66650831168453848, 20383307116317470, -19622085090164660, -36524041128613956, 32003692578962102, 34755098872913998, -45181102944543431, 61263957272681677, -32662191540798463, 37986181554126843, -35332155052433523, 55686064948776994, -17177333079092515, -27312653460376229, -41997123153122646, 52519485580219343, 1778371227355485, 28803154194565117, 68379569460764599, 67960426107096145, 25689712147441108, 9010367005178538, 62655377891688796, -36153653203954746, -52365883298608509, -14222341103933840, -60048277867094637, -14961848768737891, -49702666894144053, 4334725541468282, 44424681612680270, 21752943199807505, -61097362237679227, -13934456932039153, -66789203222855432, 36905773013305902, -51559902017260783, -25439383875819593, -44609921368439770, 40246044127011507, -4198470400170365, -53412346669126125, 60596181337297286, 32388448319483994, 12145717536179682, 33822207952104473, 4430635183552624, -53701689202551241, 58914841150999081, 48895011748729327, -19010025315669784, 14117997616347286, 34807013288792311, -28683599772680177, 62366262125670427, 44616909004427249, -29663204214557653, 48997718717367440, -16248207073616571, 31749967762055506, 1845165406034398, -34014603153912719, -32991885938231297, -59752584514153993, -49194234334227099, -22448994875948245, 30536135259735623, 60340595009386687, -27168642011183802, -27119052367401580, -30715220863599200, 23164236800221037, -29860708764071407, -14372018648541224, 38424559727368036, 4234400992332975, 57155798837796685, 59985807609969039, -34553052613931951, 67625673005492228, -23275636006330496, 38524924686604477, 50733670290584758, 10017216091236485, 48128111649043024, -11603961730671210, -61961995409590761, -59727859522665323, 15754356876843365, 33296505449749099, -53200957305608569, 39154271589833972, 31283389397533223, -29403619410453049, -52477074140161777, 60861955818867822, 39869629949528198, -50950620951557852, -31932115629356045, -10798381122300474, 18563292614144195, -45692736148559154, -54895474736628515, 5606213767511263, 29252466013454853, -44332892328689419, 3707077663042824, 63235886453472865, -34312457696279899, -36029396331237623, -61847816252528863, 54935530200143565, -71092674637001798, 63340237536005971, -34718936289891717, -64313505361658228, -56190502784390307, 50258261110579084, 33568080121523431, 3231373563934209, 13766544975549709, -6475984344846075, -31864039010461340, -35192751518377548, 30037986546739809, -27332612321273272, 45362851971589067, -59047377851748440, -13467906916962261, 27763262290880210, -52653306228290904, -10772987628514778, 29028488186569707, -33627420701741792, 46162668911900893, -36518669124612740, -70032586367095911, 48677926493027894, -64335247305201874, -52167461316243584, 10018325827491774, 1209787549407830, 19428780452063092, 58895287662842279, 52714181069717700, 51376789523414146, 39838746530145598, 40151650815036049, -35672264621565096, -2753396807092582, 22364489662437466, 25726836615335708, -16057318367063532, -21485079907427950, -29446821625877657, 46248216880742043, -37145888396109403, -58542996415727029, -17303103807689176, -39666175804512287, -30315547410174660, -59373152526870005, -40318724023369585, 40929137746619007, 44769230140753527, -63343825784615842, -20778300239322073, -46104888039257400, 71931387729728979, -45023595177944386, 52620927522876137, 29240284243378591, 35771990905248326, -58614511680820756, -60285596576516891, 25636640106376514, 17320099610194218, 61400943764126048, 63390054293295265, -18839263055960924, -70159448278623311, 40356895669108822, -20936376438363661, 55667839452358749, -29041826985738046, 53894386806633589, 48426709490184180, -46886840986426840, -2225502386203256, 54384761263538447, 7008217125431372, 44892226022933884, 375348372867458, -36769616937606363, 10930250945468942, 21448376946897791, -30906299672720727, -49010186910554977, 12957218933198715, 38746232650679928, -6783304831666683, 24140729333367297, 52523688684545772, -50785096982569085, 2880406890497550, -23378227775240440, 44237340907143405, -2737534836450813, -60551217385672459, -35201548866200720, -11283815521785959, 27096330360463234, -31588803600296296, 67833589298962929, -58551098099244876, -24243206072316629, 49660161647956304, -49714462149664533, -45850343232992323, 52593825554808822, -50940438450714789, -31357238341058795, -44818033200912110, 38102285285311841, 10186186725578273, 41467437429473260, -19262116534435889, 1729960973560529, 59552845327120944, -68953395560271875, -21604387171398705, 20138000996498037, -70516767012719203, 58001562749101612, -60618173569669439, -5233540054020120, -27941386074595992, -21283565267791376, 69849156383559400, -52905126282636468, 62411652722437899, -70054037065079575, -19834424189483501, 51870606182442008, -32933834634146825, -49317433628036501, -26965310273808173, -9987724782125480, -64576595097679402, 975891683155550, 35169160635718674, 30831567655075440, -70768968981388074, 18974164370666177, 13844923314569061, 52108205495498104, -57772704355067450, 59879992914933148, -10742685899854659, 58478136056288790, -58786951634052779, 16177574306992063, -52137657338486666, 3112598299923142, -490590355193208, -71657239411524989, -1755072264003374, 24213582325854617, 51649934978669575, 46081696152071090, -59006726032323177, 18463905606742600, -64413492164578131, 16503761568583304, 54615065977442066, 59588212458617571, -27268492557730459, -6166734977572168, 36521778986567712, -38558967771625922, -44093943150022158, -37348214204082820, 44160973365108232, -1413870555014654, -7730150817563312, 40879292723915035, -4826913294389688, -8614589430120522, -13921276494627353, -66778229711110663, -64373278065099514, -53556647487179823, 54859404305989234, 68115296639861288, 12622300819987921, 27956409055373958, -49644999883716899, 67505408135166003, -67186098998361290, -9184234785431512, 13622294819490923, 29629022504641090, -34723964826905460, -2411113917411933, 12493173042105373, 8768014345857097, -29391497820528670, -43144803056303392, -67458624236758665, -5187894827955483, -44385914298306906, -43106125161917199, -25651148044339736, 69030955087558162, -9047241622999923, -3893414786372278, -66615605740793557, 4945278666090529, 20502201929671888, 58920166355924460, -63347425887554442, 56881600393822614, -4939701060226342, -58378796059070040, 63895385775585272, -29409281174144180, -47429627388173074, -56131146612351448, 5721185643901589, -40129872020968130, 5053135791514053, -51508740197091020, 71998036101977893, 36342682634835461, -17908280727937929, -6770614868289082, -28772053267171162, -68773245857061550, 49440097493949133, -5763028196903423, -22025574541417222, 33163605795833710, 68361286591608552, 54118128037859587, -524865348679355, 13924709850292026, -17564183029578227, -43465122042647715, -5864373896235484, -8655242244008194, -16680445782396056, 57616656326998674, 2590530338041191, 5983680123192215, 63007884159641735, -13279971560495575, 22490021478105316, 1177380158895419, -69354524864414705, 20271538029970874, 28900211440987489, 26296489706529515, 34773337936176504, -36955273275655654, 5177762223011336, -62799724430941574, 52333507172541700, -24421070792680070, 34570682778276048, 44665357647885352, 961756179527229, -56687362688881445, -36479428178198199, 32742144544220534, -11001034430930785, 10397566660956696, -54615897672685193, 49900200248711902, 9453495569370401, 69625190265344083, 45323663845470830, 9373150506835301, -32321262896678438, 35078770063528436, -33550052939425237, 49133293665429253, 5243737286486520, -10335117104111110, 41112008277869814, -35981468372307698, 29480507011018901, -14889822934982235, -37163596430511383, -60823573807398239, -5639085816908920, -25999523023202430, -50338190316482254, 10839879631132602, 70211839444866115, -43289671716913708, 5239904709412906, 49362086951935204, 47361408142870508, 63948975646704898, 52260487389249757, 57540662688806220, -2000147844883684, -66235938135965427, 56570345419364912, -19922567866702784, 35283607551066148, -27641040430907754, -16638660178629742, -25958585754211037, -22937750265004302, 57912206148909283, 25394500790835426, 9147042883923634, 53054057111726102, 67832284850254832, -48877490479012753, -31907418597360080, -68805532206696578, -59424305084859246, 17985893950185551, -22831281162074809, 39458858589205365, -34746010875810797, -30632651610048782, -374966650985560, -23901885035605381, -7212392421976872, -36079394714002019, -16885805901515496, 6576403427683340, 14520375163522882, 30584690225374185, 16998455748467518, -56640588011562205, 23721976354842127, -40177569494498594, 58767817559541490, -54994537972510533, 46828313724509612, -4870478195903286, 13344673656735459, 65514540264984998, -18556779509689124, -54265867541857175, -6396605994624342, -65371306523309197, 53831377560691957, 914706338406913, 138542033111550, 63214121467825101, -52472646866538127, 53799988521048429, 1952714303457492, 60996766159916114, -8290324000697683, 9680701926173706, 31978790816412549, 37133555877642317, 34962462471072670, 28539324278381211, 56055669109488458, -23146243335677098, -68377981591117117, -1725349435797746, 63459683745177747, -36114650161074635, 12473035399664836, 60931338088294417, -16169886250504432, 34965643413558235, -35201953107081034, -58724829227377508, -20980154178876031, 43581392035327148, -7133103889369499, -13506690410074190, 16404702086005270, -26325011489240895, 44586375918141406, -7405450925016613, 48598506404460376, 59526748315651443, 41147600991955117, -62047736235913877, 69967345234468399, 18283970483221288, 39467495473892978, -39990422256560638, -68852298006294293, 60934035398156314, 58704051770048246, -63589366793022383, 21605809159070642, -63887348837674916, 29529859576426069, 19178390987360622, -23413448360571037, 36590068599243650, 37897318210524795, 5993165401096494, 29799880874416713, -34571566982280223, 62446535463205153, 4663175418337080, 33827183264514593, -6768536403179965, 41539613514161138, 2446305645153129, 7119132942531456, 48276484762646486, 15288215211183641, 23462762894948229, -36567723986525241, -24300488721027574, 56287458556094978, 70071794338548950, 64299741790620054, 46250522532829109, -59287977298820351, -28037535593149244, -44851702366696011, 47724804627706991, 43460693010161006, 41745050439044756, 69189299161181357, 20646969436688799, 9217650784123564, -21787969112869083, -63909326161712182, 62260175385377506, -15597328444227334, -1234045245600605, 30113567266780096, -13071116264903108, 28319593125340765, -5773234303385446, -61821700079306941, 11639866496754090, 64247489015149248, -52396491508136240, 68270707536750427, 68142425043810446, -2538374011172828, 457671204576721, 30922253652006736, -69557711567292955, -2210064249137995, -22348742591487912, 11519619622894160, 48871624829340616, 69592908960818636, 70528240483519026, -8908896051242780, -59612417805259089, 32001139453267559, 51674570911105027, 15488563569831832, -42583945213154665, 42540225741013732, -9351919331735832, -16540753795085139, -52663449244979386, -55909059300542192, -44178697878359093, 21131138485785761, 45716967958364717, -7519525193142125, -58839095928627617, 42962177948420695, -23197870309634000, 12571261055808041, 47621732860722882, -46455540266602661, 11719638276908527, -32887031304589398, -35760319012307154, -26266563510371259, 7204283118718134, 5317745096231762, -31168607896230702, 42503913626337930, -38328095955758585, -20180519035131225, -24717288460444867, 60478952394395925, -52533097607726002, -52563857501036775, -51888810655238726, 34534353372182578, 28436948143878029, -21335409389306081, -71079253863746427, -10114705165055523, 60084696766148655, 35472436407051307, -15766677148710969, -43100949802242572, -30622970440018873, -60233322671198846, 10408654388159349, 20273812028048845, -30431222195746907, -48698114895884292, -36758804221135686, 60600761113626316, -59789954863202248]
c= [[-38732166084541479, 67532485079828334, 62039575732238474, -71790100601656610, 50835928661346549, 26231033342083062, 59450678589336125, 4002671475435083, -63832065049789761, -33507924741136449, -16653184954255799, -11792562447863193, -70936626556928704, -27216877745763733, -38306076355674875, -56570573009351563, 48219134801368906, 68449487391031363, -12751675576338201, 12543373751639234, -18888491347784097, -53348963274918224, 2825523596088873, -5966081491816885, 47109232669275407, 33007557331045320, -14524152650459263, -56717659666082267, -71994128381964, 13912749413864521, -29921867333018887, 16837368085927397, 61289884558729968, -64684457313699106, 42747700854482418, 10856401439226507, 23928506320268924, 27378554290859526, -1862731065998301, 24508036634781516, -13946902812492985, 40495236952040637, 18376814982091164, 864167278427278, -17406845380142339, -28381550697631702, 53262624495786466, 43982581591905629, 34782467336172068, 192367121180132, 43324474239567819, -50534180175818616, 36790944624035813, 44593621485514303, -29391398870408085, -1283411396444367, -48487598332756594, 11429958264846368, -35933797786026210, -27272050679241410, -39894858476207886, 69612530569508266, -55959247913092925, -491747990553914, 52333728827003718, 69911071930416163, -71479398432435255, 42862543107010412, 30577333758333778, -31114502330693829, 1621493679105488, 5635524868717625, 51137307430460070, 68382916654817769, -65967149290115695, 33504795550559132, 43410935112519476, 30493133897381106, 17021398444599780, -4914139979629740, -65084215088427955, 13317569507466155, 13617828210659089, -5427940822950489, -40431272300355705, -17371347342250306, 53557917681454222, 28917361691027376, 26555295543034565, -4468695831095385, -11826679804122959, 35862122021583856, 47610975490333905, 29745746007680489, 3881479030312855, -61140871180240932, 16364124503646101, -49582903875525698, 48451863446215743, 3638466586950151, 44734027162089966, 6609520427269218, -52784873568582840, -69669546597508159, 51523822556233456, -67907127811000615, -20215653368650895, 278462299213853, 11770897148019028, 64823628182748446, -17115653078468562, 21182388856982706, 50088198096733713, 11247698436969067, 34435661730498611, 27744715349618127, -53648175240917391, 32336657051738312, 63566677994814214, -30475865742762817, 48615351992075578, 7701768401470418, -28861405086140979, 36440389829939840, -51233844408802290, -27051122359013668, -28255602969941427, 45661299315089709, -23045306392216792, 29947499446939481, 25183107892027561, -11701461851989989, 49170462387988064, 66857695912623089, -47764773411202262, -8872175941825261, -33006297920202019, -48117786021040750, 63647928685516901, 15884823705665994, 44131787723273982, 33976748789922012, -237905440926044, 63110612316215129, -21329152310095090, -63476539216076317, 11537881066333177, -26591643319883011, -42343143459104493, 10027318754935421, -40295562390104052, 50107384467998514, -66794481072103053, -60178434439910589, 42830476410737667, 49696481334614958, 30715999732359931, -68263860020364162, -25463620898996212, -11412271303405505, 44485135241434793, -37922899406969103, -14728247510002539, 69849260076211921, -51365807780786935, -8629512028234540, 58357288358451664, 45372195947669066, 17714679234522608, 28114286359173445, -18307100020092885, -63243633478710874, 18673070206200321, -22763964576793182, 50800993442954373, 65331553666499731, 38497094453948949, 39425848129953048, -33108211366561859, -48999949945709105, 50844488904156112, -38586182978414166, -10384194950642635, 29861943634760323, -32636124820016207, 17342394230406976, -66866267689956279, 51361063464708505, -13113420280707395, -19501354085284763, 40382745513504234, 11162078391887216, 60483533992530423, 7021198805650261, 71608175001872034, -43860270450207853, -5002646429935767, 37891726230434311, 20402542102495480, -34407616132838344, 66071870637581198, 27996769991444531, -49704816891541813, -34965132621604086, 66913870825041908, -18573777835481490, 6573623869321720, 24381408046346447, -54316178602157579, -54111129275962067, 36034046910300003, -64909487068011096, -67495651994962337, -21847412130516898, -16745171681303169, -6066539953706310, 13054518166314824, 48197624157064106, 25069216167204214, -69981792886932773, 64088136845454483, 8376232539069751, 40984353729034142, 42318812023031789, -46107904315566177, -17591087680893791, 63837431441257150, -51634078545413151, -64644602610612649, 59443759746378681, 39824598958287188, 40132478223081072, -6756449054846867, -15721866979278917, -52798897556740561, -69138272347674499, 10288497927470815, 25504193398838675, -28422696419831099, 24862175455440646, 3755131998392670, 4117151997155450, -38724940861737659, 41827443951302419, 3843469361884742, -67733853192987431, -53946294770045106, -70383577834275662, 57718362704948373, 27993073023014404, -70477960720194675, -15955144257381891, 23665689932363396, -32000432578917816, -389094926459900, -37868657999621917, 13627516248058737, -37665314922988129, -26378365002432125, -26834300199102569, -35488707605302230, 1548019567846487, -31328427190763470, 54345236487178485, 11515838204008675, -32803956568533666, 52304471825764326, 17370869638595487, -8820790723315452, -17485425223094868, -24435107269944134, 67788139816354317, -568266426154537, 47478288551329968, -60219061216154526, 40987257968862649, -38189305077879850, -69651342535791049, -58731626241691612, -18725725436285560, 41092086828072710, 14037416458451467, -13508747718867321, 59650791427788087, -33372343581446636, 11667533462333813, 35808619267284394, -42537972117620840, 50383076292158631, -56934977849990694, 8733870755926822, 62349670788952919, -39310456882235, -50060245304300827, -9818954297141911, -25647622673851391, -63549063707758102, -44452372280420215, 66128115129649248, -1292380465175906, 11888366489894012, -43989734928540940, 29685045374722739, 40122380094401659, -31009773721872337, 29410093339048949, 26279610151378696, -18115794434734633, 29808350764669397, 18263838059603703, 18263999173992791, -58745162998782113, -39527363616209829, 25482775239170082, 48514537250124527, 65271548577426015, -63825538264213809, -18779763416663697, 13168735956423930, 277159280423755, 67467309242932342, 9156772684198790, -8227703138259325, 48948990942997271, -52404501438659696, -9093369617006238, -3189630152559565, 67026816846523706, 57682086946341326, -4685853208935887, 16517835122726321, -44631630235250947, 28439468624755773, -37837406052647458, -11904838819410697, -67347843254220786, -68592802867969012, 39562636307045470, 65088383073657299, 14572787213005580, -40347981184246947, -13077971660835727, -6606237600920585, 25452320721523016, -42536593523451939, 711071461966814, 23255703153852882, -60559534612205511, 45297198025089788, -2452530210739620, 41585796764915601, -27121273021789956, 68746113206481468, -15003794262022536, 42448130139886631, -32608130951597998, 56484353426718923, -9411524182360003, -13199731212711779, 43982646223737074, -49338193110832862, -66277679086126035, 5212350945755471, 36926858389952451, 19587255034497178, -36942143941445533, -22872469354128319, 34356905278451863, -40419459669262591, 72019260685086759, -28004861891975180, -30820249570470236, -35446357180426334, 16747807260505433, -15863361633126906, -20909746752740130, 50341499889970459, -60063405763732618, -3232193774865713, -21343314578935401, 69650441453253232, -26152212473687100, -6775024512073940, 57289384213892703, -36945649087551849, -69268506211298503, 3022241681553079, 55768267909617436, -12973145210856377, -52624566493649616, 20133186933673597, -19245054922782102, 13691919000396802, -22219380870434606, -3473088196013890, -10169205735274702, -20293897015833412, 52097160349483722, 16697570757102292, -65526211569728090, -46148212046586520, 49667371503051763, -64177719710179100, -66455977469382633, -30175606927280060, -50482844948179529, 65640027320338174, -69870277481815882, 29127309077897868, -49068468645889543, 6206758198020106, 25289983542013384, -34858938433986148, -54724674353995822, -35069155915962958, -45764943109316986, 38057473674334560, -43095920888941575, 53401935238797352, -55786304443675833, -59825723489501244, 68986712827992266, -26381194033112302, -61039855370915135, -42577761120953405, -62971983947533652, 51183112561850708, -29335442543091341, 71508893449798804, -43516970085127943, 13279741378744079, -51864160725329965, -35234750423580310, 70013741278865963, 35830961303896743, -69007893913148717, 41614330123766935, -12296152978138622, -24785773500549565, 14647963401139913, 45610928181961604, -38054451085637138, -71048222918753698, 10430322831295467, 3436979783728358, 59939028683424668, -11432626329921956, 62252345888430953, -54834581411232693, -64141294393540228, -69109537660773917, 56934975756472103, 53664695760340021, 36051560448830049, -21702063507409139, 48473575061738312, 10641036113046311, -62560240794855153, 49566095436921710, -46761967196963661, 64722792141343697, -58077197720215824, -57332777694978819, 39397477136917982, 20380161059923407, -25856665711652831, 38132850980153733, -63237586426776881, -68666347149343420, 16121583625906826, -36440576984776053, -14514862527696826, -66897167535389438, -7136408813248298, -9110573469122, -59006219089978770, -45130760336097958, 51563592074444335, 52647453111394422, 67504051138770113, -70128908945839495, 53509179213107318, 41987541661116842, -9581615945518907, -12676688638117089, -6339729173881920, -35275168839664700, -44196425391360490, -4482151252016010, 11284279482903092, 64067350591520240, -201613133404049, 44679257043103165, -68300600671615538, -37862556319160879, 67717580712808680, -9963240715104468, 43547130547041634, -30877576927814208, -26265233147679620, 47205544101537906, 71280300799066320, -54141158074710863, -56756329805219219, 51434915425243137, 49920693476873587, 3820152989651514, 54675169479573209, 16770336530196562, -56466612164382283, 44769864123577117, -37726685447423207, 39792070138082903, -29363384352519665, 52624525971644487, -15545085841777281, 38378593824213791, -19919705797674271, -61367122975890977, 67483979069973852, -26223298326452941, -69072352113857143, 26719481960701994, -31747571654617625, -28842399423162225, 57797925109050726, -71337607654031760, 28975019108280315, -39955241356741122, -8870971103357019, 29818700209741402, -50039889452901699, 33027813712286925, -29363157717513623, -61328291429993332, 47953473177194178, -40351115830830757, 38437409813717206, -16781404537755882, 43584652420164037, -1047232096058895, -41694740113989206, 40642452105902098, 36627008592942211, 22823702805040519, -13442624603056023, 35320599910021546, 7166966110761394, -11173838534901413, -19794957275142894, -33591171087762908, 20467847369100836, -49577588286741454, 14216326980129613, 41192977838213958, -29498879792056900, -10691952710781416, -58987584171124257, -57811751458519363, 49984168624766316, -69643551947879856, -44657933059030678, 50273569525977224, -71476523088827151, 59750919348256834, -65232708120764464, -38605431841133979, -52699733006933185, 8117824531024092, 64632256572168817, 51751609216650182, -56778731570680401, -54653075808248425, -785820581654087, 17275287517441621, -23051143373306589, -61297482983944651, 71866292838379083, -30299283413976415, -61147613935728598, -53015752409630222, -6739064826110265, -48041668157153091, -25067121977192272, -63536334832128387, 53848812386683234, -32277851958159543, -36891409141053415, -24948256273054380, 60797080493386679, -26542914360005011, 33199010386927296, -23932814506861275, -54858810098413307, -62986488952783003, 60719538250058437, 6714077675114136, -15539599861121997, 46961581305366505, -23342810060217326, 42275941818973341, -69829684512605141, 16177546878922729, 28334316903029237, 39390404451323824, -58670814879588508, 43883348645135437, 68979639946794105, -8724318824835822, 16100354080982026, -39682142213349749, -9915962334614421, -8681174957429418, -7347388712080062, 57484666551895958, -12932803926441566, 35189863522453603, -31311530954355381, 54165232182453947, 7705335791491847, 704014794810538, 5998272156461438, -48650401287791328, 3274151152900117, -51996629828238323, -13121878232965587, 22026110894323701, -10775724692952730, 55551071439515420, -14497565372407738, 66593242055805925, -68873505926513542, -27774043465110611, 50674613255876690, 17756559299213724, -9296510958670460, -4489326289458907, 3282231285470454, -44990180357532217, -71011575848641277, 69750997660282333, 60375823813295914, -20977275880550374, 11359068260922114, -37476401705849701, 71321480205058659, -66914656673694609, 33771156802996501, -48718908332438739, -20142023663715812, -61091844496471753, -41189508275501691, -64810580347281684, 30276182097943004, -5567151697328059, 32364748163201878, -29486106318457389, 651701338661371, 58187822423685567, -32754284451783966, -11527533124145095, -8093964300516716, 34302689629713479, -53221184371411676, 71232817022070234, 32467299992159960, -68137029256821035, 16554358218064354, 7321357878596166, -20149788285528400, 50726156839824993, 47690047987757401, -58558781080191003, 69490356375638501, 3940208968430565, -19884351639306554, 52115318048162304, -17120234128506162, -45865602840175515, 25865873133910830, 42357147675338293, 2818715272065633, -808319336371357, 68053674905227009, 30491544965156981, -65923041112974479, -19342046369018649, -15979060110794046, -69673151145035088, 15449910391085339, -57427318152347644, -44883448665273482, 26842618489525508, -46203288035517448, 26001770465787100, -26947874604754084, -2538030792360264, -61192300352200002, -47664506372476396, -3150164037615089, 25730758716399408, 14079180457904641, 37319714954153414, -58956233121543279, 53339764949715274, -29924135835707589, -33703640249688883, -11367195693449279, 2653072236417249, -69598227114182773, 6523768225653227, 48844051688931575, -35048487004862585, 65021114690921827, 6653232357452956, 53826298766845914, 56561406732478020, -62914723572061500, 37645943733788383, 23467563822459903, -63563191731507596, 65887775070072398, 22413820157771655, 52603241179304482, 36491790793374899, 38072286410852572, -6150542479909528, -61745939955178330, 52984602917816879, 46332960438963683, -34174947656763830, -32575295200203811, 27844227828527443, -20532331029958703, -45907694851402384, -63982555346143065, 68951923246613704, 59832995827053386, 52638258483132493, -38999582255928204, 27680445145947340, 68732378087984640, -62843872192356043, 59345460921637111, 66657468989611251, -67439733699814411, -60136511600227185, 4267950205409427, 32081355951422882, -25119087245452344, -45789001611855030, -54470221955853943, -62922350034475635, 47130515638862321, 54606510881997244, -31297510284527261, 26043341734699663, 33851688274408803, -14098045306711318, -38024493464801628, 46834497123618172, 48799124834272039, -11541185075873272, -4400700974309331, 25524049541828867, 45320774714686427, 16553048845714842, 15660685292907935, -28562238878358267, -39007115486607245, -57499629843986231, 14037654119529465, 57099037710550315, 39989791462035421, -22177894824855593, 43280662716244497, 70408730565311945, -15222253885723466, -32622826963281362, -69693059346771614, -8516715791948624, 36659550988877015, -14417637942758890, -63461497042768992, 17473523699934014, -56487261217485684, 35883856515178995, 33179957046863635, 48056062113195741, -64348378576560111, 68468948349913473, -55815674447827604, 4793895956240005, 57182557070508825, 60973666810314244, -14152457437698959, 32942361294437692, -48995505591482203, -43767640564365804, -46138750601908114, -51662589051147279, -70881331091993396, -4819448834794503, -9202939835991339, -7737171545541261, -18276391286706165, -45830774265555458, 41012088833864886, 22327382100530450, 37140163066563665, -5543495116979305, -51533467636268532, 866452692154982, -60484370861990476, -49884926295136160, -55993222334187503, -24080676042886583, -9098532223461371, 34957055323412873, -8897241507032636, -14504290019782520, 3631950100519268, 18618762159829325, -52854906874847137, -33900443827839996, -24285119589398023, 25495976401452873, -55674190076257988, -20436073215723449, -14392477661308969, -43167606193117126, -54726358792755408, -1827395338750153, -5773626640635569, 69157649182799439, -40181021134783731, -38602998453864102, -53471825012042535, 36045178071880135, -27870461788479233, -10493520488981908, 1699113248996036, -67797781556504258, 11512472521020316, -34967807545271395, -32639159901408982, -63249813716339387, -4557403854393611, 37127754030538027, 62144768831097841, 50560037168997385, 2295415572698400, 47714244238444589, -4485091454591995, -52188892207621396, 38650571274845679, -29595619834689688, -11463622972618279, 36865754589875901, -54516403980071784, 26984506741330544, -64777449877313789, 42907471500370981, -5828639707318828, 68304315518290266, -45440961034633760, -61580699875360881, -63897018586637955, 54980438542519601, -47120112216568444, 56254865293059612, 37967070825945875, -43179957906694134, -21524420573474512, 50829607955057919, -36463803727640952, -35931881381952272, 49828207062305055, -35744733179884689, 58003689411188965, -16001272875069336, 51506786307421417, -12916648657367207, 37619691894528832, -20534992678628133, 35943075184841233, -43441726286810952, -43712641529449757, -45092740113648374, 18200628551213478, 25914284746976590, -62610013755111399, 3319951700611676, 1177058853219489, 5067537397164276, -41191118444602998, -70196764501346272, 63210025477714659, -46192371310291993, -51100168061692343, -51388970769851231, -57558685459903204, -15344028379090904, 52992086266016394, 33917911783540350, 21150800828963939, 20590337808960, 10696608134907941, -42847089620357607, 54245286022519665, -66295376824268538, -62958334072708778, 48520496165831112, -710494060120166, 10200924394153821, 37674136369951751, 48056509584745160, 43684995102635635, 43000268424559605, 8768250038245141, -63590897542508951, -37517340166201193, -14778818376949438, 35338163143220101, -69008407028630726, 55486588395740765, -47852607299886569, 32095465790187721, -70759191591613780, 56281115212409159, 68578078467618895, 42064645681797166, 13867851781141529, -67785196983534597, 54316803636773575, -16352491603488956, -23015531468978693, 1594185995865947, -39953035670130716, -8046132507461617, 18508107929216841, -20513295230940834, 39209977237184803, 43711340947866299, -48932071706141051, 67548379459678195, -19029546668612191, 10110227562370344, 60606747008864320, 64393303737157839, 46824823459398948, 31003321182791243, -39207008602661364, 43946291972248408, 46486031363494749, 51297386404598733, -65839791128962772, 2898269481418923, -11985717422468231, -19340694259703680, 16990975537693438, -38731455273040786, 24308405349381411, 70587605730459160, -32549342991328787, -13648422457875548, -11862273069557885, -55990063503347168, -51266793792989229, -2080097569887954, 11456964627373384, -24427888781389075, -44686701199549140, 39260976612079121, -67247006362579731, 16827911327465878, 37418797267952724, 35938123508378601, -69310057249329788, -65182670740260044, 65472888334340804, -33399960050921478, -22884716489375982, -53635336793042194, 9766618433889210, 46102605178437286, -31831524559700727, -35208041580226712, -25336616244098825, -38700065273173220, 3943016194602476, 25305742136533605, 21589574344522558, -67930983913590112, 7521000404533084, 14933807658462549, 47847778445973570, 27962644208257367, -19729478266887894, -60838508383736874, 44258003828818220, -13618947193183103, -11242742036841522, 30110441450911680, 69678440746855423, -16133590309963693, -25048686409762546, -38956192505012154, -1659596581107282, -39984133528250645, 63192688479835870, -52487007243821009, 28795974974398283, -24442212961255745, -69013372944068028, 24896674298703183, 53939537751718506, 46362589420373985, -28722398768720808, 30157441702498970, -57085141619418051, 1902737140871689, 33306273835679945, 70414806132151352], [-31873369760783040, -40106108146887953, 30747378154165755, 67658475612076467, 7124164847037017, -66481918002200664, -46835632198362456, -48055332341220496, 20276466031629995, -7027683560270544, -53728220919410851, -57209270468942876, -2338445613487279, -38187470352713837, -35757918746131714, -36249420909873760, -61916341576049690, 5871431455066668, -28893671009266687, 62751854749749873, -53186861360119455, 65147143934482762, 6157596703018767, 14548609400154559, -2448296733498143, -4850917921955579, 52923871844339445, 11704965795951044, -49486228664070322, 16863196642210758, 33825331456235162, 25347921077607180, -58780066836870903, -65757657446365263, -62457328689414658, -8997780210660493, -39688004919627828, -65427531821583731, -71038556721411726, -4246215775839651, 71996558198644385, -71361679651268109, 6477662296282148, 48049860694435733, 26613998449774581, -62775844712833310, -12654927163332398, -35313086890317723, 37653210320740712, 14284159665715412, -17519264192622046, 58185497878597519, 56534655284012250, 2428162900347674, 26691193292508246, -11395531711197523, -5635781949583743, 37262792143822930, -13490860760986777, -20708612984010981, 18737645903887713, -58622071747847120, -51734445488141350, -10982369672636961, -4368842248372003, -31199917303859638, -13735415356640691, 51474446600927548, -10696016305249623, 36424997816106164, 68291655346248464, 45004764715465088, -49241728604928075, -12690258591439529, 38337056551623803, 38245506322872858, 64262310276581828, -17922194533690015, 15059835895534742, -29266488670736470, 25950342745764368, 20196571983943033, 33673094789771447, -2994488343430393, 40101093005250229, -35756867937164234, 23575546908123356, -68400186557862681, 12254267177620156, 17869636424450197, 66784542664444455, 31903489468699579, 60840669943586816, -61233645592429504, -27054808324391050, -10805257651932061, -53095419586936462, 14127489401887167, 46589649816387705, 47995588721136938, -55628671996672752, 22969434395949438, 59715729940986281, 3488066283622495, -48943657327686936, 19216338883844112, -14376715314744427, 41449013369373856, 38359699680606852, 9176714376216901, 29804884945825996, -21062066830160889, -53754583882418892, -9241206396862411, 15459429463358688, -14169757289424183, 61988060099339637, 1059129230379099, 32116749464963250, 64953932022187967, 20382150274199189, 45546837449272258, 44388961252104751, -65915325663362322, -22998370771318116, 43211026962643916, 37847981093084436, 15508841694690298, -26207619119131433, 7788623439388980, 52065007948446906, 27973363878286190, 37170847231028403, -10074591154668206, 46246175135358373, -10109463694819900, -42675009409813160, -21077786495901909, -26099643704260610, -24164524186544804, -57831717702956257, 12632146742460380, -54431159130457033, -24988217857680058, 171342947140070, -65497461651259968, -13826906820171798, 37417171530801939, -58541088365547943, 40955908727632554, 20919673144368581, 2079085093593023, 25195689354229414, -52508016676729688, 60542640380498515, -2342201924491554, -20199692429580616, -64201378654544793, 23123174080681077, 48280179349830344, 9931091531007962, -39668657318326796, -39612481148938527, -64726235188916442, -30308771063606883, 19103870320410340, 34527934488825, -20134371824490794, -65501531459300120, -15537325732477486, 30106443262717294, -26825062261534061, 12694014865498792, -71231483732810425, 6821641634930185, 55433472484533875, 68123927077513202, 9750137173106721, -29257529345957828, -34439805754579473, -7233854474914635, 48832713659228642, 43304657126793994, -34623288489640059, -14897350182800740, 1523961719387597, 10466573508258027, -61640828094708324, -58068186425268132, -42819103898821175, -67600915604323895, 25716663537226766, 58316392259438888, -55794380040604485, -59474525506872816, 70080227396856080, 3854651237263188, 50852389589873224, -65038161257133174, -18062351205271758, 18586227034401285, -23746953262589667, 63687486774653994, 32664063041941189, 6502572270171894, -33215900281596134, 16111995936994735, 22970621870373092, -51296158836278773, -47321670349359435, -57101215435649608, -21469105439855813, -64974851403707373, -36179487114155449, 17501528308815593, -46919704419132177, 59566226547772852, -19750325704739975, -49431901741613047, 30417744349283439, 55471035675566587, -29706234193777361, 374741320514327, 8793589314408414, 34255855607609814, -40603261450513080, -31807487675800937, -60406434842342608, 24932625558512083, 16648129056626394, 17791166372461886, 14248284130786700, 31627470338102975, -38273552248110319, -69171744743825023, -47575913486280818, 21030417044582268, -68987681128617900, 32879755174260896, -48389268283167528, 17405224131072336, 71471707239958490, 24581342579431104, 69519529941991638, -18124994804387340, -26817948727908890, -30760823720683922, -56622154519283821, 58215609670357847, 690910392106350, -10643339675525491, 39797194609449713, 56500756373646566, -59579738876183516, 34155862676531805, 31618793156313559, 9047309020794467, -44666658372747820, 62872555718637935, 38857992263376580, 18359166162781022, -58735247124359802, 20727228366372199, -7319641093669673, -62301686616984051, 54776525742144648, 49708798157799289, -10706882290274013, -48471647312536265, -42920838518362102, -56442701485569698, 66173974791275057, -26101406422851951, -20762188937744829, -58594593896102534, -37876482462090228, 3241702329641590, 56207421633639410, -50665145136385774, 37630773656325849, 70662820969797959, -35599173116481046, 1304717476626296, -48327551956796899, -16441289531132162, 56730124808962676, -14677356821855377, -26896495561027408, -55238988964278666, -2933591402787798, -28830644939559551, 60495624297019558, -34841305486886103, -1210056893053934, -23928318129349588, 68444598444542551, 59967205666794816, -16133400596430394, 58130017614374394, -7507613097040853, 57755756382894188, 60390075579620492, 63680702527591207, -33099445708937258, -65973921946557530, -3415025202270463, 4601730373433214, 6654963490760096, -7891921602338197, 6611590106798297, 55799215831097668, -9306114571769856, -57419509402455136, 6761957464764721, -68195436240346463, 17723951422121054, -16714899862963361, 29319204919838964, -25810012911796620, 17444417553151305, -42132682939177187, -11063497077430515, -65470204628713453, -70382151189639434, 17582444964765583, 11741478983182876, -39496536421624053, -19802942463897232, -55690545044574607, -18770564054713688, 54110417345769715, -42255034637550812, 34660127695618189, 49041009693899234, 23417039426986370, 29765614154982276, -48052743686799969, -10209242510225871, -55862573602329211, 58772879392317620, -32948979902580782, 54935545088066230, 3693448268208164, 45030869991177731, -56679420732851525, -54927944024290691, -58082216735525714, -46954080764767122, -39371106429682628, 32695251087224848, 47398698987824343, -2449236151906574, 446761066893779, 8619196714751256, -71644798525990473, 32063342625042640, -63513735047210561, -32361289859008604, -69534346672721632, -16930300767828543, 16997011792060785, -59648022295901716, -24591433558365303, 6680506325202615, 14153420103874764, 42102699790605845, 23237452384945951, 46400874027586206, -40466683681250610, -35322669172436528, 43590383630282985, -56241406975142178, -41269321003933742, 42111625601085021, 10268131959949016, 69820775359495441, 10778436994552941, 3484165346353426, 44953571386979415, 11113941466857114, -27331200016263072, 64950672109046460, -60396723848744679, -20868580261714196, -15608997461141507, 7572411624610909, 52230859530738982, -1525068069263398, -2570778234057278, 41468655963127423, -38230675795171914, -43406878460915866, -67782650967582587, 10218424419679710, 55939141410425240, -33099646556277175, -60518628641702421, 12569178264339155, -71932815221097445, -24899994181522354, 55027992577947240, 38209883875379749, 20723387497819238, -11658776889970106, 44949299106742267, 71624881757919090, 22227725613284118, 20447316782666700, -56536218020703146, 16377739889480848, -3610806724842458, 59590761439445790, 62477782552369915, 29659096402254576, 54159454490321873, 8630201114420292, -68879312420689349, 54977290796703237, 57160743694375964, -34695288599519576, -51229734009121852, 33665080475090444, -19951222782837105, 22190048206703076, -17302351246513991, -2945058925792131, -71216290933264934, 12197043772110935, -35602073889596793, 19773572082053044, 31444104793225251, 39371568676032536, 40319849491513034, 35095061104513340, -7891335955438242, 47843185294218603, -41147951209413932, -60290411309073398, 50792421622742165, 28641754390663901, 63693183047323179, 20089934650834893, 54588931527577331, -24020037796551687, 42585580148492825, 47730001601590246, 45872130445994435, 11908319284694258, -30304035356568404, -55300965178834714, 48210235275277881, 40137350291942871, 50098036488205179, -40012322558158742, -53478832953517119, 37201688560430917, -57785377006941432, -48909166798062685, 25128303764271964, 51429177793787247, -18667010222057391, 11281323043338533, -9722876596275330, 35414980571819642, 56757298974163092, 12874716779938784, 549495140285786, 63809955036998957, 61441562261778742, 57440216452987336, 17869574778951790, -21360064997283511, 23211914372854108, 60642314257987246, -61709766785290227, -20359405544786783, 36786245207818113, 13766231380740429, 37019881411807297, -16331371222704778, 620697748580278, -65763855299596270, -52885610162454014, 39339590985800559, 59803670768340908, 18988193922157098, -67126117819964085, 21663408784064841, 68323304113233110, -63149541178401561, 3633764658677533, -14839337929055349, 65483536097752794, 47120793518795903, -50264912872409425, -2014840477948024, -26005652991583283, -39258398559612506, -30755110977051515, 47504260064046438, -43233531755488459, 62152483831368652, -35167917732053615, -67140812339226948, -63290409119783072, -1607639185895431, 55304835876773415, -17429952343312826, 57751018719869271, 64973083069999412, -30273868658951126, 480425116582593, -9685612918701247, -38906705778536329, 30099064654406186, -15493103401640504, 3386178925533287, -39415347579024102, -2163523915148428, 15579170482476593, 30461967072865174, 16207920372767027, -13935446399097769, -70086807777717487, 42116575298814935, -44400327660474936, 10608690719541844, 59001906933148598, -45264346119102097, -37945269284665111, 14676240126814815, -34062681590844180, -21350106187636542, -40161068638728350, -67031859847339756, -44979677489553727, 47315606441572009, -35428828152750728, 62531724161790212, 37855021077590760, 12142719988643662, 2901894289208393, -30893051122793308, 3347837432741730, -63045403608387738, -71593104006186985, 42370885854804007, -25263393581779973, 49564170884180982, 67145916807927587, -23861872439806907, -24642464889167157, -27898109440897528, -24015189912902841, -62824517708536287, 60877353984238505, -64413413083983387, -58077943296287954, 34106604254229172, 26667337492492697, -40354131674434318, 53400336770016456, -3208915499923569, -13122762595058872, -45503029040449260, 31754263799826425, 6684714794770296, 3892284261790566, -17394254844753926, -19332562952687067, -70759327530697718, -40800028960100318, -43401140956791320, 71493887515926434, -7801780231294670, -18118061052869474, 7819177743745734, -7954489857391677, 59394614370038389, 25192444699614402, 68734193479449448, 26805710469987628, -26059015630892487, 53746701247574760, -63602727132550251, 40848774145747709, -8988368968194606, 21779122348748668, 21240696121154049, 53052247691792982, 13767484783702664, 66759683611852581, 68833146306721988, -35350991042851604, 27026000482515169, -20176895966719983, -44421780727473723, 65391106077657120, 25886163419920760, 5608821959577565, 34948157526226563, -45305871784181903, 24958310605091494, -66780227189235258, 44795631680335287, 20023015708245397, -25398039663762608, -68641368193243140, -60979468910934834, -55548088200880995, 27112236528052069, -70430834118678082, -33026576467656335, 34855787957589218, -45347683703053627, 43445549977018187, 64971455750535698, -24697099111477750, -33101050006590231, -6335047833064352, 56337906658955741, 19032081426273308, -39296256030287513, 3902607939112510, 35730025594715275, 44295502505289203, 58633904679349319, 67688878578600437, 1341249173742832, 17076843453853368, -50179858296430305, 66140816522443820, 4599888981354694, 53498325153774453, -67499241592210037, 50326833654995560, -57543472964186583, -24140454982812128, -17512063829941554, 35923098476575312, 63894334276463978, -43397730942611480, -28304353850372327, -4027853171106938, -13063764609299831, 45510026423563276, 59216438120757207, -60857890755839650, 33034949193683563, -28486441805110838, -19606121243858083, -39496444327132123, 13468069642236852, -52692382893078094, -34538773263576401, -35496582457842402, 5727595286718793, -57673442026873020, 12864749059070034, 41658501607876663, -15822552691812004, -48617329547057411, -17619345807178563, -39768902945319453, -67740933279599929, -42383688560055182, -11383740891225060, -6168433627734001, -47547628921003143, -64138735907379137, 61047226651534537, -48892308987241445, -55849910985349557, 12697851999989290, -16947843295627077, 5381846024600275, 29113718082196154, -63902312960977803, -7606793157922771, -4593526658927651, 52646450871638011, 21061026242184546, 71653541131985211, -28711805171554486, 53950595288838906, -46950660792898225, -28253566278720930, 23560027613306456, -21829376598414866, 18178320068816905, -7880148045762192, -42541052082593409, -5538709921552513, -29250351216931573, 68501570439761804, 45506105302381527, 54761089037619393, 62362751961282304, 7068526684742508, 27703893609718997, 4793746193407575, 1741350000447728, 33420233071335526, -28706694200860388, -7216822228069778, 2820062848195777, 42021410411430689, -59605664362525341, -35166244435824707, 7577654840622634, 57391306870486937, 66319747518574888, 71450136022913321, 36322768296865224, -32275317405653243, -21068263200663672, -5402269643094558, -3876405454807108, -29769851275590747, -21739038083715852, -16143354083867581, -27739058335755299, -69812907604077919, -51280066698215319, 38365994461221397, 15260080499992193, 22866358745067615, -32506689865686927, -51809927631239812, -16397360456021701, 9774958979493918, 6849648874897846, -56134571821052852, 68793849076325838, -1942944181747433, -63152504892093414, -26248830843569893, -28593259555700484, -34960533988090982, -29168908342438571, -9103548173118316, 468846672640374, -70535029398124195, 12279623349493807, -13229884939668210, 62994077738451583, -20639144467046339, 41634642612959732, 42462001451850437, 13984105083698529, 42152097627050435, -57477069458045704, -52876136495922948, 44160091436676352, 64091464606998506, -802209179647980, -49448783965014694, -30181055548568263, -18477076744366848, -66250587913773545, 71213399831090568, -24097426076479932, 54003703098912661, 51533633734945265, 66835328311694637, -53764098520755517, -27904910207081649, -24045250699715984, -3527748189792629, -2128187028044555, 42745754930388714, 22755398551278928, -27374108609812014, 17893859948174732, -44956166846025791, 4018525699573627, 29246926160766912, 58620440190674577, -65396655950379918, -39701113916806600, -36169488261780366, -27949116213326756, -45559340225442057, 43303173356839099, 32221806483853398, -56638103604554234, 58750416687905019, 30669357180267257, -41215954808073202, 22246552810790955, 49975472634689575, -67863234280410451, -53782605144602153, -58245430376389526, -39463720102013828, -60304897304566064, -24044456461982631, 51243512212244107, 22789098268476643, -19824199630174979, -40598492594727923, 31961291299747146, -17818303028141461, 7720763329743317, -21347338946037983, -12177120379571268, 911055429435163, -46200944346083080, -19991058053337837, 3822949825745154, 55047503083922998, 9776451252539123, -26063614724532746, 35686583875749500, 2728091544946881, 70557165136649124, 46089067776487392, -46883476310587745, -26511829197232881, 4926453838264232, 14667891462232299, -30066875578274398, 23970397822920642, -44253263255961957, 39621734027823093, 18682038343549704, -5272261812465382, -63096013091554495, 54628450403179688, -40263602669867305, -22387252809553717, 15672734329415209, 71652798372906018, -17124030059506845, -10219696463544681, 32725117865198396, 61612666049918128, -37662555909255922, 54139293191529550, -36609170771309448, -15957140743244580, -29256933064319287, -34310101055017565, 10621919824011076, 38275273682830962, -23285554736244472, 22009513054839955, 24560466953143623, 20382645471390911, -50067761154358586, -50955287051150964, -37828640889747121, -60580595093709465, 13910145060369123, 58801227518716817, -34412895681045347, 34060029890503679, 26333322121954069, 2890721963826946, 6750043300888244, 2707623063417497, 49656714284859623, 2014574871714134, -14053894805748024, 57553283392773957, 17160432673685698, -46914013307951994, 28714214790432993, -50544437692278449, -26365822069758938, -33230414988073472, 12204595324016376, -8814609655607065, 20497155639901308, 20209021079291569, 41423289720009729, -59600646864764128, 56109690917518341, 20067866217605112, 18442354118918895, -71978232853843023, 19069889858498171, -11403894180024404, -52521911042845155, -36480270270094501, 33186563239069686, 60971437321095247, -8188242676157802, 28986059763519025, -7494139645269084, -71307329487928452, -4348044342051301, 33131785123712406, 58589048793771037, 46806876870558513, -12932083068829588, -52118596412746634, 20883424794847446, -61389783518432720, 23558341027161689, 69583542781804204, 31792208026578859, 39343150725051275, 60494598519866683, 4498077712568812, -47034216364068017, 15666101159424185, 70962801822740868, 46624752185361737, 39050493169791517, -54429314123793048, 30788025511916015, 25721040365030731, -49899712929499499, 61777325444948528, 9571452435012812, 16410429858508695, -35249783563634311, -10388446037936156, -66561224884674317, 70919654039295311, 53537461528695863, 12804025660883716, 15745123586080052, 61608227408410558, -52286909746724754, 69816941650645500, -43659485859577290, 36404044317125481, 54526886394436448, 9708309973144377, -31203099836697947, 7810625666815187, 59204777255341649, 56426570445542802, -62018475910393939, 35009704669461958, 17223819222295360, 41628394999008137, 52857480192460313, 11359096783274775, -42553080670452027, 48937836038960787, 24820637750063185, 2776806856155404, 59797526820684142, 54221456948321237, -38226761721527876, -54043295180331186, -67854079085432016, -6192415502738399, -44595592043506965, -68414334856527882, -53118810970216102, -65157183591350281, 12124616290585992, -11448948947642858, 32093069470384599, -54092819705486493, 42903881364416564, 71401392658840983, 53800704757423219, 56756611747545517, -41682760580193542, -29023310280032976, 61241466238348237, 11996275386824010, -39306113929486730, -16179373224865878, -44183309299963954, 25200846533939701, 41300391475822287, 3373483506755461, -60994705598209676, -36790163346483631, -10304381468894969, -20225228341967997, -26553883158782650, -23148483980832279, 53872642979631443, 18829809753613057, 3479465239782484, 11547595348542463, -37702585972458122, 43441426604454509, 11824976936359403, -63695716774391736, -38269502368944265, -25285491195650089, 63153587403234548, -532625668666153, 7688105666608241, -57786583305439931, 69057053972544725, -7492412252379829, 45878305782403921, 71113540648381519, -56482777402953231, -3430190142947741, 19465165819227686, 33700941006968024, 10137364407267414, -11662015604033170, -2870637484256125, 63067166534690730, -2884355608489186, -53390257143552947, -59106447167210341, -37749393354025925, -45249077235451164, 42327025863570897, -68106795791007986, 36225772152882137, 51391316975269425, -8719774087221274, 39231537003744386, -56877473906859238, 4443270865806023, -31683330133709453, 23756059486670606, -57535185681009063, -37591047837129045, -47950439872418668, -10061883344988268]]

In [ ]:
# BFV1方式テスト
import math
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k

t = 10
delta= math.floor(p/t)
a_poly = random_poly(k, p)
m = [random.randint(0, t-1) for i in range(n)]
s = [random.randint(0,10) for i in range(n)]
e = [random.randint(0,10) for i in range(n)]
e1 = [random.randint(0,10) for i in range(n)]
e2 = [random.randint(0,10) for i in range(n)]
u = [random.randint(0,10) for i in range(n)]

pk0, pk1 = BFV_keygen(p, n, k, s, e, N, Ninv, w_list, w_inv_list, a_poly)
c = BFV_enc(delta, m, [pk0, pk1], e1, e2, u, p, n, N, Ninv, w_list, w_inv_list)
mcand = BFV_dec(s, c, t, p, k, n, N, Ninv, w_list, w_inv_list)

print("delta=", delta)
print("a_poly", a_poly)
print("m=", m)
print("s=", s)
print("e=", e)
print("e1=", e1)
print("e2=", e2)
print("u=", u)
print("pk0=", pk0)
print("pk1=", pk1)
print("c=", c)
print(m==mcand)

delta= 14411518807606068
a_poly [-48217947215535356, 48836427320084643, 57503991283778789, 20542077431857536, -26557549017205586, -40975845127638592, -22150585584112916, -38941243566963244, 39471777546962453, -8865871128255102, -14853625879533248, -2247366400277496, -32086333672407034, -35448881884894637, 35311157939133845, -60202026035221414, 7456681156288417, 67990225831537674, -8335432561552449, 20951817954627931, 25049061903314647, 34737111101725229, -48522670653903881, -70414736332595844, 52839101107845140, 29394141802701329, 19281736455765393, 55459811869351208, -16801956006360147, -5911310069771067, 2118059643742679, -14617968533846833, 17034344223694486, -19548038229067659, -59662064392657123, -31219605772392185, 51525299078959240, -36559022343071528, 20605603796942996, 9316926009540150, -51411756707628230, -2576585975173800, -63767677759076543, 4598800617678648, 50117352376711438, -9422689628468768, 54438935465839641, -12247751289137985, -71522927621743836, -34373273836733981,

In [ ]:
# BFV1方式テスト（テストデータ）加法準同型

m1 = [random.randint(0, t-1) for i in range(n)]
m2 = [random.randint(0, t-1) for i in range(n)]
c1 = BFV_enc(delta, m1, [pk0, pk1], e1, e2, u, p, n, N, Ninv, w_list, w_inv_list)
c2 = BFV_enc(delta, m2, [pk0, pk1], e1, e2, u, p, n, N, Ninv, w_list, w_inv_list)
c = BFV_add(c1, c2, n, k, p)
mcand = BFV_dec(s, c, t, p, k, n, N, Ninv, w_list, w_inv_list)
m = [poly_add(m1, m2, k, p)[i] % t for i in range(n)]
print(m==mcand)

NameError: name 'BFV_add' is not defined

In [107]:
# 演習5
import sympy as sp
p=144115188076060673
k = 10
kN = 11
wN=65170666404517193
N = 1<<kN
w_inv = sp.mod_inverse(wN, p)
w_inv_list = [pow(w_inv, i, p) for i in range(N)]
Ninv = sp.mod_inverse(N, p)
w_list = [pow(wN, i, p) for i in range(N)]
n = 1<<k
t = 101
e_bound = 10 # エラー多項式の係数の最大値

s1 = random_poly(k, p)
a = random_poly(k, p)
r = [random.randint(0,1) for i in range(n)]
te = [t*random.randint(0, e_bound) for i in range(n)] # t*e
s, A = BGV_keygen(n, t, N, Ninv, p, k, s1, a, te, w_list, w_inv_list)
m1 = [0]*n # 平文多項式
m2 = [0]*n

prod = 0
for i in range(100):
    m1[i] = random.randint(0,1)
    m2[i] = random.randint(0,1)
    prod += m1[i] * m2[i]

print("m1 =",m1)
print("m2 =",m2)
print("len(m1) =",len(m1))
print("len(m2) =",len(m2))
print("inner_prod =",prod)

m2_rev = m2
for i in range(100):
    m2_rev[i] = m2[99-i]

c1 = BGV_enc(A,m1,n,t,k,N,Ninv,p,r,w_list,w_inv_list)
c2 = BGV_enc(A,m2_rev,n,t,k,N,Ninv,p,r,w_list,w_inv_list)

c_prod = BGV_mult(c1, c2, k, q, N, Ninv, w_list, w_inv_list)
c_dec = BGV_dec(s, c_prod, n, k, t, p, N, Ninv, w_list, w_inv_list)

print("c_dec[99] =", c_dec[99])
print("c_dec =", c_dec)

# print("s=", s)
# print("A", A)
# print("m=",m)
# print("s1=", s1)
# print("a=", a)
# print("r=", r)
# print("te=", te)
# c = BGV_enc(A, m, n, t, k, N, Ninv, p, r, w_list, w_inv_list)
# print("c=", c)
# print("type(c)=", type(c))
# mcand = BGV_dec(s, c, n, k, t, p, N, Ninv, w_list, w_inv_list)
# print(m==mcand)

m1 = [1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0